# Prediction of Housing Prices for the City of Melbourne

Contents of the demo:
* **Uploading CSV** with spatial data (housing sales) to HANA Cloud
* Training a **regression model** for the housing prices **without using spatial features**
* Retrieve & Upload OpenStreetMap **Points of Interest** for the respective area
* **Compute spatial features** by using database functions
* Train the same **regression model including spatial features**
* **Compare** both models
* **Predict the price of a reference house** for different locations across the city

This Jupyter notebook is almost self-contained. You only need the following to get started:
* Vanilla instance of HANA Cloud or HANA2 SPS05 (i.e. HANA Express)
* Python dependencies installed
* CSV of dataset downloaded

## Preparation

In [ ]:
# dealing with datasets in Python
import pandas as pd
import geopandas as gpd

# db connectivity
import sqlalchemy

# load osm data
import osmnx as ox

# visualization of spatial data
import folium
from folium import plugins
from shapely import wkt

# Python ML
import xgboost as xgb
from sklearn.metrics import median_absolute_error
import shap

#### Setup Database Connection

For HANA Cloud we need to used an encrypted connection

In [ ]:
hdb_host = 'xxx'
hdb_port = 443
hdb_user = 'xxx'
hdb_password = 'xxx'

connection_string = 'hana://%s:%s@%s:%s/?encrypt=true&sslvalidatecertificate=false' % (hdb_user, hdb_password, hdb_host, hdb_port)

Connect to DB. We configure SqlMagic to automatically return Pandas DataFrames instead of SQL ResultSets.

In [ ]:
%reload_ext sql
%config SqlMagic.displaylimit = 100
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False
%sql $connection_string

**Clean up demo**

In [ ]:
%sql DROP TABLE MELB_HOUSING
%sql DROP TABLE MELB_OSM_POI
%sql DROP TABLE MELB_AREA_OF_INTEREST
%sql DROP VIEW MELB_HOUSING_CLOSEST_POI 
%sql DROP VIEW MELB_HOUSING_COUNT_POI
%sql DROP VIEW MELB_HOUSING_PREDICT_VIEW
%sql DROP VIEW MELB_HOUSING_PREDICT_GEO_VIEW
%sql DROP VIEW MELB_OSM_POI_VIEW
%sql DROP VIEW MELB_REGION_PROXY

#### Determine SRS ID to be used for the dataset

GDA94 / MGA zone 55 http://epsg.io/28355

In [ ]:
 srid = 28355

Install the SRS in case it is not yet existing on the instance

In [ ]:
df_srs_exists = %sql SELECT * FROM ST_SPATIAL_REFERENCE_SYSTEMS WHERE SRS_ID = $srid

if len(df_srs_exists) == 0:
    %sql CREATE PREDEFINED SPATIAL REFERENCE SYSTEM IDENTIFIED BY $srid
    print('SRS ' + str(srid) + ' installed')
else:
    print('SRS ' + str(srid) + ' already existed')

#### Helper functions to quickly generate maps

Convert pandas dataframe with wkt column to geojson

In [ ]:
def df2geojson(df_input, geom_col_name):
    df_temp = df_input.copy()
    df_temp[geom_col_name] = df_temp[geom_col_name].apply(wkt.loads)
    gdf_result = gpd.GeoDataFrame(df_temp, geometry = geom_col_name)
    return gdf_result.to_json()

Bring pandas dataframe with wkt column on a Folium map.  
https://python-visualization.github.io/folium/

In [ ]:
def df2map(df_input, geom_col_name, location, zoom_start,style_function=None,tiles='OpenStreetMap'):
    m = folium.Map(location=location,zoom_start=zoom_start,tiles=tiles)    
    tooltip_fields = list(df_input.loc[:, df_input.columns != geom_col_name].columns)
    folium.GeoJson(
        df2geojson(df_input[df_input[geom_col_name].notnull()], geom_col_name), 
        tooltip=folium.features.GeoJsonTooltip(fields=tooltip_fields,labels=True, toLocaleString=True),
        style_function=style_function
    ).add_to(m)
    return m

Generate a heatmap with Folium's heatmap plugin

In [ ]:
def df2heatmap(df_input, lat_col_name, lon_col_name, location, zoom_start):
    m = folium.Map(location=location, zoom_start=zoom_start)
    arr_input = df_input[[lat_col_name,lon_col_name]].to_numpy()
    m.add_child(plugins.HeatMap(arr_input, radius=15))
    return m

#### Helper functions for handling ML models

Train a regression model with xgboost

In [ ]:
def train_model(x_train, y_train):
    model = xgb.XGBRegressor(objective='reg:squarederror')
    model.fit(x_train, y_train)
    return model

Determine MAE based on sample prediction

In [ ]:
def mae_predict(model, x, y):
    y_pred = model.predict(x)
    mae = median_absolute_error(y, y_pred)
    return mae

Visually explain the model with shap

In [ ]:
def explain_model(model, x):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x)
    shap.summary_plot(shap_values, x)

## Upload CSV File with Housing Data

Download data from https://www.kaggle.com/anthonypino/melbourne-housing-market/data?select=Melbourne_housing_FULL.csv

In [ ]:
df_import = pd.read_csv('../data/Melbourne_housing_FULL.csv')
df_import.head(3)

Add ID field, format Date and adjust column naming (upper case and no spaces)

In [ ]:
df_import['ID'] = df_import.index
df_import.columns = df_import.columns.str.strip().str.replace(' ', '_').str.upper()
df_import['DATE'] = pd.to_datetime(df_import['DATE'])

As a proof point, drop all fields with a spatial reference - except for lat and lon.  
Later on, we would like to show how to generate even more insight by only using lat/lon and advanced spatial data processing.

In [ ]:
del df_import['SUBURB']
del df_import['ADDRESS']
del df_import['DISTANCE']
del df_import['POSTCODE']
del df_import['COUNCILAREA']
del df_import['REGIONNAME']
del df_import['PROPERTYCOUNT']

This is the data that we are going to upload to HANA

In [ ]:
df_import.head(3)

Create table and do the actual upload

In [ ]:
%%time
hdb_connection = sqlalchemy.create_engine(connection_string).connect()
obj_cols = df_import.select_dtypes(include=[object]).columns.values.tolist()
df_import.to_sql(name = 'melb_housing', con = hdb_connection, if_exists = 'replace', chunksize = 500, dtype={c: sqlalchemy.types.String(512) for c in obj_cols})

Generate geometries from LONGTITUDE and LATTITUDE (sic!)

In [ ]:
%sql ALTER TABLE MELB_HOUSING ADD (SHAPE ST_GEOMETRY($srid))
%sql UPDATE MELB_HOUSING SET SHAPE = ST_GEOMFROMTEXT('Point('||LONGTITUDE||' '||LATTITUDE||')', 4326).ST_TRANSFORM($srid)

Create view which filters out houses without location information

In [ ]:
%%sql 
CREATE OR REPLACE VIEW MELB_HOUSING_PREDICT_VIEW AS
(
    SELECT 
        CAST(ID AS INTEGER) ID,
        CAST(TYPE AS NVARCHAR(1)) TYPE,
        SHAPE,
        CAST(ROOMS AS INTEGER) ROOMS,
        CAST(BEDROOM2 AS INTEGER) BEDROOM2,
        CAST(BATHROOM AS INTEGER) BATHROOM,
        CAST(CAR AS INTEGER) CAR,
        CAST(LANDSIZE AS INTEGER) LANDSIZE,
        CAST(BUILDINGAREA AS INTEGER) BUILDINGAREA,
        CAST(YEARBUILT AS INTEGER) YEARBUILT,
        CAST(PRICE AS DOUBLE) PRICE,
        YEAR(DATE) AS YEAR,
        ST_GEOMFROMTEXT('POINT EMPTY').ST_ASWKT() AS SHAPE_CELL_WKT -- a hack that will save hassle later..
    FROM MELB_HOUSING 
    WHERE PRICE IS NOT NULL AND SHAPE IS NOT NULL
)

Check records per year

In [ ]:
%%sql
SELECT year, count(*) as count
FROM MELB_HOUSING_PREDICT_VIEW 
GROUP BY year

We will use 2016 and 2017 for training and 2018 as a test dataset

## Train a Regression Model for Housing Prices (without spatial features)

Retrieve data for training and test

In [ ]:
%sql df_housing << SELECT * FROM MELB_HOUSING_PREDICT_VIEW

Generate numerical features for xgboost. Note that there are **only non-geo-spatial features** describing the house itself, but not the location.

In [ ]:
x_housing = pd.get_dummies(df_housing[[
    'rooms',
    'type',
    'bedroom2',
    'bathroom',
    'car',
    'landsize',
    'buildingarea',
    'yearbuilt'
]])
y_housing = df_housing[['price']]

Use the data of 2016 and 2017 to predict housing prices in 2018. In our example we are going to ignore the influence of time on the housing price, since we want to focus on the spatial aspect.

In [ ]:
x_train = x_housing[df_housing["year"] <= 2017]
y_train = y_housing[df_housing["year"] <= 2017]
x_test = x_housing[df_housing["year"] == 2018]
y_test = y_housing[df_housing["year"] == 2018]

Train the regression model

In [ ]:
model = train_model(x_train, y_train)

Calculate the mean absolute error on training and test dataset

In [ ]:
mae_train_nogeo = mae_predict(model,x_train,y_train)
print(str(round(mae_train_nogeo,2)) + ' AUD - mae on training data')
mae_test_nogeo = mae_predict(model,x_test,y_test)
print(str(round(mae_test_nogeo,2)) + ' AUD - mae on test data')

Visually explain the model  
* red color corresponds to a high feature value
* blue color corresponds to a low feature value
* the x axis depicts the negative or positive influence on the price

Example *yearbuilt*: According to the data, old houses tend to be expensive. This could be related to the fact, that old house are typically located in long-established and thus central regions. New houses are often located in development areas in the suburbs.

In [ ]:
explain_model(model, x_train)

Possible values for field Type:
* br - bedroom(s)
* h - house,cottage,villa, semi,terrace
* u - unit, duplex
* t - townhouse
* dev site - development site
* o res - other residential

## Retrieve and Upload OSM POI Data

To understand the context of a location, we need to incorporate additional data. OSM Point of Interest data is suitable, since this data helps to understand the surrounding of a house. How far is the next school? Is it a nightlife area? etc.

We use ST_ConvexHullAggr to determine the relevant area to load POIs

In [ ]:
df_poi_shape = %sql SELECT ST_CONVEXHULLAGGR(SHAPE).ST_TRANSFORM(4326).ST_ASWKT() AS SHAPE FROM MELB_HOUSING_PREDICT_VIEW
covered_area = df_poi_shape[df_poi_shape.columns[0]].apply(wkt.loads).iloc[0]

Visualize the area that is covered by our housing data set

In [ ]:
df2map(df_poi_shape, 'shape', location=[-37.817301, 144.9665769], zoom_start=9)

Re-usable function for downloading POI data and getting the datatypes straight for upload to HANA

In [ ]:
def get_pois_df(area, tag_filter, field_list):
    gdf_tmp = ox.pois_from_polygon(area, tag_filter)
    df_tmp = pd.DataFrame(gdf_tmp)
    df_tmp = df_tmp[field_list]
    df_tmp["geometry"] = df_tmp["geometry"].astype("str")
    df_tmp = df_tmp.infer_objects()
    return df_tmp

Download the POIs (i.e. Amenities, Shops and Airports) within this area using OSMnx  
https://github.com/gboeing/osmnx

In [ ]:
%%time
df_poi = get_pois_df(
    covered_area,
    {'amenity':True,'shop':True,'aeroway':True},
    ['osmid','geometry', 'amenity','shop','aeroway','name']
)
df_poi.shape

Check the data that we are going to upload

In [ ]:
df_poi.head(3)

Do the actual upload of POIs to HANA

In [ ]:
%%time
hdb_connection = sqlalchemy.create_engine(connection_string).connect()
obj_cols = df_poi.select_dtypes(include=[object]).columns.values.tolist()
obj_cols.remove('geometry')
df_poi.to_sql(name = 'melb_osm_poi', con = hdb_connection, if_exists = 'replace', chunksize = 100, dtype={c:sqlalchemy.types.String(512) for c in obj_cols})

In [ ]:
%sql ALTER TABLE MELB_OSM_POI ADD (SHAPE ST_GEOMETRY($srid))
%sql UPDATE MELB_OSM_POI SET SHAPE = ST_GEOMFROMTEXT(GEOMETRY, 4326).ST_TRANSFORM($srid)
%sql ALTER TABLE MELB_OSM_POI DROP (GEOMETRY)

## Computation of Geo-spatial Features

We would like to prepare a few views, that we can later use to compute geo-spatial features for model training and application on the fly.

Regions and more specifically neighbourhoods are important to housing prices. One way to bring this into our model would be to include the name of the neighbourhood. The more data-driven way, which is independent of local names, is to introduce a cluster as a proxy for the regions. In this example we will devide the area into 15 clusters, which are the Voronoi cells of the k-Means centroids.

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_REGION_PROXY AS
(
    SELECT
        ST_ClusterID() AS CLUSTER_ID,
        'REGION-' || ST_ClusterID() AS CLUSTER_NAME,
        ST_ClusterCell() AS CLUSTER_CELL
    FROM MELB_HOUSING_PREDICT_VIEW
    GROUP CLUSTER BY SHAPE USING KMEANS CLUSTERS 30
)

Visualizing the cluster that have been identified based on the listings

In [ ]:
df_regions = %sql SELECT cluster_id, cluster_cell.st_transform(4326).st_aswkt() as shape FROM MELB_REGION_PROXY
df2map(df_regions, 'shape', location=[-37.817301, 144.9665769], zoom_start=9)

Create a view that adds a few analytical fields to the osm data. First and foremost, we will make sure that each poi is represented by a point (*SHAPE_PT*) rather than a polygon. This is required, because a few of them are represented by polygons.

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_OSM_POI_VIEW AS 
(
    SELECT *
    FROM 
    (
        SELECT 
            OSMID, 
            AMENITY,
            NAME,
            SHAPE,
            CASE WHEN SHAPE.ST_GEOMETRYTYPE() = 'ST_Point' THEN SHAPE ELSE SHAPE.ST_CENTROID() END AS SHAPE_PT,
            CASE
                WHEN AMENITY in ('bar','restaurant','cafe','fast_food','pub','bbq','bar','theatre','cinema','nightclub') THEN 'entertainment'
                WHEN AMENITY in ('pharmacy','clinic','doctors','dentist','hospital','veterinary') THEN 'health'
                WHEN AMENITY in ('school','library','college','university') THEN 'education'
                WHEN AMENITY in ('kindergarten','childcare') THEN 'childcare'
                WHEN AMENITY in ('community_centre','community_hall') THEN 'community'
                WHEN AMENITY in ('fire_station','police','hospital') THEN 'emergency'
                WHEN SHOP in ('convenience','supermarket') THEN 'grocery'
                WHEN SHOP in ('clothes','mall','furniture','jewelry','electronics','books') THEN 'shopping'
                WHEN AEROWAY in ('aerodrome','helipad') THEN 'airport'
                ELSE 'other'
            END AS POI_TYPE
        FROM MELB_OSM_POI
    )
    WHERE POI_TYPE <> 'other'
)

Create a heatmap of entertainment POIs to check that everything is working.  
(*Try to zoom in to see CBD, Carlton and Fitzroy*)

In [ ]:
%%sql df_entertainment << 
SELECT 
    SHAPE_PT.ST_TRANSFORM(4326).ST_X() as lon,
    SHAPE_PT.ST_TRANSFORM(4326).ST_Y() as lat
FROM MELB_OSM_POI_VIEW 
WHERE POI_TYPE = 'entertainment' AND SHAPE_PT IS NOT NULL

In [ ]:
df2heatmap(df_entertainment, 'lat', 'lon', location=[-37.817301, 144.9665769], zoom_start=10)

Create a parametrized view to determine the closest amenity of a certain type to the individual houses. We use Voronoi tesselation to boost performance and readability of the statement.

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_HOUSING_CLOSEST_POI (IN POI_TYPE VARCHAR(100)) AS
(
    SELECT A.ID, A.SHAPE.ST_DISTANCE(B.SHAPE_PT) AS DIST_POI
    FROM MELB_HOUSING_PREDICT_VIEW A
    LEFT JOIN
    (
        SELECT OSMID, SHAPE_PT, ST_VoronoiCell(SHAPE_PT, 10.0) OVER () AS VCELL
        FROM MELB_OSM_POI_VIEW
        WHERE POI_TYPE = \:POI_TYPE
    ) B ON A.SHAPE.ST_INTERSECTS(B.VCELL) = 1
)

Create a parametrized view to determine the number of POIs of a given type and within a certain range

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_HOUSING_COUNT_POI(IN POI_TYPE VARCHAR(100), IN DISTANCE_METER INTEGER) AS
(
    SELECT A.ID, COUNT(*) AS CNT_POI
    FROM MELB_HOUSING_PREDICT_VIEW A
    JOIN MELB_OSM_POI_VIEW B ON A.SHAPE.ST_WITHINDISTANCE(B.SHAPE,\:DISTANCE_METER) = 1
    WHERE \:POI_TYPE IS NULL OR B.POI_TYPE = \:POI_TYPE
    GROUP BY A.ID
)

Instead of calculating the distance to the center of the city (i.e. the town hall), we calculate the distance to the central business district (as a polygon). This way the distance for all houses in the polygon will be 0 and for others the value will better reflect the centrality of the housing as opposed to calculating the distance to an arbitrary point in cbd.

Approximate polygon for CBD and Bay (can e.g. be drawn with https://arthur-e.github.io/Wicket/sandbox-gmaps3.html)

In [ ]:
cbd_poly = "POLYGON((144.9560499704408 -37.80598074335628,144.9714136636781 -37.80770998156641,144.97497563724988 -37.81547410106952,144.95999818211072 -37.81964401162185,144.955363324933 -37.80947308471637,144.9560499704408 -37.80598074335628))"
bay_poly = "MULTILINESTRING ((16139350.388793945 -4566440.82434082, 16139319.78704834 -4566519.42565918, 16139230.063537598 -4566483.518127441, 16139244.39038086 -4566449.176818848, 16139318.774047852 -4566443.307495117, 16139350.388793945 -4566440.82434082), (16113009.737121582 -4626977.554443359, 16113013.633300781 -4626978.7607421875, 16113015.069274902 -4626980.563110352, 16113017.663024902 -4626983.117614746, 16113017.930236816 -4627006.321411133, 16113017.974731445 -4627009.670715332, 16113023.585266113 -4627009.684875488, 16113022.060180664 -4627016.965332031, 16113020.501708984 -4627023.294921875, 16113019.010009766 -4627030.22064209, 16113014.757629395 -4627039.615722656, 16113009.603515625 -4627056.433166504, 16113006.853942871 -4627063.344665527, 16113000.653442383 -4627070.511657715, 16112960.73425293 -4627106.601989746, 16112948.667236328 -4627116.607421875, 16112933.472106934 -4627123.973083496, 16112923.119384766 -4627127.166320801, 16112910.651611328 -4627129.351867676, 16112894.610473633 -4627131.069152832, 16112859.522583008 -4627250.7091674805, 16112879.259521484 -4627289.184448242, 16112897.41571045 -4627336.955871582, 16112919.390197754 -4627369.442321777, 16112969.07208252 -4627434.415405273, 16113011.11743164 -4627493.655090332, 16113112.395935059 -4627596.83581543, 16113293.579528809 -4627742.440124512, 16113395.815368652 -4627820.785400391, 16113499.965881348 -4627888.614196777, 16113634.684692383 -4627967.92578125, 16113774.179199219 -4628035.755493164, 16113907.951782227 -4628093.096740723, 16114095.213439941 -4628163.794433594, 16114168.060913086 -4628190.818908691, 16114239.494628906 -4628217.304138184, 16114333.125488281 -4628245.96105957, 16114419.119750977 -4628260.296630859, 16114442.051574707 -4628267.932800293, 16114510.847045898 -4628321.443054199, 16114539.511779785 -4628378.771728516, 16114531.864135742 -4628446.604187012, 16114581.546020508 -4628522.087646484, 16114679.963623047 -4628598.522705078, 16114763.72039795 -4628644.88079834, 16114819.647277832 -4628675.824157715, 16114876.231018066 -4628707.136657715, 16114884.746948242 -4628711.849182129, 16114999.40600586 -4628750.074462891, 16115047.173217773 -4628807.405456543, 16115008.957214355 -4628922.054321289, 16114906.409729004 -4629014.149108887, 16114896.847351074 -4629083.902038574, 16114893.986450195 -4629198.554016113, 16114913.411682129 -4629323.356750488, 16115066.286743164 -4629581.327880859, 16115191.55456543 -4629716.384521484, 16115352.923339844 -4629829.751708984, 16115506.399475098 -4629937.356567383, 16115679.111694336 -4630035.8203125, 16115942.460205078 -4630087.622009277, 16116017.22241211 -4630169.34967041, 16116227.44921875 -4630302.29876709, 16116528.145446777 -4630422.131896973, 16116814.781982422 -4630498.566894531, 16117158.74810791 -4630584.556945801, 16117559.976928711 -4630734.263916016, 16118276.640686035 -4630909.414611816, 16119346.754943848 -4631205.6173706055, 16120187.562194824 -4631387.151000977, 16120302.221252441 -4631415.816772461, 16120541.090637207 -4631559.132507324, 16120779.948852539 -4631645.131408691, 16121095.261352539 -4631673.783630371, 16121358.966064453 -4631642.362731934, 16121452.98651123 -4631641.879943848, 16121555.734375 -4631657.4697265625, 16121637.899291992 -4631688.819702148, 16121785.464416504 -4631698.5314331055, 16121792.622253418 -4631669.382141113, 16121798.266174316 -4631670.702636719, 16121793.523986816 -4631704.168212891, 16121818.960449219 -4631737.875305176, 16121831.138793945 -4631781.862182617, 16121938.717956543 -4631807.064575195, 16122229.929748535 -4631839.721313477, 16122480.231628418 -4631846.91998291, 16122497.21899414 -4631837.690917969, 16122662.606384277 -4631838.954589844, 16122845.871643066 -4631824.500427246, 16123024.606201172 -4631825.877685547, 16123218.235351562 -4631815.086791992, 16123363.529541016 -4631817.926452637, 16123565.552124023 -4631815.810913086, 16123638.366210938 -4631830.236633301, 16123942.535583496 -4631817.102966309, 16124869.337158203 -4631626.020446777, 16125671.928405762 -4631425.372009277, 16126321.633483887 -4631205.6173706055, 16126837.5881958 -4631004.96295166, 16127125.104187012 -4630887.181640625, 16127527.368286133 -4630722.409362793, 16127630.628234863 -4630680.102233887, 16127824.65814209 -4630612.425415039, 16128452.32196045 -4630393.4689331055, 16129111.589355469 -4630154.5997924805, 16130067.055664062 -4629801.076171875, 16130630.77758789 -4629571.774353027, 16130984.294921875 -4629409.3369140625, 16131786.886169434 -4629227.809814453, 16132255.062561035 -4629094.03717041, 16132904.778747559 -4628826.511169434, 16133573.59741211 -4628492.09576416, 16134366.637451172 -4628138.572570801, 16134595.944458008 -4627985.709899902, 16135150.11517334 -4627737.2880859375, 16135570.524353027 -4627527.078735352, 16136182.013427734 -4627183.111694336, 16136997.183776855 -4626686.128295898, 16137104.885375977 -4626619.896850586, 16137189.365783691 -4626563.500427246, 16137289.608947754 -4626497.723937988, 16137402.041625977 -4626421.418029785, 16137430.895690918 -4626405.453063965, 16137467.319396973 -4626388.835266113, 16137505.947265625 -4626365.434265137, 16137575.93383789 -4626327.686157227, 16137631.170532227 -4626295.132080078, 16137722.508178711 -4626238.453613281, 16137808.235351562 -4626188.090393066, 16137922.994628906 -4626114.880371094, 16138017.48260498 -4626060.019287109, 16138135.559204102 -4625978.153930664, 16138289.291381836 -4625871.739929199, 16138349.014282227 -4625830.034545898, 16138407.534973145 -4625788.286804199, 16138621.646850586 -4625639.41809082, 16138906.535705566 -4625427.236694336, 16139014.081481934 -4625331.896240234, 16139352.336853027 -4625032.003173828, 16139667.014831543 -4624689.751281738, 16139934.66027832 -4624390.6005859375, 16140110.289001465 -4624147.699523926, 16140263.375610352 -4623913.756652832, 16140343.781677246 -4623760.0771484375, 16140346.631408691 -4623754.629150391, 16140349.069335938 -4623749.975708008, 16140351.874572754 -4623744.612854004, 16140354.590759277 -4623738.597351074, 16140450.05834961 -4623526.66796875, 16140593.727294922 -4623160.973083496, 16140613.018981934 -4623103.701904297, 16140625.742797852 -4623057.92199707, 16140635.57232666 -4623012.511230469, 16140641.65032959 -4622980.847229004, 16140646.02520752 -4622948.771850586, 16140649.231201172 -4622912.0861206055, 16140650.644958496 -4622879.1740112305, 16140649.420471191 -4622851.96484375, 16140647.338745117 -4622830.8416137695, 16140528.49407959 -4622807.888427734, 16140521.347351074 -4622806.441467285, 16140518.542114258 -4622804.1716918945, 16140517.206298828 -4622800.497497559, 16140518.04119873 -4622796.723999023, 16140520.612670898 -4622793.262573242, 16140526.802001953 -4622792.184448242, 16140570.940185547 -4622801.135864258, 16140710.468078613 -4622829.735107422, 16140711.558959961 -4622823.521606445, 16140714.787231445 -4622784.651611328, 16140715.3772583 -4622779.133239746, 16140531.08782959 -4622743.058044434, 16140519.443786621 -4622743.1005859375, 16140509.836914062 -4622746.90246582, 16140501.343261719 -4622749.455932617, 16140493.88482666 -4622749.427612305, 16140486.99420166 -4622745.881103516, 16140484.600830078 -4622740.334350586, 16140486.203796387 -4622735.09967041, 16140491.580566406 -4622731.2127075195, 16140501.098327637 -4622726.786682129, 16140513.56616211 -4622724.3466796875, 16140525.187866211 -4622723.665771484, 16140601.497375488 -4622741.823852539, 16140634.31439209 -4622749.257324219, 16140642.90826416 -4622729.921813965, 16140651.123657227 -4622703.635131836, 16140657.268493652 -4622670.666931152, 16140660.062561035 -4622641.373046875, 16140660.307495117 -4622597.43939209, 16140656.032836914 -4622567.450561523, 16140645.847106934 -4622510.381286621, 16140640.971313477 -4622505.288574219, 16140491.157531738 -4622444.020690918, 16140330.523498535 -4622219.379516602, 16140325.647705078 -4622173.87310791, 16140280.140319824 -4622159.418395996, 16140263.42010498 -4622135.473693848, 16140232.729309082 -4622018.7869262695, 16140202.984741211 -4621975.38079834, 16140154.304748535 -4621951.167053223, 16140056.521728516 -4621943.1525268555, 16139896.099182129 -4621840.411560059, 16139781.38446045 -4621813.488830566, 16139642.491149902 -4621723.373474121, 16139538.696838379 -4621694.706359863, 16139478.361694336 -4621650.989562988, 16139392.367370605 -4621526.776306152, 16139399.202392578 -4621385.316650391, 16139459.24810791 -4621316.580383301, 16139554.804748535 -4621259.248535156, 16139631.236694336 -4621106.374572754, 16139726.782226562 -4620915.278198242, 16139772.790588379 -4620835.92175293, 16139827.359375 -4620792.719177246, 16139860.688476562 -4620667.764587402, 16139917.873291016 -4620619.088012695, 16140022.981140137 -4620456.664306641, 16140061.197143555 -4620332.451293945, 16140137.629089355 -4620208.239807129, 16140252.288146973 -4620131.796203613, 16140347.8336792 -4620007.587097168, 16140443.379211426 -4619787.835205078, 16140510.404663086 -4619708.473205566, 16140517.094970703 -4619578.539855957, 16140577.140686035 -4619510.751647949, 16140710.913330078 -4619376.991821289, 16140806.458862305 -4619252.777648926, 16140902.004394531 -4619138.123046875, 16141121.760192871 -4618880.144226074, 16141226.856933594 -4618784.594970703, 16141299.615356445 -4618667.0244140625, 16141313.95324707 -4618520.839477539, 16141383.694946289 -4618453.001525879, 16141444.842712402 -4618388.028381348, 16141406.62677002 -4618174.961303711, 16141437.206237793 -4618124.325561523, 16141435.291503906 -4618026.854980469, 16141472.561279297 -4617946.599060059, 16141465.726257324 -4617857.793640137, 16141503.942260742 -4617790.910095215, 16141533.709106445 -4617740.219482422, 16141572.882385254 -4617745.962097168, 16141657.919372559 -4617688.621704102, 16141710.462158203 -4617751.6904296875, 16141825.12121582 -4617762.19720459, 16141851.748840332 -4617753.916564941, 16141885.790344238 -4617750.782958984, 16141925.475769043 -4617752.768066406, 16141948.051330566 -4617760.835998535, 16142000.326965332 -4617754.356140137, 16142066.940551758 -4617730.59185791, 16142135.613586426 -4617685.204589844, 16142217.522460938 -4617620.321350098, 16142352.37487793 -4617477.398193359, 16142403.737670898 -4617433.20300293, 16142446.15045166 -4617396.792114258, 16142501.765625 -4617348.414611816, 16142573.600097656 -4617270.063903809, 16142711.280029297 -4617132.235656738, 16142765.882263184 -4617074.78527832, 16142897.139099121 -4616944.247131348, 16142945.017578125 -4616877.100097656, 16143005.185791016 -4616809.372253418, 16143204.202758789 -4616620.935974121, 16143332.598632812 -4616442.38470459, 16143491.306884766 -4616262.177856445, 16143715.381835938 -4615987.655944824, 16143759.698120117 -4615916.24798584, 16143758.97454834 -4615871.492431641, 16143769.494262695 -4615844.585327148, 16143862.000732422 -4615715.962402344, 16143908.020263672 -4615644.939086914, 16143933.957702637 -4615606.280517578, 16143960.173400879 -4615575.929260254, 16144027.488342285 -4615499.193969727, 16144042.215881348 -4615473.2517700195, 16144054.928588867 -4615444.644592285, 16144084.227844238 -4615405.136230469, 16144132.2399292 -4615348.418457031, 16144172.726867676 -4615292.636657715, 16144192.641906738 -4615246.848999023, 16144192.140991211 -4615234.799621582, 16144202.315551758 -4615182.448913574, 16144206.044799805 -4615157.769165039, 16144215.89654541 -4615139.454284668, 16144235.19934082 -4615107.942077637, 16144249.782226562 -4615074.445373535, 16144253.322143555 -4615048.844482422, 16144244.06036377 -4615020.862182617, 16144235.19934082 -4615005.893005371, 16144220.026489258 -4614980.4765625, 16144220.816894531 -4614959.397827148, 16144229.087890625 -4614947.788269043, 16144241.500061035 -4614943.450622559, 16144269.474609375 -4614941.480224609, 16144287.363647461 -4614936.6748046875, 16144295.82397461 -4614927.375854492, 16144323.820800781 -4614877.167114258, 16144361.513549805 -4614790.032531738, 16144384.74597168 -4614688.08581543, 16144390.078186035 -4614627.970031738, 16144388.386108398 -4614526.691101074, 16144378.022277832 -4614502.976867676, 16144380.382263184 -4614481.303771973, 16144404.694396973 -4614443.429138184, 16144433.737670898 -4614392.329772949, 16144449.890136719 -4614344.547546387, 16144454.810424805 -4614305.326843262, 16144446.439208984 -4614280.663391113, 16144437.611572266 -4614255.546386719, 16144442.14227295 -4614236.765380859, 16144471.987060547 -4614189.281494141, 16144479.367553711 -4614148.33215332, 16144481.78314209 -4614101.628295898, 16144470.795898438 -4614082.011291504, 16144450.112792969 -4614067.822998047, 16144434.550292969 -4614059.743774414, 16144418.208618164 -4614053.450500488, 16144410.126831055 -4614043.004211426, 16144411.306762695 -4614030.5877075195, 16144430.420349121 -4614020.552490234, 16144442.832458496 -4614003.798828125, 16144451.225952148 -4613946.309204102, 16144447.229614258 -4613924.949157715, 16144437.311035156 -4613904.099365234, 16144426.69116211 -4613885.843444824, 16144381.361816406 -4613810.453063965, 16144375.383972168 -4613797.583312988, 16144371.9664917 -4613783.523010254, 16144385.102172852 -4613755.104736328, 16144398.660888672 -4613610.165893555, 16144456.324401855 -4613434.005432129, 16144513.653930664 -4613252.47644043, 16144509.011901855 -4613101.450317383, 16144445.960571289 -4612992.533203125, 16144398.994873047 -4612927.608703613, 16144402.957824707 -4612775.637817383, 16144415.381103516 -4612747.930969238, 16144408.690795898 -4612699.2068481445, 16144380.983337402 -4612675.3125, 16144271.11102295 -4612665.760437012, 16144217.45501709 -4612612.303222656, 16144207.903808594 -4612554.977233887, 16144246.119812012 -4612507.203491211, 16144303.44934082 -4612411.656677246, 16144303.594055176 -4612326.569458008, 16144237.670654297 -4612097.259765625, 16144179.383789062 -4611932.915588379, 16144179.940368652 -4611889.608581543, 16144211.844543457 -4611800.756225586, 16144246.565063477 -4611758.399230957, 16144283.088989258 -4611725.7353515625, 16144280.083374023 -4611689.160400391, 16144272.65838623 -4611639.066650391, 16144234.219726562 -4611577.509094238, 16144207.358337402 -4611566.852722168, 16144170.500488281 -4611542.436767578, 16144171.313110352 -4611517.113891602, 16144172.58215332 -4611477.365478516, 16144181.788269043 -4611469.245727539, 16144194.178100586 -4611470.436096191, 16144206.18951416 -4611474.729736328, 16144219.848388672 -4611482.948669434, 16144255.782348633 -4611504.572937012, 16144303.22668457 -4611526.990783691, 16144351.149719238 -4611530.278381348, 16144377.643798828 -4611526.211364746, 16144390.55682373 -4611524.213317871, 16144391.959472656 -4611523.759887695, 16144422.104797363 -4611513.854614258, 16144437.878723145 -4611508.668212891, 16144466.354248047 -4611482.452697754, 16144483.419555664 -4611470.832824707, 16144573.900024414 -4611401.921813965, 16144645.166748047 -4611328.561767578, 16144791.307006836 -4611153.885681152, 16144838.484191895 -4611072.096252441, 16144864.37713623 -4611005.8943481445, 16144864.521850586 -4610962.60546875, 16144865.345581055 -4610916.75201416, 16144864.488464355 -4610869.05670166, 16144886.429504395 -4610748.147399902, 16144849.159729004 -4610551.320373535, 16144875.909790039 -4610487.303894043, 16144885.472167969 -4610339.2088012695, 16144924.500793457 -4610300.088378906, 16145007.76776123 -4610263.731018066, 16145069.872924805 -4610139.513305664, 16145058.262268066 -4610070.78125, 16145010.495056152 -4609984.792358398, 16144953.165527344 -4609831.915649414, 16144953.165527344 -4609669.477722168, 16144956.694396973 -4609587.997619629, 16144937.246826172 -4609509.025817871, 16145017.719726562 -4609488.525024414, 16145075.962097168 -4609436.061828613, 16145079.680114746 -4609423.438354492, 16145095.554321289 -4609423.013305664, 16145101.8772583 -4609430.564697266, 16145104.827209473 -4609438.8670043945, 16145109.680725098 -4609447.013427734, 16145096.422607422 -4609459.877746582, 16145064.473876953 -4609482.475341797, 16145053.553466797 -4609494.376342773, 16145062.49243164 -4609508.331604004, 16145079.813720703 -4609496.57232666, 16145096.489379883 -4609516.6056518555, 16145104.448730469 -4609530.461791992, 16145088.830627441 -4609541.753540039, 16145085.257263184 -4609547.902404785, 16145085.546691895 -4609556.332336426, 16145096.422607422 -4609574.5380859375, 16145115.870117188 -4609611.629760742, 16145133.469726562 -4609603.667358398, 16145139.280578613 -4609613.83996582, 16145149.444091797 -4609630.685668945, 16145131.777648926 -4609641.424987793, 16145142.286254883 -4609666.2615356445, 16145149.933898926 -4609679.636169434, 16145128.872253418 -4609690.97064209, 16145119.55480957 -4609724.889099121, 16145115.747680664 -4609744.271118164, 16145118.397033691 -4609761.811279297, 16145142.286254883 -4609780.924133301, 16145192.03491211 -4609803.253234863, 16145287.580444336 -4609822.352050781, 16145411.79071045 -4609831.915649414, 16145583.768188477 -4609803.253234863, 16145698.427246094 -4609765.02746582, 16145798.269714355 -4609683.730773926, 16145827.557861328 -4609659.871765137, 16145888.705688477 -4609592.035461426, 16145925.58581543 -4609574.495544434, 16145958.580932617 -4609574.042175293, 16145965.950256348 -4609573.942993164, 16146005.067932129 -4609577.35748291, 16146029.10180664 -4609588.2951049805, 16146040.634521484 -4609616.333496094, 16146047.436157227 -4609641.325866699, 16146062.130310059 -4609642.119262695, 16146067.262145996 -4609642.204284668, 16146072.215881348 -4609639.044799805, 16146083.180847168 -4609631.875793457, 16146102.383422852 -4609625.826049805, 16146115.61932373 -4609625.259338379, 16146139.541870117 -4609635.446105957, 16146172.893188477 -4609655.097167969, 16146193.776733398 -4609658.979187012, 16146230.801574707 -4609650.081665039, 16146340.885437012 -4609591.086181641, 16146543.553710938 -4609449.8611450195, 16146844.10522461 -4609167.076904297, 16146883.890808105 -4609100.433837891, 16146893.853881836 -4609083.7306518555, 16146952.14074707 -4609019.709106445, 16146990.356750488 -4608935.6416015625, 16147055.178100586 -4608866.889160156, 16147109.791442871 -4608756.005126953, 16147186.223388672 -4608709.183959961, 16147231.129699707 -4608622.2431640625, 16147341.825805664 -4608608.926513672, 16147446.922485352 -4608522.9357299805, 16147542.468017578 -4608408.272949219, 16147630.388183594 -4608312.353271484, 16147647.5647583 -4608293.611450195, 16147781.337402344 -4608178.965393066, 16147895.985351562 -4608073.868408203, 16148037.53918457 -4607994.497436523, 16148170.955627441 -4607905.87689209, 16148205.38671875 -4607884.954284668, 16148300.987915039 -4607733.284362793, 16148403.479736328 -4607567.409057617, 16148467.499572754 -4607453.705993652, 16148526.599121094 -4607424.143676758, 16148581.134521484 -4607388.249694824, 16148661.395874023 -4607243.981079102, 16148711.077758789 -4607172.322021484, 16148764.589050293 -4607041.42767334, 16148852.27545166 -4606870.308532715, 16148868.76184082 -4606800.578430176, 16148864.431518555 -4606786.272705078, 16148867.348083496 -4606779.218994141, 16148882.309448242 -4606779.6013793945, 16148899.508300781 -4606777.108520508, 16148929.731506348 -4606755.706665039, 16149028.071166992 -4606598.401977539, 16149093.593811035 -4606481.2111206055, 16149141.227416992 -4606385.946777344, 16149193.480773926 -4606262.115722656, 16149234.012207031 -4606178.467224121, 16149251.901245117 -4606143.753173828, 16149312.292114258 -4606014.9962768555, 16149353.10180664 -4605890.716491699, 16149415.229248047 -4605707.748168945, 16149449.626953125 -4605609.559326172, 16149461.270935059 -4605515.931640625, 16149469.63104248 -4605476.064697266, 16149472.169128418 -4605418.594299316, 16149469.63104248 -4605394.702697754, 16149476.18774414 -4605363.829223633, 16149489.334594727 -4605333.791381836, 16149506.35534668 -4605306.01953125, 16149503.961975098 -4605259.440673828, 16149531.279785156 -4605182.4138793945, 16149528.741699219 -4605146.584411621, 16149512.02154541 -4605133.144836426, 16149496.047180176 -4605097.457092285, 16149496.64831543 -4605068.949523926, 16149508.592895508 -4605044.308166504, 16149539.383850098 -4605018.718017578, 16149548.155822754 -4604996.045227051, 16149547.5546875 -4604946.083068848, 16149534.419006348 -4604909.95703125, 16149537.112915039 -4604895.469787598, 16149562.482666016 -4604863.379943848, 16149597.12524414 -4604856.05847168, 16149614.145996094 -4604835.453613281, 16149608.914001465 -4604808.433654785, 16149618.76574707 -4604797.840942383, 16149632.803161621 -4604797.090393066, 16149655.790649414 -4604802.457580566, 16149686.692932129 -4604778.128356934, 16149688.039916992 -4604756.334106445, 16149698.047546387 -4604749.465881348, 16149704.47064209 -4604743.758911133, 16149719.387451172 -4604730.503967285, 16149731.632629395 -4604690.498474121, 16149727.491516113 -4604678.673828125, 16149712.229614258 -4604635.114135742, 16149716.559936523 -4604620.3298950195, 16149732.679016113 -4604624.80480957, 16149746.861083984 -4604641.373352051, 16149761.34375 -4604644.517089844, 16149809.801147461 -4604625.017211914, 16149842.5625 -4604588.22668457, 16149897.053344727 -4604515.085083008, 16149955.117614746 -4604425.050048828, 16149992.4430542 -4604333.982055664, 16150050.073120117 -4604264.11328125, 16150080.519042969 -4604214.225524902, 16150075.253601074 -4604173.868041992, 16150074.106994629 -4604160.953674316, 16150058.121520996 -4604082.9296875, 16150035.289916992 -4604015.257507324, 16150052.75592041 -4603977.336364746, 16150083.51348877 -4603954.198547363, 16150114.716369629 -4603943.436828613, 16150130.245422363 -4603952.555969238, 16150160.991882324 -4603935.238037109, 16150171.745361328 -4603909.551513672, 16150166.669189453 -4603888.198059082, 16150173.982849121 -4603870.285583496, 16150195.478637695 -4603862.68157959, 16150225.935668945 -4603870.880310059, 16150253.554016113 -4603878.059448242, 16150260.567138672 -4603871.333435059, 16150263.261108398 -4603862.072692871, 16150268.337280273 -4603862.384216309, 16150284.834838867 -4603875.694702148, 16150304.961364746 -4603874.661010742, 16150331.956359863 -4603863.276306152, 16150353.485534668 -4603843.055786133, 16150369.671386719 -4603825.185791016, 16150366.565612793 -4603770.414978027, 16150367.912536621 -4603734.293029785, 16150387.89440918 -4603697.463195801, 16150401.26385498 -4603679.706787109, 16150395.998474121 -4603675.005737305, 16150410.102661133 -4603655.05456543, 16150420.4553833 -4603660.463562012, 16150432.21069336 -4603644.760375977, 16150485.855529785 -4603683.685668945, 16150476.148498535 -4603692.18157959, 16150478.463928223 -4603701.512939453, 16150474.434204102 -4603702.716491699, 16150485.699707031 -4603738.767578125, 16150523.6819458 -4603739.886169434, 16150559.749450684 -4603719.9490356445, 16150581.245239258 -4603703.9767456055, 16150629.76940918 -4603665.915100098, 16150665.447265625 -4603632.625488281, 16150689.926452637 -4603600.893493652, 16150757.51965332 -4603501.124633789, 16150779.805786133 -4603424.026062012, 16150784.859680176 -4603366.906921387, 16150785.995178223 -4603345.823608398, 16150802.403686523 -4603332.740356445, 16150826.982971191 -4603313.441162109, 16150846.385986328 -4603291.041137695, 16150849.07989502 -4603260.442993164, 16150851.039123535 -4603224.535217285, 16150864.8984375 -4603183.261230469, 16150896.401855469 -4603095.899719238, 16150921.415344238 -4603056.268676758, 16150933.437805176 -4603022.612854004, 16150980.603881836 -4602978.125671387, 16151006.162841797 -4602941.525085449, 16151019.565734863 -4602909.752746582, 16151024.051879883 -4602889.151733398, 16151020.612121582 -4602869.1455078125, 16151027.33581543 -4602855.114196777, 16151033.747802734 -4602844.665100098, 16151037.332336426 -4602820.623718262, 16151029.272766113 -4602793.4533081055, 16151026.567687988 -4602781.050354004, 16151036.274780273 -4602752.619995117, 16151034.259887695 -4602730.461853027, 16151057.60357666 -4602651.89654541, 16151073.600219727 -4602639.451293945, 16151115.712341309 -4602618.482727051, 16151133.256286621 -4602595.093139648, 16151147.905944824 -4602583.667358398, 16151149.408752441 -4602567.215454102, 16151153.082336426 -4602557.0498046875, 16151160.607543945 -4602555.265869141, 16151200.203857422 -4602566.649108887, 16151223.681152344 -4602578.315551758, 16151241.113769531 -4602586.980407715, 16151255.451721191 -4602586.683105469, 16151270.680236816 -4602582.8037109375, 16151303.218933105 -4602571.165588379, 16151339.05267334 -4602554.444641113, 16151405.33227539 -4602504.876647949, 16151431.013671875 -4602477.409790039, 16151449.826660156 -4602455.308898926, 16151458.632080078 -4602440.074707031, 16151468.639648438 -4602427.842102051, 16151481.92010498 -4602422.759338379, 16151511.93182373 -4602434.255737305, 16151595.232177734 -4602507.26940918, 16151607.655456543 -4602514.914794922, 16151608.445800781 -4602516.896972656, 16151607.777893066 -4602518.383605957, 16151603.592285156 -4602520.847106934, 16151593.78503418 -4602528.747436523, 16151610.850341797 -4602578.188110352, 16151627.559387207 -4602592.289794922, 16151637.745117188 -4602594.894958496, 16151645.927124023 -4602590.16607666, 16151648.888183594 -4602593.833068848, 16151665.597229004 -4602622.149780273, 16151673.58996582 -4602630.474914551, 16151676.840515137 -4602633.872924805, 16151707.308654785 -4602649.50378418, 16151722.425842285 -4602655.960021973, 16151745.669372559 -4602657.220092773, 16151820.164367676 -4602632.811035156, 16151846.992370605 -4602624.018676758, 16151858.881286621 -4602620.12512207, 16151991.718811035 -4602535.81237793, 16152047.59008789 -4602539.50769043, 16152078.937683105 -4602530.856994629, 16152208.012634277 -4602428.181884766, 16152231.222717285 -4602392.942443848, 16152320.144714355 -4602331.893127441, 16152410.068603516 -4602209.271606445, 16152455.609436035 -4602133.060424805, 16152487.10168457 -4602072.224975586, 16152494.860656738 -4602061.479370117, 16152510.389709473 -4602049.233032227, 16152523.82598877 -4602028.931091309, 16152562.041992188 -4601959.955688477, 16152603.63092041 -4601877.149047852, 16152667.43927002 -4601796.933837891, 16152688.044494629 -4601782.012084961, 16152702.37133789 -4601760.804626465, 16152706.5569458 -4601740.800537109, 16152725.959960938 -4601721.391052246, 16152750.750793457 -4601685.262145996, 16152820.525817871 -4601558.981689453, 16152849.279663086 -4601536.868652344, 16152894.664611816 -4601512.688720703, 16152945.125732422 -4601489.995300293, 16152988.11730957 -4601449.988220215, 16153030.819458008 -4601414.157531738, 16153062.467590332 -4601387.273925781, 16153170.258300781 -4601285.459655762, 16153211.635742188 -4601247.619262695, 16153251.465881348 -4601223.652404785, 16153288.991638184 -4601204.187255859, 16153300.145874023 -4601193.258544922, 16153305.444702148 -4601186.038757324, 16153321.396728516 -4601192.012756348, 16153341.879516602 -4601206.084228516, 16153371.423706055 -4601205.518005371, 16153386.94165039 -4601202.516784668, 16153398.36303711 -4601193.81060791, 16153430.912841797 -4601177.969604492, 16153462.471923828 -4601156.211242676, 16153502.36883545 -4601126.893493652, 16153575.1272583 -4601076.75189209, 16153644.100830078 -4601028.988830566, 16153691.767822266 -4601005.815185547, 16153791.009155273 -4600962.101074219, 16153827.733459473 -4600913.729797363, 16153931.639099121 -4600736.667724609, 16153965.680541992 -4600666.497741699, 16154013.147216797 -4600568.866455078, 16154069.285644531 -4600461.681152344, 16154101.234313965 -4600393.891113281, 16154161.62512207 -4600253.4005737305, 16154185.803710938 -4600199.66784668, 16154207.23272705 -4600151.144470215, 16154245.593444824 -4600069.0458984375, 16154262.469482422 -4600043.369018555, 16154270.028076172 -4600041.500549316, 16154283.82055664 -4600038.358215332, 16154284.009765625 -4600030.799499512, 16154361.900024414 -4599911.758178711, 16154409.366638184 -4599801.508178711, 16154458.93725586 -4599675.80279541, 16154503.721069336 -4599546.815185547, 16154535.001831055 -4599451.57232666, 16154575.377441406 -4599321.695861816, 16154595.982666016 -4599276.899047852, 16154647.634887695 -4599086.107788086, 16154712.734558105 -4598799.771850586, 16154745.874328613 -4598645.702026367, 16154779.61529541 -4598512.2267456055, 16154797.226013184 -4598441.463256836, 16154816.33959961 -4598331.894165039, 16154847.687133789 -4598195.436767578, 16154873.513305664 -4598042.861877441, 16154902.33392334 -4597885.505981445, 16154924.720275879 -4597737.563293457, 16154987.125976562 -4597344.182556152, 16154993.693786621 -4597289.842346191, 16155007.430603027 -4597148.007263184, 16155021.468017578 -4597058.729797363, 16155035.79486084 -4596902.874694824, 16155038.633483887 -4596870.328491211, 16155044.154907227 -4596797.170776367, 16155047.138305664 -4596744.630493164, 16155060.574584961 -4596659.82800293, 16155069.535766602 -4596577.120422363, 16155079.086975098 -4596519.204345703, 16155086.556518555 -4596434.106506348, 16155090.140991211 -4596377.365661621, 16155092.222717285 -4596327.204650879, 16155097.59942627 -4596262.116088867, 16155100.293334961 -4596214.0498046875, 16155105.848205566 -4596171.176635742, 16155105.959533691 -4596128.360229492, 16155111.636779785 -4596084.157409668, 16155110.445678711 -4596050.128112793, 16155116.111877441 -4596008.0197143555, 16155124.772521973 -4595923.520385742, 16155125.373657227 -4595865.296630859, 16155127.455322266 -4595785.283752441, 16155133.433166504 -4595708.242736816, 16155144.186645508 -4595655.990966797, 16155147.760009766 -4595613.601196289, 16155146.8694458 -4595574.777038574, 16155141.793273926 -4595545.814697266, 16155142.39440918 -4595522.526000977, 16155149.552246094 -4595501.331359863, 16155151.945617676 -4595484.0134887695, 16155146.624511719 -4595444.227722168, 16155157.62286377 -4595308.742248535, 16155166.873535156 -4595228.719482422, 16155173.74194336 -4595164.529174805, 16155176.424743652 -4595065.407653809, 16155178.21697998 -4595003.595214844, 16155175.233642578 -4594944.768249512, 16155177.916442871 -4594874.9064331055, 16155184.495422363 -4594825.941162109, 16155189.560424805 -4594745.625061035, 16155189.270996094 -4594613.346069336, 16155187.77935791 -4594537.5021362305, 16155194.63659668 -4594448.827270508, 16155196.139404297 -4594357.465270996, 16155185.987121582 -4594243.709472656, 16155180.77734375 -4594060.791809082, 16155180.877502441 -4594054.043884277, 16155181.189208984 -4594041.5665893555, 16155183.89428711 -4594000.513183594, 16155192.254394531 -4593964.227294922, 16155195.538330078 -4593924.518127441, 16155187.178222656 -4593849.2734375, 16155187.378601074 -4593827.657836914, 16155184.940673828 -4593792.0798950195, 16155177.916442871 -4593759.402038574, 16155182.102050781 -4593731.03894043, 16155182.40258789 -4593692.518920898, 16155180.009216309 -4593642.9509887695, 16155182.40258789 -4593601.446533203, 16155176.43585205 -4593553.845275879, 16155174.042541504 -4593523.827636719, 16155180.009216309 -4593470.370422363, 16155180.610351562 -4593427.381286621, 16155177.025878906 -4593382.440307617, 16155175.834777832 -4593346.170715332, 16155176.424743652 -4593261.367858887, 16155168.06463623 -4593181.049743652, 16155169.266906738 -4593106.701599121, 16155157.91229248 -4592905.457885742, 16155149.852783203 -4592774.376464844, 16155143.585510254 -4592692.2658081055, 16155141.492675781 -4592625.092529297, 16155141.793273926 -4592577.014831543, 16155138.208740234 -4592522.671386719, 16155129.247558594 -4592466.2490234375, 16155121.488586426 -4592388.016296387, 16155117.013549805 -4592339.048706055, 16155106.2600708 -4592232.160888672, 16155094.014953613 -4592158.399475098, 16155089.094604492 -4592085.40234375, 16155084.319030762 -4592021.811401367, 16155067.387329102 -4591952.337097168, 16155059.383422852 -4591932.22467041, 16155055.498413086 -4591920.881408691, 16155050.878601074 -4591912.975097656, 16155047.917541504 -4591908.958251953, 16155044.010192871 -4591874.150695801, 16155035.950683594 -4591848.918457031, 16155018.250915527 -4591746.024169922, 16155012.206237793 -4591687.24407959, 16155002.44354248 -4591600.701049805, 16154995.653015137 -4591544.311950684, 16154973.266662598 -4591405.865112305, 16154969.515197754 -4591329.535583496, 16154956.969482422 -4591252.202514648, 16154948.30883789 -4591168.307739258, 16154932.490356445 -4591065.886962891, 16154915.169006348 -4590987.961853027, 16154895.164916992 -4590846.736450195, 16154883.220336914 -4590770.297912598, 16154866.500183105 -4590685.20501709, 16154843.212097168 -4590589.350830078, 16154822.30633545 -4590511.429321289, 16154801.411621094 -4590446.334899902, 16154783.199768066 -4590363.6345825195, 16154764.01940918 -4590289.490539551, 16154755.035949707 -4590234.098510742, 16154749.536743164 -4590200.215759277, 16154743.480957031 -4590154.921081543, 16154717.81072998 -4590031.906677246, 16154688.38897705 -4589876.581726074, 16154676.110412598 -4589841.85168457, 16154661.839294434 -4589851.283630371, 16154653.991271973 -4589842.191040039, 16154684.75994873 -4589817.331420898, 16154674.730041504 -4589785.401489258, 16154665.023010254 -4589750.841430664, 16154641.289672852 -4589768.814331055, 16154635.823913574 -4589757.44519043, 16154664.076782227 -4589738.0158081055, 16154652.71105957 -4589693.317077637, 16154641.668151855 -4589653.299072266, 16154624.346862793 -4589614.483154297, 16154606.101623535 -4589568.031494141, 16154597.351867676 -4589528.805847168, 16154586.598449707 -4589488.194580078, 16154577.34777832 -4589434.447143555, 16154565.993164062 -4589390.2587890625, 16154546.289611816 -4589318.002319336, 16154524.793823242 -4589231.4083862305, 16154510.455871582 -4589144.829345703, 16154493.446289062 -4589041.8064575195, 16154480.010009766 -4588968.364685059, 16154467.764892578 -4588925.366027832, 16154455.15234375 -4588874.972595215, 16154444.844177246 -4588828.863586426, 16154431.652832031 -4588773.960144043, 16154422.101623535 -4588733.139770508, 16154412.6171875 -4588689.378540039, 16154401.006591797 -4588637.826843262, 16154388.772583008 -4588572.730041504, 16154379.065490723 -4588520.924377441, 16154368.467895508 -4588476.287475586, 16154344.879272461 -4588411.799682617, 16154325.175720215 -4588342.519226074, 16154318.930725098 -4588281.920410156, 16154301.587158203 -4588212.640869141, 16154288.150878906 -4588149.342346191, 16154271.274841309 -4588072.315795898, 16154260.532531738 -4588010.502685547, 16154243.65649414 -4587965.415527344, 16154226.045715332 -4587898.527404785, 16154202.156555176 -4587820.908935547, 16154181.851867676 -4587754.318664551, 16154116.7633667 -4587549.786010742, 16154060.034973145 -4587393.635925293, 16153991.350830078 -4587184.62890625, 16153945.375915527 -4587038.917419434, 16153900.881469727 -4586892.9111328125, 16153844.453674316 -4586722.422302246, 16153779.66571045 -4586547.15814209, 16153730.696289062 -4586445.334411621, 16153662.91381836 -4586235.42767334, 16153580.50402832 -4585996.787536621, 16153550.648132324 -4585902.81439209, 16153487.051269531 -4585732.624511719, 16153413.302124023 -4585575.865783691, 16153360.447631836 -4585458.2205200195, 16153333.875671387 -4585378.50012207, 16153324.625 -4585334.018066406, 16153296.706115723 -4585245.77545166, 16153232.062866211 -4585082.310241699, 16153161.297058105 -4584917.490722656, 16153126.064453125 -4584838.664916992, 16153016.481567383 -4584572.029846191, 16152938.557922363 -4584389.594055176, 16152872.378479004 -4584245.448303223, 16152745.674621582 -4583960.837402344, 16152668.185119629 -4583778.765686035, 16152634.299438477 -4583704.951477051, 16152551.288513184 -4583537.7373046875, 16152522.924316406 -4583455.629211426, 16152504.712463379 -4583425.174499512, 16152448.585144043 -4583314.407653809, 16152359.60748291 -4583132.559326172, 16152206.431884766 -4582831.289916992, 16152073.560913086 -4582579.289611816, 16152012.35748291 -4582467.627502441, 16151941.29107666 -4582346.400085449, 16151916.288757324 -4582300.150634766, 16151728.103149414 -4581985.411193848, 16151569.55078125 -4581746.5439453125, 16151432.204833984 -4581560.823364258, 16151347.412780762 -4581461.0974731445, 16151249.329162598 -4581332.859619141, 16151181.847290039 -4581247.618225098, 16151145.12298584 -4581199.847717285, 16151066.97668457 -4581114.536804199, 16150946.26184082 -4580997.295410156, 16150888.987976074 -4580950.896606445, 16150797.65032959 -4580974.223144531, 16150770.121032715 -4580981.259277344, 16150745.831115723 -4581016.976867676, 16150733.463500977 -4581012.385009766, 16150723.711914062 -4581008.428894043, 16150716.921447754 -4581002.819763184, 16150691.117553711 -4580982.728637695, 16150719.793457031 -4580915.885681152, 16150663.354492188 -4580844.677368164, 16150644.842041016 -4580811.983886719, 16150630.91595459 -4580770.728637695, 16150603.5647583 -4580704.466247559, 16150544.87713623 -4580578.0603637695, 16150426.288513184 -4580387.995178223, 16150315.592407227 -4580230.088195801, 16150266.945800781 -4580160.805969238, 16150116.352783203 -4579951.775390625, 16149981.422424316 -4579808.892211914, 16149937.963256836 -4579755.986083984, 16149907.060974121 -4579690.987548828, 16149858.9821167 -4579612.879577637, 16149745.63659668 -4579509.93737793, 16149692.960205078 -4579461.284912109, 16149687.817260742 -4579459.16595459, 16149682.217895508 -4579447.469055176, 16149606.075378418 -4579355.504760742, 16149598.015808105 -4579325.937988281, 16149502.16973877 -4579221.444702148, 16149431.749023438 -4579144.498657227, 16149383.33618164 -4579102.896911621, 16149352.28918457 -4579062.001586914, 16149258.235351562 -4578968.246765137, 16149097.590209961 -4578801.038391113, 16149051.014099121 -4578757.742919922, 16148973.379882812 -4578686.083312988, 16148920.536560059 -4578640.401123047, 16148853.0546875 -4578577.697875977, 16148771.546508789 -4578513.794372559, 16148641.514221191 -4578410.637084961, 16148627.777404785 -4578406.003967285, 16148618.972045898 -4578405.114074707, 16148607.918029785 -4578432.1357421875, 16148601.36126709 -4578430.539550781, 16148610.456115723 -4578406.314697266, 16148577.761535645 -4578383.022216797, 16148462.813049316 -4578294.047790527, 16148397.123413086 -4578244.172180176, 16148208.75970459 -4578103.035766602, 16148020.195617676 -4577985.178771973, 16147831.30871582 -4577861.701538086, 16147666.823059082 -4577761.587463379, 16147501.046020508 -4577672.293518066, 16147365.815124512 -4577616.855957031, 16147254.15057373 -4577579.836547852, 16147132.92364502 -4577543.707092285, 16147083.063598633 -4577535.049072266, 16147062.469543457 -4577555.613708496, 16147059.920288086 -4577559.398986816, 16147029.975341797 -4577586.2771606445, 16147022.572631836 -4577592.110473633, 16147012.442565918 -4577597.4494018555, 16146997.214050293 -4577606.856079102, 16146985.12475586 -4577615.217590332, 16146970.052062988 -4577617.901184082, 16146961.691955566 -4577621.333374023, 16146952.285461426 -4577629.553649902, 16146940.185058594 -4577642.830383301, 16146914.659484863 -4577667.4630737305, 16146908.091674805 -4577664.172119141, 16146899.430969238 -4577664.482849121, 16146889.445617676 -4577670.386779785, 16146880.528930664 -4577681.474304199, 16146872.602966309 -4577699.157897949, 16146858.721435547 -4577717.773742676, 16146822.097351074 -4577743.663635254, 16146807.425415039 -4577749.680603027, 16146790.315612793 -4577756.460327148, 16146774.730895996 -4577768.014038086, 16146762.786315918 -4577781.177978516, 16146750.25177002 -4577791.813659668, 16146722.789245605 -4577808.551147461, 16146702.094909668 -4577810.994628906, 16146685.352478027 -4577813.070922852, 16146663.912353516 -4577820.782897949, 16146655.073547363 -4577828.876220703, 16146634.757751465 -4577837.704040527, 16146618.772277832 -4577840.345275879, 16146598.456481934 -4577846.743713379, 16146588.860717773 -4577854.455688477, 16146576.070129395 -4577869.681884766, 16146558.014099121 -4577895.4591674805, 16146547.861755371 -4577910.120483398, 16146539.20111084 -4577921.603759766, 16146523.026367188 -4577928.185852051, 16146507.230163574 -4577935.332885742, 16146489.552612305 -4577946.802062988, 16146482.405944824 -4577949.62701416, 16146475.069946289 -4577953.76550293, 16146454.564880371 -4577969.373291016, 16146441.206542969 -4577978.215332031, 16146439.893005371 -4577986.873718262, 16146408.478637695 -4577990.63092041, 16146401.5211792 -4577994.58581543, 16146401.33190918 -4578002.862915039, 16146443.27709961 -4578022.60925293, 16146446.661254883 -4578025.999206543, 16146447.039733887 -4578032.016296387, 16146441.774291992 -4578033.711303711, 16146398.70477295 -4578018.852050781, 16146385.346435547 -4578017.340698242, 16146371.420410156 -4578020.7306518555, 16146303.192687988 -4578048.683532715, 16146291.203552246 -4578060.731994629, 16146283.21081543 -4578063.232055664, 16146233.295166016 -4578112.810180664, 16146230.456481934 -4578118.629638672, 16146231.013122559 -4578127.853149414, 16146233.818359375 -4578133.926879883, 16146232.11517334 -4578137.443969727, 16146229.04272461 -4578139.887573242, 16146223.109436035 -4578140.918701172, 16146214.649169922 -4578144.026184082, 16146200.734191895 -4578151.540588379, 16146178.915588379 -4578168.942504883, 16146167.68347168 -4578176.852478027, 16146159.6350708 -4578182.078735352, 16146147.812927246 -4578182.064575195, 16146140.922241211 -4578177.714111328, 16146118.246459961 -4578182.940307617, 16146104.832458496 -4578186.400939941, 16146100.201599121 -4578197.376037598, 16146095.726501465 -4578210.512268066, 16146094.156921387 -4578225.739013672, 16146087.956420898 -4578239.030639648, 16146080.798583984 -4578251.192321777, 16146072.805847168 -4578258.141845703, 16146057.733215332 -4578268.890991211, 16146051.087402344 -4578275.600402832, 16146048.326721191 -4578284.64050293, 16146045.93334961 -4578296.279541016, 16146046.901794434 -4578310.164550781, 16146043.840515137 -4578319.204650879, 16146038.619628906 -4578324.57220459, 16146034.8125 -4578333.908935547, 16146033.487792969 -4578345.519775391, 16146031.606506348 -4578357.271972656, 16146031.673278809 -4578385.705993652, 16146030.259521484 -4578395.0427856445, 16146027.721435547 -4578403.02355957, 16146024.582214355 -4578408.3204956055, 16146019.806640625 -4578412.3603515625, 16146011.145996094 -4578415.114746094, 16146004.35546875 -4578418.024597168, 16145997.386901855 -4578424.423339844, 16145986.889465332 -4578437.814086914, 16145978.6852417 -4578445.498291016, 16145970.024536133 -4578448.337463379, 16145960.985412598 -4578455.880371094, 16145953.304382324 -4578461.770629883, 16145950.309875488 -4578466.177734375, 16145949.196655273 -4578471.10748291, 16145946.057495117 -4578477.449768066, 16145947.025939941 -4578481.927490234, 16145946.73651123 -4578487.153869629, 16145937.697387695 -4578497.606689453, 16145924.038452148 -4578509.175354004, 16145910.156921387 -4578519.176147461, 16145899.036132812 -4578528.724914551, 16145888.583251953 -4578540.675048828, 16145880.445739746 -4578549.107910156, 16145875.069030762 -4578557.470214844, 16145864.549316406 -4578565.521728516, 16145860.21899414 -4578572.316040039, 16145856.935119629 -4578582.316894531, 16145851.936828613 -4578587.176086426, 16145842.530334473 -4578588.8994140625, 16145796.176940918 -4578593.744445801, 16145782.139526367 -4578600.538818359, 16145774.747924805 -4578609.791015625, 16145770.339660645 -4578617.927368164, 16145767.211608887 -4578626.967712402, 16145765.341430664 -4578635.104003906, 16145766.020446777 -4578642.5623168945, 16145762.502746582 -4578647.336791992, 16145757.882995605 -4578650.105407715, 16145753.77532959 -4578655.40246582, 16145751.459899902 -4578663.383422852, 16145751.014587402 -4578668.087280273, 16145748.77709961 -4578671.307922363, 16145743.467163086 -4578673.6951293945, 16145739.070007324 -4578674.514465332, 16145731.455749512 -4578673.017150879, 16145723.908325195 -4578675.333740234, 16145720.702270508 -4578680.263549805, 16145720.780212402 -4578686.5353393555, 16145721.604003906 -4578690.928405762, 16145718.843261719 -4578693.923034668, 16145714.880310059 -4578695.265014648, 16145712.041625977 -4578693.838317871, 16145708.16772461 -4578691.832458496, 16145701.967224121 -4578691.747680664, 16145694.653564453 -4578695.039001465, 16145692.037536621 -4578700.717468262, 16145688.686828613 -4578709.743774414, 16145686.81665039 -4578718.247436523, 16145687.184020996 -4578726.242614746, 16145689.655273438 -4578732.1329956055, 16145692.260192871 -4578740.128173828, 16145688.30834961 -4578749.535888672, 16145683.44366455 -4578761.161376953, 16145682.041015625 -4578774.6796875, 16145678.679199219 -4578793.848388672, 16145664.786499023 -4578800.204956055, 16145639.895507812 -4578813.356079102, 16145623.787536621 -4578822.721496582, 16145604.94116211 -4578836.112792969, 16145604.94116211 -4578846.1845703125, 16145608.970947266 -4578860.084411621, 16145605.653625488 -4578872.953063965, 16145600.454956055 -4578880.015991211, 16145598.206359863 -4578888.223144531, 16145597.015197754 -4578903.902954102, 16145596.714660645 -4578910.316101074, 16145594.477111816 -4578916.432678223, 16145591.048461914 -4578920.7693481445, 16145584.92590332 -4578921.065979004, 16145578.981445312 -4578915.387329102, 16145575.96472168 -4578913.000061035, 16145573.126037598 -4578912.562133789, 16145571.344909668 -4578914.497375488, 16145572.680786133 -4578920.176025391, 16145569.99798584 -4578922.252563477, 16145565.36706543 -4578922.859985352, 16145561.938415527 -4578919.86529541, 16145558.053405762 -4578918.52331543, 16145553.867797852 -4578919.568603516, 16145551.329711914 -4578924.046569824, 16145546.854675293 -4578924.795227051, 16145533.641052246 -4578920.091308594, 16145518.312316895 -4578910.6834106445, 16145503.707214355 -4578911.361450195, 16145498.04107666 -4578911.954711914, 16145493.265441895 -4578906.290222168, 16145490.571533203 -4578897.475646973, 16145476.389404297 -4578885.22845459, 16145455.338867188 -4578873.744140625, 16145448.002929688 -4578865.84777832, 16145426.829956055 -4578835.222900391, 16145416.410461426 -4578824.8544921875, 16145384.940429688 -4578798.001342773, 16145376.06829834 -4578787.4494018555, 16145364.724853516 -4578779.24230957, 16145353.826660156 -4578770.724487305, 16145344.264282227 -4578766.698669434, 16145331.807678223 -4578763.350830078, 16145317.992919922 -4578766.10534668, 16145309.33227539 -4578766.10534668, 16145300.22631836 -4578763.266113281, 16145296.497131348 -4578757.149658203, 16145283.650817871 -4578745.354675293, 16145264.392578125 -4578734.901672363, 16145240.213989258 -4578726.087219238, 16145186.468933105 -4578713.105712891, 16145153.919128418 -4578705.336608887, 16145124.063232422 -4578696.536315918, 16145053.45324707 -4578683.24407959, 16144981.785766602 -4578680.560180664, 16144934.608581543 -4578680.263549805, 16144904.162719727 -4578673.6951293945, 16144855.03741455 -4578660.106323242, 16144824.713989258 -4578653.227172852, 16144808.016052246 -4578654.583190918, 16144789.804199219 -4578657.267089844, 16144771.135925293 -4578658.609008789, 16144758.156066895 -4578654.42779541, 16144755.829467773 -4578648.198425293, 16144758.60131836 -4578634.115234375, 16144769.800048828 -4578601.880737305, 16144769.644226074 -4578576.497192383, 16144765.770324707 -4578552.611022949, 16144755.016845703 -4578525.589050293, 16144748.003723145 -4578514.543029785, 16144743.51751709 -4578509.019958496, 16144738.74194336 -4578510.065246582, 16144714.262756348 -4578526.634338379, 16144706.147583008 -4578517.33984375, 16144686.499694824 -4578504.245605469, 16144652.78100586 -4578489.343322754, 16144645.166748047 -4578481.99810791, 16144647.526733398 -4578471.700744629, 16144654.840454102 -4578424.366821289, 16144652.00177002 -4578375.860717773, 16144633.945739746 -4578322.848937988, 16144617.069763184 -4578287.168884277, 16144591.989440918 -4578250.443725586, 16144568.55670166 -4578226.854919434, 16144554.96459961 -4578208.944396973, 16144545.257507324 -4578199.099304199, 16144517.193908691 -4578177.897705078, 16144500.919006348 -4578163.857543945, 16144483.308227539 -4578142.6560668945, 16144454.342895508 -4578109.519104004, 16144431.65600586 -4578055.4775390625, 16144424.186462402 -4578019.643066406, 16144418.664978027 -4577992.02923584, 16144413.432983398 -4577961.124450684, 16144409.85961914 -4577934.993896484, 16144392.983581543 -4577889.315002441, 16144382.241271973 -4577861.249572754, 16144371.387634277 -4577838.636230469, 16144349.39086914 -4577791.827819824, 16144330.433166504 -4577753.2540893555, 16144311.59790039 -4577726.544921875, 16144298.807312012 -4577710.188964844, 16144287.708740234 -4577699.454528809, 16144278.491516113 -4577693.253967285, 16144273.983093262 -4577684.59576416, 16144273.059143066 -4577675.725708008, 16144261.381713867 -4577665.231384277, 16144229.967346191 -4577645.669372559, 16144210.030029297 -4577626.672302246, 16144175.977355957 -4577589.991821289, 16144141.368164062 -4577529.229919434, 16144119.549560547 -4577491.23626709, 16144093.411743164 -4577467.352600098, 16144049.162231445 -4577429.048461914, 16144027.365844727 -4577410.532043457, 16144005.758728027 -4577409.2326049805, 16143996.352233887 -4577412.057373047, 16143987.880859375 -4577419.2041015625, 16143976.971557617 -4577420.136291504, 16143963.813537598 -4577411.492431641, 16143950.833740234 -4577401.888183594, 16143934.091247559 -4577388.159790039, 16143929.760925293 -4577371.606628418, 16143926.755310059 -4577329.856628418, 16143921.11138916 -4577311.9900512695, 16143907.007202148 -4577281.510986328, 16143892.335327148 -4577261.766052246, 16143874.479675293 -4577250.848449707, 16143860.920959473 -4577237.120239258, 16143791.190429688 -4577120.911376953, 16143749.389953613 -4577063.584228516, 16143718.932922363 -4577032.526855469, 16143681.318115234 -4577006.554016113, 16143623.153686523 -4576975.90637207, 16143585.238220215 -4576958.111022949, 16143572.681396484 -4576951.9533081055, 16143544.717956543 -4576937.8865356445, 16143447.224365234 -4576892.197875977, 16143359.437805176 -4576858.457641602, 16143346.457946777 -4576849.206970215, 16143343.853088379 -4576835.182739258, 16143348.817932129 -4576805.947937012, 16143346.892089844 -4576784.763305664, 16143334.034667969 -4576765.0193481445, 16143298.523742676 -4576724.571105957, 16143251.702758789 -4576671.9490356445, 16143247.72869873 -4576659.294921875, 16143248.864135742 -4576651.301391602, 16143258.137023926 -4576609.130615234, 16143262.879272461 -4576576.817687988, 16143262.58984375 -4576569.106628418, 16143260.33001709 -4576561.29675293, 16143251.969970703 -4576537.415222168, 16143239.925170898 -4576504.113952637, 16143229.394348145 -4576476.179382324, 16143207.297424316 -4576431.608459473, 16143196.666442871 -4576410.452880859, 16143173.901611328 -4576369.907165527, 16143158.016296387 -4576346.492126465, 16143130.297729492 -4576273.366333008, 16143110.237976074 -4576240.221252441, 16143095.187561035 -4576223.85345459, 16143078.634399414 -4576205.60748291, 16143066.600708008 -4576185.864624023, 16143060.010620117 -4576169.313354492, 16143054.555969238 -4576155.007568359, 16143042.900817871 -4576135.264770508, 16143032.358886719 -4576116.835388184, 16143028.785522461 -4576107.430053711, 16143023.330871582 -4576098.024719238, 16143017.876220703 -4576092.757202148, 16143003.772033691 -4576077.70300293, 16142984.780883789 -4576036.890258789, 16142944.338562012 -4575940.578491211, 16142903.328430176 -4575842.587158203, 16142875.120056152 -4575795.943054199, 16142858.566894531 -4575772.797607422, 16142849.538879395 -4575752.081176758, 16142785.39654541 -4575646.409484863, 16142736.86126709 -4575588.850402832, 16142700.370727539 -4575546.330932617, 16142681.35736084 -4575527.676635742, 16142658.058227539 -4575515.87121582, 16142637.931640625 -4575500.253051758, 16142631.909240723 -4575465.076965332, 16142621.378417969 -4575444.770629883, 16142610.65838623 -4575429.533813477, 16142596.220214844 -4575406.262084961, 16142587.893554688 -4575402.6329956055, 16142577.741210938 -4575399.80871582, 16142562.880065918 -4575392.663452148, 16142557.993103027 -4575383.0751953125, 16142555.733337402 -4575365.197814941, 16142548.208129883 -4575342.067504883, 16142543.321228027 -4575336.800292969, 16142538.423156738 -4575331.71673584, 16142536.541870117 -4575320.052734375, 16142532.22265625 -4575314.601989746, 16142523.562011719 -4575301.808349609, 16142519.621276855 -4575287.334289551, 16142511.717590332 -4575269.838378906, 16142497.657958984 -4575247.061279297, 16142446.885131836 -4575189.928100586, 16142430.443237305 -4575187.21685791, 16142376.86517334 -4575132.611633301, 16142368.282409668 -4575130.705322266, 16142363.81854248 -4575125.155822754, 16142362.115356445 -4575115.172424316, 16142365.733215332 -4575100.458618164, 16142378.991394043 -4575091.633117676, 16142420.057128906 -4575088.865478516, 16142436.81072998 -4575087.481628418, 16142454.52166748 -4575082.087524414, 16142461.813049316 -4575078.472595215, 16142463.11553955 -4575073.55859375, 16142467.022827148 -4575071.073364258, 16142469.995056152 -4575070.720336914, 16142472.477478027 -4575072.372497559, 16142482.373779297 -4575067.077209473, 16142504.737854004 -4575053.154174805, 16142526.066711426 -4575052.095153809, 16142532.22265625 -4575047.477661133, 16142535.606750488 -4575043.029663086, 16142549.532836914 -4575021.919311523, 16142549.766601562 -4575019.123413086, 16142541.395385742 -4575014.971923828, 16142542.63104248 -4575010.312133789, 16142545.892700195 -4574998.973266602, 16142553.529174805 -4574973.1466674805, 16142570.293945312 -4574942.660400391, 16142580.290405273 -4574925.885253906, 16142587.604125977 -4574890.202758789, 16142594.628356934 -4574843.478210449, 16142595.674743652 -4574789.7361450195, 16142597.611694336 -4574760.916564941, 16142597.455871582 -4574738.521850586, 16142595.674743652 -4574724.6416015625, 16142594.47253418 -4574711.947509766, 16142591.043884277 -4574697.17779541, 16142581.937927246 -4574670.448303223, 16142570.594482422 -4574642.419799805, 16142561.588745117 -4574627.946655273, 16142562.612854004 -4574613.219421387, 16142547.81854248 -4574549.072021484, 16142546.482666016 -4574544.172424316, 16142547.039245605 -4574535.5451049805, 16142552.816772461 -4574520.380249023, 16142550.089416504 -4574503.1962890625, 16142541.439880371 -4574481.635131836, 16142535.0390625 -4574464.564208984, 16142527.691955566 -4574454.609680176, 16142520.70111084 -4574450.825561523, 16142517.984863281 -4574445.2482299805, 16142517.973754883 -4574423.786071777, 16142516.081298828 -4574405.9103393555, 16142511.58404541 -4574395.955871582, 16142504.236938477 -4574390.858642578, 16142495.587402344 -4574385.182495117, 16142493.327636719 -4574379.252197266, 16142497.001159668 -4574371.909912109, 16142501.743347168 -4574367.758728027, 16142506.964233398 -4574371.076843262, 16142518.39678955 -4574382.8103637695, 16142535.873901367 -4574388.9666137695, 16142555.544067383 -4574377.360168457, 16142567.154724121 -4574366.332641602, 16142573.19934082 -4574354.4861450195, 16142578.531555176 -4574340.267578125, 16142582.31640625 -4574320.8388671875, 16142583.3850708 -4574296.778930664, 16142578.999084473 -4574265.263916016, 16142572.008239746 -4574234.935058594, 16142559.217651367 -4574190.7267456055, 16142547.606994629 -4574156.006958008, 16142537.532592773 -4574138.004638672, 16142526.155761719 -4574108.975097656, 16142514.311340332 -4574077.926574707, 16142506.36315918 -4574062.635314941, 16142503.29071045 -4574051.975219727, 16142495.46496582 -4574024.244934082, 16142473.90234375 -4573976.027648926, 16142451.026245117 -4573922.812438965, 16142434.205810547 -4573888.573608398, 16142407.06616211 -4573843.194885254, 16142380.193603516 -4573801.162597656, 16142337.44696045 -4573741.3829956055, 16142289.813354492 -4573700.268859863, 16142266.347167969 -4573713.921813965, 16142258.966674805 -4573703.403259277, 16142291.082397461 -4573682.041442871, 16142290.035949707 -4573677.650512695, 16142288.989562988 -4573672.864196777, 16142284.158325195 -4573666.863708496, 16142277.913269043 -4573659.860778809, 16142267.805480957 -4573648.6787109375, 16142230.9475708 -4573588.236450195, 16142173.773864746 -4573514.339477539, 16142155.406188965 -4573492.9921875, 16142146.600830078 -4573475.372192383, 16142127.787780762 -4573437.746398926, 16142084.050354004 -4573387.145874023, 16142041.504089355 -4573346.611938477, 16141993.881591797 -4573374.905212402, 16141985.37677002 -4573362.579833984, 16142032.821166992 -4573333.552490234, 16142048.528320312 -4573323.810791016, 16142052.201904297 -4573307.814758301, 16142049.752868652 -4573295.785949707, 16142038.465026855 -4573272.829650879, 16142009.878234863 -4573218.29095459, 16141985.799804688 -4573183.108459473, 16141948.563415527 -4573117.473449707, 16141930.31817627 -4573087.938537598, 16141915.45703125 -4573068.00390625, 16141840.349731445 -4572987.57409668, 16141830.976623535 -4572977.126831055, 16141824.475585938 -4572966.425537109, 16141804.015075684 -4572911.95904541, 16141775.80670166 -4572880.871704102, 16141739.104675293 -4572852.410400391, 16141718.143188477 -4572827.4080200195, 16141700.654907227 -4572807.671569824, 16141682.031188965 -4572776.824584961, 16141669.24053955 -4572761.408203125, 16141655.325622559 -4572741.841247559, 16141647.421936035 -4572733.370727539, 16141625.792541504 -4572707.2392578125, 16141621.651489258 -4572684.4677734375, 16141617.510375977 -4572674.882019043, 16141606.601074219 -4572657.5740356445, 16141587.231506348 -4572638.388427734, 16141542.314086914 -4572591.250549316, 16141515.508361816 -4572564.441772461, 16141488.090332031 -4572522.668762207, 16141438.653381348 -4572442.21496582, 16141360.774230957 -4572359.757080078, 16141303.812072754 -4572293.237915039, 16141221.892028809 -4572220.281860352, 16141163.850097656 -4572163.828857422, 16141155.044677734 -4572152.563659668, 16141154.877746582 -4572148.003967285, 16141159.285949707 -4572143.2607421875, 16141159.452941895 -4572134.621337891, 16141156.024291992 -4572126.786560059, 16141150.469482422 -4572117.653015137, 16141121.593200684 -4572086.483459473, 16141071.332458496 -4572040.463256836, 16141029.899353027 -4572009.180969238, 16140977.66821289 -4571979.762145996, 16140967.549255371 -4571971.758056641, 16140951.062866211 -4571956.102905273, 16140941.43371582 -4571946.461364746, 16140928.220092773 -4571942.7205200195, 16140922.019592285 -4571941.0830078125, 16140914.182739258 -4571929.013427734, 16140909.618652344 -4571914.318237305, 16140893.132202148 -4571911.706665039, 16140879.183898926 -4571903.476806641, 16140859.112976074 -4571872.053771973, 16140819.984191895 -4571772.025756836, 16140778.18371582 -4571675.259521484, 16140738.943603516 -4571643.808898926, 16140689.172607422 -4571621.872619629, 16140644.544677734 -4571601.474975586, 16140607.208068848 -4571581.3173828125, 16140583.296691895 -4571565.0134887695, 16140568.40209961 -4571542.950256348, 16140561.555969238 -4571513.518676758, 16140539.447937012 -4571272.803527832, 16140531.800292969 -4571195.408325195, 16140543.266174316 -4571180.121337891, 16140553.774719238 -4571171.525024414, 16140563.337097168 -4571176.296020508, 16140552.450012207 -4571205.712585449, 16140557.437133789 -4571258.193969727, 16140566.09777832 -4571351.949707031, 16140572.520935059 -4571423.4033203125, 16140585.77911377 -4571529.427246094, 16140605.716430664 -4571557.193237305, 16140683.528747559 -4571595.743896484, 16140765.894042969 -4571632.050292969, 16140783.771911621 -4571635.748657227, 16140800.113647461 -4571639.1365356445, 16140808.740905762 -4571640.929260254, 16140811.223327637 -4571641.183349609, 16140813.995178223 -4571641.465637207, 16140840.411315918 -4571645.7286987305, 16140877.068786621 -4571651.43157959, 16140880.586486816 -4571650.090576172, 16140900.71307373 -4571642.383178711, 16140906.546203613 -4571639.799987793, 16140956.762451172 -4571617.764831543, 16140951.77532959 -4571607.601318359, 16140957.2522583 -4571604.735778809, 16140961.426696777 -4571602.88659668, 16140966.569702148 -4571611.864379883, 16140992.206542969 -4571595.772155762, 16141022.151489258 -4571582.9689331055, 16141029.665588379 -4571571.944396973, 16141031.613647461 -4571559.1271362305, 16141038.994140625 -4571540.352966309, 16141047.665893555 -4571531.149353027, 16141070.074523926 -4571513.024597168, 16141086.650024414 -4571498.767578125, 16141138.51373291 -4571452.397094727, 16141177.776123047 -4571396.4140625, 16141225.187133789 -4571312.722351074, 16141252.438110352 -4571232.673156738, 16141253.050354004 -4571197.14453125, 16141250.579101562 -4571162.914672852, 16141247.284057617 -4571146.724304199, 16141239.146606445 -4571130.957458496, 16141225.754821777 -4571132.891235352, 16141222.393005371 -4571132.241943359, 16141220.188842773 -4571128.868408203, 16141220.700927734 -4571082.118469238, 16141220.478271484 -4571011.923400879, 16141217.539489746 -4571002.296875, 16141210.849182129 -4570991.851623535, 16141194.529724121 -4570969.817932129, 16141189.475830078 -4570953.670227051, 16141181.638916016 -4570930.662597656, 16141168.915100098 -4570903.251159668, 16141161.07824707 -4570886.934204102, 16141156.514099121 -4570885.621520996, 16141072.957702637 -4570909.941711426, 16141039.18334961 -4570922.179443359, 16141038.203796387 -4570913.541015625, 16141064.731201172 -4570903.745178223, 16141072.145080566 -4570885.621520996, 16141061.380493164 -4570841.074645996, 16141036.634155273 -4570765.715148926, 16141006.544494629 -4570664.187561035, 16140970.977905273 -4570577.537658691, 16140925.938049316 -4570482.36340332, 16140838.819396973 -4570351.84777832, 16140792.543884277 -4570287.359802246, 16140774.131652832 -4570268.7431640625, 16140767.786437988 -4570264.212524414, 16140761.073913574 -4570263.238647461, 16140743.485412598 -4570272.398742676, 16140658.882629395 -4570336.44909668, 16140650.622680664 -4570324.917724609, 16140723.26977539 -4570270.690917969, 16140730.038024902 -4570259.964172363, 16140723.303161621 -4570228.0662231445, 16140707.941101074 -4570204.241638184, 16140690.196777344 -4570178.342285156, 16140648.451965332 -4570124.2150268555, 16140592.914672852 -4570052.3751831055, 16140522.78338623 -4570000.464416504, 16140471.799072266 -4569977.642456055, 16140433.738952637 -4570058.811096191, 16140420.536437988 -4570051.034362793, 16140507.977905273 -4569892.0568237305, 16140509.3805542 -4569888.726013184, 16140419.33416748 -4569790.664794922, 16140354.657592773 -4569730.611938477, 16140362.327453613 -4569711.050720215, 16140348.64630127 -4569679.888305664, 16140251.45324707 -4569562.395141602, 16140225.148498535 -4569545.600463867, 16140244.262023926 -4569530.075927734, 16140241.534667969 -4569512.9990234375, 16140238.239624023 -4569495.10357666, 16140230.224609375 -4569480.919921875, 16140206.380004883 -4569456.250244141, 16140151.065368652 -4569382.029846191, 16140087.546447754 -4569305.961242676, 16140019.69720459 -4569232.772155762, 16139947.528808594 -4569166.9786987305, 16139891.39038086 -4569131.82409668, 16139840.617553711 -4569093.028503418, 16139778.735046387 -4569047.727233887, 16139741.932800293 -4569036.63482666, 16139724.455688477 -4569039.711364746, 16139719.112304688 -4569033.1349487305, 16139718.500061035 -4569021.605041504, 16139726.381469727 -4569008.212280273, 16139723.932434082 -4568998.093688965, 16139731.279541016 -4568943.111755371, 16139754.122314453 -4568691.64642334, 16139757.918273926 -4568680.145141602, 16139768.304382324 -4568676.320800781, 16139813.221801758 -4568686.806030273, 16139829.42993164 -4568652.810180664, 16139834.094238281 -4568629.440795898, 16139833.35949707 -4568607.059265137, 16139821.893615723 -4568557.371337891, 16139783.677612305 -4568487.630859375, 16139783.677612305 -4568468.509460449, 16139790.367919922 -4568454.186035156, 16139814.847106934 -4568448.259094238, 16139817.017822266 -4568438.536193848, 16139829.942016602 -4568424.551513672, 16139838.880981445 -4568420.529663086, 16139847.430297852 -4568417.552124023, 16139861.033569336 -4568394.239685059, 16139882.662902832 -4568355.898498535, 16139895.097290039 -4568320.803039551, 16139911.02709961 -4568281.036865234, 16139919.776855469 -4568223.4904174805, 16139917.76196289 -4568150.407775879, 16139913.197875977 -4568088.417053223, 16139899.071411133 -4568046.591552734, 16139894.139953613 -4568011.214904785, 16139870.428894043 -4567930.527954102, 16139844.057312012 -4567878.03515625, 16139819.02154541 -4567840.782409668, 16139817.385192871 -4567834.9122924805, 16139794.842956543 -4567807.269165039, 16139778.657104492 -4567783.309020996, 16139771.666259766 -4567743.671875, 16139775.41772461 -4567710.779785156, 16139768.037231445 -4567646.279968262, 16139755.736450195 -4567588.892272949, 16139747.075805664 -4567538.658935547, 16139741.932800293 -4567481.300048828, 16139729.186767578 -4567416.321960449, 16139703.483093262 -4567350.949157715, 16139680.250671387 -4567301.196716309, 16139638.728515625 -4567250.823608398, 16139597.6071167 -4567236.953430176, 16139593.421508789 -4567229.489196777, 16139598.798217773 -4567219.626281738, 16139685.660827637 -4567142.444763184, 16139675.29699707 -4567139.989624023, 16139671.679077148 -4567132.991088867, 16139669.59741211 -4567126.782714844, 16139667.404418945 -4567117.978149414, 16139661.960876465 -4567104.630187988, 16139660.402404785 -4567097.772827148, 16139660.536010742 -4567081.193725586, 16139646.164672852 -4567054.1170043945, 16139643.826965332 -4567047.51361084, 16139647.077514648 -4567039.4287109375, 16139645.608093262 -4567030.4548950195, 16139640.164550781 -4567012.535461426, 16139629.266357422 -4566990.326782227, 16139630.50201416 -4566977.176513672, 16139631.481628418 -4566962.629455566, 16139629.21069336 -4566957.789794922, 16139629.834106445 -4566950.241149902, 16139634.865722656 -4566945.965942383, 16139638.58380127 -4566945.189880371, 16139651.88647461 -4566935.637634277, 16139656.29473877 -4566929.274230957, 16139672.369262695 -4566854.973144531, 16139654.391174316 -4566800.750366211, 16139653.723266602 -4566782.66204834, 16139652.342895508 -4566763.600219727, 16139652.131408691 -4566736.425537109, 16139646.966186523 -4566707.374389648, 16139642.446594238 -4566676.757202148, 16139670.944396973 -4566686.746582031, 16139671.957397461 -4566677.505004883, 16139681.998413086 -4566661.603820801, 16139684.848205566 -4566649.399291992, 16139682.276733398 -4566627.163146973, 16139677.18939209 -4566597.604370117, 16139673.727355957 -4566557.30859375, 16139686.184020996 -4566558.973510742, 16139703.928344727 -4566558.578430176, 16139719.858154297 -4566557.421508789, 16139718.822875977 -4566547.968383789, 16139716.874816895 -4566540.4482421875, 16139714.180847168 -4566534.0568237305, 16139705.475708008 -4566520.1170043945, 16139697.137878418 -4566496.639526367, 16139694.276977539 -4566479.299560547, 16139697.215759277 -4566466.248718262, 16139702.848571777 -4566449.966918945, 16139705.219665527 -4566440.076538086, 16139706.510986328 -4566432.048522949, 16139704.718688965 -4566422.059387207, 16139701.534973145 -4566405.721252441, 16139697.19354248 -4566396.564575195, 16139692.139587402 -4566387.619506836, 16139683.067077637 -4566379.7185668945, 16139681.285949707 -4566372.0009765625, 16139684.67010498 -4566354.830505371, 16139684.269348145 -4566338.901611328, 16139677.4788208 -4566309.498840332, 16139675.00756836 -4566301.174682617, 16139666.680847168 -4566283.905578613, 16139663.352416992 -4566280.914489746, 16139652.298339844 -4566270.953735352, 16139633.997436523 -4566262.516723633, 16139615.08428955 -4566257.790283203, 16139591.64038086 -4566254.291320801, 16139569.73272705 -4566254.517028809, 16139550.107055664 -4566248.944091797, 16139536.13647461 -4566241.748657227, 16139523.590759277 -4566239.08215332, 16139500.770263672 -4566234.553222656, 16139409.443786621 -4566238.080444336, 16139392.901672363 -4566264.37902832, 16139382.84954834 -4566272.703186035, 16139376.905090332 -4566283.129577637, 16139374.76776123 -4566293.019836426, 16139367.598754883 -4566286.755493164, 16139365.316711426 -4566272.279968262, 16139368.589538574 -4566253.726989746, 16139405.3694458 -4566224.479614258, 16139416.646118164 -4566218.793823242, 16139466.639709473 -4566187.684265137, 16139498.098632812 -4566154.162231445, 16139532.385009766 -4566118.072570801, 16139568.207641602 -4566062.273498535, 16139577.246765137 -4566040.885070801, 16139580.541809082 -4566021.965698242, 16139579.428649902 -4566005.374206543, 16139579.272766113 -4565997.276000977, 16139578.493530273 -4565988.881530762, 16139565.825378418 -4565919.341674805, 16139557.298339844 -4565853.315246582, 16139556.452270508 -4565831.475830078, 16139551.621032715 -4565807.999938965, 16139546.188659668 -4565778.725646973, 16139524.414550781 -4565707.141540527, 16139518.45892334 -4565705.575561523, 16139518.581420898 -4565697.943115234, 16139514.696350098 -4565687.065917969, 16139497.59765625 -4565657.396911621, 16139489.315490723 -4565645.221801758, 16139487.801574707 -4565641.765380859, 16139486.209716797 -4565638.097351074, 16139486.07611084 -4565633.441772461, 16139492.554870605 -4565623.594482422, 16139495.404663086 -4565615.821044922, 16139491.51965332 -4565604.435974121, 16139490.350769043 -4565577.278442383, 16139482.703125 -4565517.094787598, 16139479.964660645 -4565485.76159668, 16139481.745788574 -4565460.7205200195, 16139519.082336426 -4565370.812927246, 16139523.791137695 -4565367.088562012, 16139537.294189453 -4565367.229614258, 16139540.989990234 -4565365.169921875, 16139539.976989746 -4565358.271362305, 16139530.481445312 -4565351.796020508, 16139530.12524414 -4565344.826904297, 16139544.808288574 -4565307.851257324, 16139571.55834961 -4565256.246398926, 16139571.925720215 -4565238.83795166, 16139566.782714844 -4565225.675842285, 16139559.10168457 -4565218.001464844, 16139556.274169922 -4565215.165893555, 16139557.231506348 -4565203.696655273, 16139575.387756348 -4565187.4591674805, 16139592.686767578 -4565169.303161621, 16139618.368164062 -4565123.172546387, 16139643.99395752 -4565073.952697754, 16139648.95880127 -4565061.341003418, 16139666.157653809 -4565025.988647461, 16139696.926330566 -4564991.158447266, 16139712.989746094 -4564968.2345581055, 16139737.724975586 -4564919.283508301, 16139749.279907227 -4564879.798339844, 16139759.487915039 -4564821.367980957, 16139758.831115723 -4564771.825134277, 16139756.92755127 -4564588.383178711, 16139759.510192871 -4564499.005065918, 16139761.792236328 -4564436.838684082, 16139761.346923828 -4564360.411315918, 16139743.736206055 -4564355.629333496, 16139730.32220459 -4564349.789428711, 16139728.57446289 -4564343.822509766, 16139730.522583008 -4564339.364990234, 16139733.995727539 -4564338.053100586, 16139752.44140625 -4564344.471374512, 16139754.155700684 -4564339.604797363, 16139752.062927246 -4564327.783874512, 16139747.543334961 -4564297.596862793, 16139737.06817627 -4564240.368713379, 16139713.657653809 -4564208.376403809, 16139693.252807617 -4564182.280456543, 16139672.202270508 -4564156.170471191, 16139643.270385742 -4564099.436889648, 16139637.125549316 -4564102.145202637, 16139616.075012207 -4564111.45501709, 16139609.874511719 -4564095.303894043, 16139635.644958496 -4564083.511413574, 16139661.793945312 -4564072.424316406, 16139597.195251465 -4563940.733825684, 16139571.55834961 -4563904.270996094, 16139562.007141113 -4563882.294616699, 16139537.17175293 -4563832.615234375, 16139519.97290039 -4563790.567016602, 16139507.705505371 -4563777.336181641, 16139483.660461426 -4563751.396484375, 16139426.330932617 -4563662.5333862305, 16139338.544372559 -4563549.396240234, 16139249.35522461 -4563433.453430176, 16139204.482299805 -4563389.333435059, 16139082.609741211 -4563234.703430176, 16139008.493225098 -4563161.782836914, 16138922.242858887 -4563079.088378906, 16138864.93560791 -4563021.514404297, 16138772.829833984 -4562940.302001953, 16138754.317443848 -4562915.069580078, 16138709.978881836 -4562884.322509766, 16138699.225402832 -4562910.584472656, 16138685.844787598 -4562928.736572266, 16138679.521850586 -4562929.145568848, 16138676.660949707 -4562924.872009277, 16138677.81866455 -4562921.543395996, 16138715.466918945 -4562869.499084473, 16138674.390014648 -4562848.991699219, 16138637.06463623 -4562822.109375, 16138614.077148438 -4562797.624755859, 16138608.110412598 -4562766.27154541, 16138609.602111816 -4562747.471008301, 16138614.377685547 -4562734.32611084, 16138623.628356934 -4562715.5255737305, 16138642.441345215 -4562692.225952148, 16138639.981201172 -4562655.485412598, 16138642.741882324 -4562588.027038574, 16138633.480102539 -4562519.356201172, 16138619.453857422 -4562387.078979492, 16138620.188598633 -4562363.8505859375, 16138626.455871582 -4562354.88079834, 16138651.625183105 -4562347.98425293, 16138651.046325684 -4562338.492614746, 16138646.159423828 -4562326.391967773, 16138623.672851562 -4562327.294555664, 16138602.878417969 -4562317.802978516, 16138593.549804688 -4562299.257080078, 16138585.412353516 -4562261.967895508, 16138554.9553833 -4562170.310974121, 16138515.247680664 -4562080.1357421875, 16138488.375183105 -4561996.533203125, 16138473.44720459 -4561893.8220825195, 16138479.714538574 -4561755.573303223, 16138463.884887695 -4561600.318908691, 16138454.62310791 -4561557.659240723, 16138439.405700684 -4561523.884216309, 16138368.027648926 -4561422.376403809, 16138339.975158691 -4561431.528686523, 16138332.227294922 -4561434.391418457, 16138328.43133545 -4561428.553161621, 16138334.041809082 -4561424.519958496, 16138340.39819336 -4561421.586669922, 16138362.049804688 -4561406.765319824, 16138351.886352539 -4561395.864379883, 16138352.43182373 -4561372.088256836, 16138350.87335205 -4561365.09362793, 16138353.979125977 -4561357.647705078, 16138362.227905273 -4561348.340332031, 16138371.77911377 -4561346.225036621, 16138382.41015625 -4561361.074523926, 16138396.347351074 -4561370.381896973, 16138406.855895996 -4561383.849365234, 16138524.49835205 -4561509.542175293, 16138508.980407715 -4561517.3125, 16138514.3460083 -4561527.466186523, 16138524.79888916 -4561544.783813477, 16138537.28894043 -4561538.931335449, 16138552.584228516 -4561680.406677246, 16138540.483825684 -4561681.055419922, 16138542.754760742 -4561709.93737793, 16138560.309814453 -4561707.948913574, 16138574.057800293 -4561848.114990234, 16138602.67803955 -4561845.1111450195, 16138601.809753418 -4561828.187866211, 16138693.492492676 -4561818.499328613, 16138661.07623291 -4561520.584228516, 16138453.33178711 -4561391.267089844, 16138451.762207031 -4561372.2998046875, 16138479.469604492 -4561366.560241699, 16138493.150756836 -4561361.4552612305, 16138535.841796875 -4561345.534057617, 16138569.137451172 -4561323.943847656, 16138579.812988281 -4561316.230041504, 16138593.293762207 -4561276.8291015625, 16138600.752197266 -4561229.0517578125, 16138606.752319336 -4561179.596557617, 16138600.162231445 -4561146.048400879, 16138573.022521973 -4561052.7091674805, 16138538.680419922 -4561004.63671875, 16138500.642578125 -4560933.720092773, 16138472.901733398 -4560883.039489746, 16138454.778930664 -4560846.093811035, 16138430.299804688 -4560801.604064941, 16138348.635803223 -4560698.298400879, 16138261.639648438 -4560594.471984863, 16138160.728515625 -4560505.241088867, 16138110.022460938 -4560435.0751953125, 16138069.680297852 -4560405.928833008, 16138017.738647461 -4560356.463439941, 16137995.39678955 -4560334.085632324, 16137964.01586914 -4560317.0802612305, 16137935.184082031 -4560320.690002441, 16137886.17010498 -4560327.4865112305, 16137849.34564209 -4560326.358459473, 16137842.321350098 -4560316.967468262, 16137849.824279785 -4560312.680847168, 16138005.682739258 -4560296.733032227, 16138023.382507324 -4560294.208984375, 16138039.757629395 -4560290.007019043, 16138051.657653809 -4560288.470031738, 16138087.046142578 -4560282.5759887695, 16138087.413513184 -4560274.298950195, 16138087.257629395 -4560268.8984375, 16138088.849487305 -4560260.565002441, 16138090.307800293 -4560252.682739258, 16138093.36907959 -4560237.877197266, 16138096.675292969 -4560208.167358398, 16138102.074279785 -4560186.128295898, 16138103.343322754 -4560176.793823242, 16138105.491760254 -4560170.942077637, 16138103.410095215 -4560128.175537109, 16138096.730957031 -4560116.7119140625, 16138093.6585083 -4560084.943786621, 16138066.708068848 -4559992.601074219, 16138051.468444824 -4559962.962463379, 16138039.534973145 -4559945.210388184, 16138034.4921875 -4559934.5224609375, 16138045.256774902 -4559926.485412598, 16138054.874816895 -4559913.964477539, 16138055.420288086 -4559897.749389648, 16138041.416259766 -4559871.833496094, 16138016.736755371 -4559849.611816406, 16137966.609558105 -4559804.928955078, 16137944.512695312 -4559785.45690918, 16137928.08190918 -4559770.158508301, 16137921.324829102 -4559742.522705078, 16137923.70703125 -4559732.018310547, 16137921.013122559 -4559726.307861328, 16137914.8460083 -4559718.341491699, 16137911.250427246 -4559708.852294922, 16137915.191101074 -4559699.419494629, 16137925.788757324 -4559695.584350586, 16137931.632995605 -4559700.956359863, 16137933.736938477 -4559716.184204102, 16137938.946716309 -4559725.884887695, 16137941.562683105 -4559736.8263549805, 16137946.950561523 -4559750.305847168, 16137953.818969727 -4559746.66809082, 16137964.962036133 -4559737.827453613, 16137966.709777832 -4559729.09967041, 16137965.808105469 -4559710.290466309, 16137953.473876953 -4559681.597351074, 16137925.120788574 -4559644.571350098, 16137883.832397461 -4559627.35559082, 16137829.753417969 -4559633.347961426, 16137808.73626709 -4559632.389221191, 16137798.227722168 -4559643.866394043, 16137788.665405273 -4559637.169006348, 16137788.665405273 -4559622.843688965, 16137799.173950195 -4559614.242858887, 16137807.778930664 -4559604.683288574, 16137804.32800293 -4559586.297302246, 16137770.13067627 -4559572.944946289, 16137744.760986328 -4559565.020935059, 16137721.317077637 -4559560.100158691, 16137708.481933594 -4559563.836547852, 16137691.015930176 -4559569.34954834, 16137682.5 -4559573.3115234375, 16137674.173278809 -4559584.675842285, 16137663.4977417 -4559586.593383789, 16137648.825805664 -4559581.855895996, 16137624.43572998 -4559563.032897949, 16137600.246032715 -4559489.785583496, 16137572.327087402 -4559436.785583496, 16137549.81829834 -4559413.606079102, 16137526.341003418 -4559369.601867676, 16137480.054382324 -4559290.335510254, 16137448.116821289 -4559251.971496582, 16137416.91394043 -4559220.614929199, 16137351.669555664 -4559153.1365356445, 16137279.412109375 -4559095.950866699, 16137227.45928955 -4559045.34967041, 16137189.098571777 -4559020.704772949, 16137149.535644531 -4558998.907897949, 16137137.591064453 -4558995.326782227, 16137059.06628418 -4558935.167175293, 16136933.35321045 -4558849.179077148, 16136858.568786621 -4558798.113830566, 16136804.222595215 -4558761.838195801, 16136774.923278809 -4558739.999572754, 16136712.81817627 -4558707.502563477, 16136617.272644043 -4558656.861022949, 16136627.781188965 -4558639.675048828, 16136627.981567383 -4558628.255371094, 16136593.572692871 -4558615.087524414, 16136546.696105957 -4558586.7216796875, 16136470.1305542 -4558541.254760742, 16136453.688659668 -4558527.452575684, 16136409.940063477 -4558502.526977539, 16136322.988464355 -4558468.6349487305, 16136265.135681152 -4558441.016723633, 16136208.329345703 -4558412.270751953, 16136114.698547363 -4558357.810058594, 16136061.354248047 -4558317.095092773, 16135960.732543945 -4558262.761779785, 16135861.747253418 -4558217.521850586, 16135852.285095215 -4558230.548217773, 16135838.359008789 -4558223.217346191, 16135843.947265625 -4558210.2755737305, 16135836.655822754 -4558205.891174316, 16135765.956848145 -4558164.810241699, 16135641.746520996 -4558096.972229004, 16135562.565002441 -4558046.432250977, 16135538.08581543 -4558024.792541504, 16135428.057678223 -4557960.141418457, 16135293.394470215 -4557881.167724609, 16135211.886352539 -4557839.961303711, 16135118.578308105 -4557790.550415039, 16135016.910217285 -4557735.458557129, 16134947.491394043 -4557702.767333984, 16134922.255310059 -4557688.585632324, 16134845.378051758 -4557647.239013672, 16134787.30267334 -4557623.936645508, 16134729.817260742 -4557596.024597168, 16134668.914367676 -4557571.397216797, 16134607.254516602 -4557528.838684082, 16134607.93359375 -4557521.776184082, 16134559.20904541 -4557479.739379883, 16134502.903625488 -4557438.379455566, 16134445.57409668 -4557404.942016602, 16134377.847290039 -4557374.26763916, 16134357.075073242 -4557396.37121582, 16134315.630859375 -4557359.705810547, 16134317.85723877 -4557355.617797852, 16134318.07989502 -4557346.567749023, 16134325.003967285 -4557337.658691406, 16134328.85559082 -4557337.038452148, 16134331.48272705 -4557328.030700684, 16134312.40258789 -4557318.797424316, 16134293.58959961 -4557311.32623291, 16134253.280822754 -4557295.199768066, 16134172.073242188 -4557264.145141602, 16134080.702209473 -4557234.739868164, 16133999.271972656 -4557211.1423950195, 16133959.475280762 -4557202.501281738, 16133903.526123047 -4557192.972106934, 16133909.492797852 -4557206.47644043, 16133898.98425293 -4557221.757019043, 16133881.985778809 -4557237.037597656, 16133876.37524414 -4557240.928222656, 16133869.128356934 -4557234.316955566, 16133852.33026123 -4557235.00769043, 16133812.9899292 -4557241.830383301, 16133778.369567871 -4557229.538269043, 16133765.679138184 -4557225.506652832, 16133783.01159668 -4557156.843017578, 16133732.528198242 -4557143.987182617, 16133708.20489502 -4557136.868530273, 16133704.664978027 -4557125.873352051, 16133690.282470703 -4557121.306152344, 16133671.569641113 -4557117.2041015625, 16133655.829101562 -4557120.544921875, 16133644.608093262 -4557131.624633789, 16133635.580078125 -4557121.841796875, 16133627.587341309 -4557101.937805176, 16133607.160217285 -4557093.085388184, 16133606.670410156 -4557074.365539551, 16133615.197509766 -4557066.076904297, 16133627.65411377 -4557065.682250977, 16133629.791442871 -4557059.240234375, 16133706.93585205 -4557078.439331055, 16133726.038269043 -4557004.87121582, 16133634.255371094 -4556978.539611816, 16133622.789489746 -4556979.441772461, 16133613.416381836 -4556976.552062988, 16133607.983947754 -4556971.237854004, 16133493.870361328 -4556938.323486328, 16133482.827453613 -4556944.83581543, 16133470.66027832 -4556947.232177734, 16133461.598876953 -4556945.329223633, 16133453.606079102 -4556940.762084961, 16133445.769226074 -4556933.1502075195, 16133439.401733398 -4556923.198364258, 16133369.927246094 -4556907.890075684, 16133194.198303223 -4556861.133666992, 16133167.715393066 -4556944.469360352, 16133158.052856445 -4556939.648498535, 16133128.15246582 -4556924.692565918, 16133048.803894043 -4556917.052490234, 16132966.694641113 -4556914.515197754, 16132909.66571045 -4556915.854370117, 16132852.035583496 -4556919.293762207, 16132830.840332031 -4556921.972045898, 16132814.420715332 -4556921.972045898, 16132798.290527344 -4556929.442932129, 16132790.431396484 -4556929.0341796875, 16132791.600219727 -4556850.082458496, 16132775.30303955 -4556840.468994141, 16132733.202026367 -4556833.900268555, 16132682.150878906 -4556839.566833496, 16132503.293884277 -4556861.62701416, 16132497.41619873 -4556861.218261719, 16132498.629577637 -4556832.913574219, 16132459.701171875 -4556820.156799316, 16132376.801513672 -4556841.371154785, 16132290.40649414 -4556862.557373047, 16132282.090942383 -4556864.714050293, 16132282.06866455 -4556824.639282227, 16132251.589355469 -4556820.311828613, 16132219.640686035 -4556819.564758301, 16132163.21282959 -4556834.041259766, 16132110.358337402 -4556849.715942383, 16132036.798400879 -4556876.737915039, 16131967.936157227 -4556912.880065918, 16131901.945983887 -4556953.025695801, 16131859.254943848 -4556984.530456543, 16131813.869995117 -4557021.998046875, 16131760.726074219 -4557074.111816406, 16131709.36328125 -4557133.9646606445, 16131670.635192871 -4557159.295776367, 16132008.389709473 -4557721.347351074, 16132075.982910156 -4557834.5197143555, 16132092.524963379 -4557863.123168945, 16132092.202148438 -4557894.405151367, 16131994.864379883 -4557959.027709961, 16131838.048583984 -4558053.974487305, 16131799.36505127 -4558054.594787598, 16131670.04522705 -4558129.114868164, 16131610.13305664 -4558163.626037598, 16131630.838500977 -4558198.362915039, 16131587.57977295 -4558224.3310546875, 16131158.999694824 -4557504.90222168, 16131122.720703125 -4557526.371765137, 16131064.021911621 -4557293.9169921875, 16131055.450317383 -4557296.144226074, 16131051.24243164 -4557279.792236328, 16131041.568786621 -4557282.54107666, 16131044.440795898 -4557292.690612793, 16131039.865600586 -4557294.734619141, 16131041.323852539 -4557297.948608398, 16131031.828308105 -4557302.699157715, 16131024.737243652 -4557298.174133301, 16130941.258789062 -4557120.671813965, 16130914.04119873 -4557133.865966797, 16130892.612182617 -4557086.037231445, 16130919.406738281 -4557073.984924316, 16130920.453186035 -4557076.212158203, 16130941.403503418 -4557066.598510742, 16130927.009887695 -4557037.658935547, 16130922.824279785 -4557039.773376465, 16130914.675720215 -4557038.504699707, 16130917.981872559 -4557009.297424316, 16130921.009765625 -4557007.591796875, 16130909.688598633 -4556986.010559082, 16130914.720214844 -4556788.00402832, 16130914.286071777 -4556778.151062012, 16130881.73626709 -4556761.334716797, 16130879.39855957 -4556742.841003418, 16130875.780639648 -4556742.4462890625, 16130871.517150879 -4556741.403198242, 16130867.609802246 -4556740.021850586, 16130864.392700195 -4556737.794677734, 16130862.533630371 -4556735.285644531, 16130861.487243652 -4556730.6904296875, 16130859.973266602 -4556726.616760254, 16130858.448242188 -4556722.669921875, 16130840.102783203 -4556717.93371582, 16130818.1171875 -4556714.282958984, 16130791.077636719 -4556713.239868164, 16130773.778625488 -4556713.676818848, 16130759.585388184 -4556714.3251953125, 16130745.380981445 -4556716.284545898, 16130719.087341309 -4556722.373901367, 16130524.567687988 -4556770.976257324, 16130629.875915527 -4557109.831726074, 16130723.083740234 -4557422.281005859, 16130801.085266113 -4557679.549438477, 16130862.155151367 -4557881.999511719, 16130834.102661133 -4557890.373352051, 16130832.076599121 -4557900.833557129, 16130832.388305664 -4557912.844543457, 16130834.41430664 -4557922.021972656, 16130838.655639648 -4557929.944702148, 16130844.733642578 -4557937.458618164, 16130853.071472168 -4557943.8447265625, 16130866.340759277 -4557951.993041992, 16130928.045166016 -4557985.812866211, 16130988.66973877 -4558018.927978516, 16130992.721801758 -4558021.747436523, 16130995.137390137 -4558024.397827148, 16130996.67364502 -4558028.908996582, 16130999.545654297 -4558040.638183594, 16131001.527160645 -4558051.5919799805, 16131001.694152832 -4558056.427490234, 16131001.093017578 -4558060.572143555, 16130998.588317871 -4558064.336242676, 16130996.684753418 -4558067.705566406, 16130995.293273926 -4558071.610595703, 16130994.881408691 -4558087.301208496, 16130992.198608398 -4558105.924194336, 16130991.308044434 -4558116.906311035, 16130989.393310547 -4558126.196655273, 16130986.933166504 -4558130.595153809, 16130983.916442871 -4558136.022766113, 16130981.11114502 -4558140.717285156, 16130979.363464355 -4558145.270812988, 16130977.604614258 -4558151.924926758, 16130975.211242676 -4558155.223815918, 16130973.107299805 -4558157.564025879, 16130970.569213867 -4558157.860107422, 16130966.862243652 -4558156.478515625, 16130964.40209961 -4558154.871398926, 16130959.426147461 -4558150.585693359, 16130955.808227539 -4558145.961608887, 16130947.39251709 -4558136.459777832, 16130938.08618164 -4558126.619567871, 16130897.621520996 -4558073.231811523, 16130851.635437012 -4558016.65826416, 16130819.842590332 -4557976.268920898, 16130799.382080078 -4557952.007141113, 16130785.055297852 -4557936.443603516, 16130758.472167969 -4557908.643493652, 16130726.879699707 -4557867.634277344, 16130711.461975098 -4557846.784362793, 16130685.680358887 -4557813.373840332, 16130652.38470459 -4557770.546447754, 16130621.771850586 -4557733.386291504, 16130534.497375488 -4557621.131286621, 16130357.88897705 -4557711.084655762, 16130162.322937012 -4557811.061889648, 16130204.936035156 -4557926.575439453, 16130237.419067383 -4557990.591918945, 16130284.239990234 -4558048.885253906, 16130342.51574707 -4558104.302978516, 16130417.044189453 -4558153.024597168, 16130556.538635254 -4558228.50402832, 16130620.55847168 -4558260.040893555, 16130804.358093262 -4558500.764709473, 16130696.222351074 -4558483.564880371, 16130653.23071289 -4558468.550354004, 16130600.28717041 -4558443.850463867, 16130310.901062012 -4558275.81640625, 16130276.380859375 -4558256.023010254, 16130246.758728027 -4558235.693908691, 16130223.081115723 -4558215.012390137, 16130201.039855957 -4558193.245422363, 16130177.184082031 -4558165.261413574, 16130154.408081055 -4558135.642089844, 16130143.242736816 -4558134.923095703, 16130124.930664062 -4558126.097961426, 16130121.39074707 -4558124.3357543945, 16130117.494567871 -4558119.810424805, 16130113.208740234 -4558113.8330078125, 16130104.347717285 -4558103.3302612305, 16130096.477416992 -4558095.3650512695, 16130089.063537598 -4558088.809692383, 16130082.718322754 -4558085.186584473, 16130076.206176758 -4558083.367980957, 16130070.306213379 -4558082.648986816, 16130067.757019043 -4558082.282470703, 16130065.82006836 -4558080.999572754, 16130064.8515625 -4558079.025939941, 16130062.614013672 -4558072.287231445, 16130060.588012695 -4558062.8982543945, 16130058.305969238 -4558054.341003418, 16130055.70111084 -4558047.588256836, 16130052.472839355 -4558043.105224609, 16130045.559875488 -4558032.518005371, 16130038.646972656 -4558025.737060547, 16130031.578186035 -4558012.13293457, 16130016.750427246 -4557975.141113281, 16130004.12677002 -4557947.665161133, 16129995.243469238 -4557935.499084473, 16129989.577331543 -4557930.55090332, 16129983.031738281 -4557926.4767456055, 16129975.762573242 -4557923.220214844, 16129967.06854248 -4557922.162902832, 16129953.331726074 -4557922.5294799805, 16129944.314819336 -4557921.218383789, 16129939.650512695 -4557919.822753906, 16129935.264526367 -4557916.0587768555, 16129931.179138184 -4557914.620788574, 16129924.266174316 -4557913.182861328, 16129921.583374023 -4557914.973266602, 16129899.197021484 -4557912.322937012, 16129895.501220703 -4557909.6162109375, 16129870.69921875 -4557905.062805176, 16129843.83782959 -4557903.004577637, 16129819.937561035 -4557901.186035156, 16129795.736694336 -4557901.580749512, 16129770.58959961 -4557909.869995117, 16129746.05480957 -4557920.682678223, 16129734.544372559 -4557928.35168457, 16129722.889221191 -4557939.361755371, 16129722.86694336 -4557939.375854492, 16129702.617919922 -4557963.651672363, 16129687.812438965 -4557986.785644531, 16129674.776916504 -4558016.968383789, 16129665.98272705 -4558042.301696777, 16129663.054992676 -4558068.523193359, 16129668.209106445 -4558088.302185059, 16129676.413330078 -4558106.770080566, 16129685.018310547 -4558122.742736816, 16129736.358886719 -4558196.812194824, 16129738.696594238 -4558197.756713867, 16129740.333007812 -4558198.419311523, 16129741.824645996 -4558198.391113281, 16129744.229187012 -4558198.334716797, 16129747.824768066 -4558200.012390137, 16129764.28894043 -4558220.228637695, 16129767.005126953 -4558226.925048828, 16129771.869812012 -4558233.579223633, 16129776.3225708 -4558240.13470459, 16129783.458129883 -4558248.297363281, 16129787.766235352 -4558251.116943359, 16129790.226379395 -4558251.469421387, 16129793.421264648 -4558249.791748047, 16129796.616149902 -4558249.862243652, 16129800.879638672 -4558252.188415527, 16129885.927734375 -4558326.907287598, 16129929.264404297 -4558358.627685547, 16129959.175964355 -4558382.114990234, 16129976.040893555 -4558395.395324707, 16129992.315795898 -4558409.733093262, 16129997.202697754 -4558416.161804199, 16129998.86138916 -4558421.265319824, 16130005.696411133 -4558426.904541016, 16130013.321777344 -4558426.989135742, 16130022.539001465 -4558429.5126953125, 16130030.576293945 -4558434.545715332, 16130059.742004395 -4558465.2373046875, 16130129.049499512 -4558541.367553711, 16130141.840148926 -4558552.039916992, 16130217.003051758 -4558638.829162598, 16130232.253845215 -4558658.961669922, 16130248.161376953 -4558673.807312012, 16130254.840515137 -4558683.944030762, 16130261.987243652 -4558694.6306762695, 16130270.659057617 -4558704.725158691, 16130273.386352539 -4558706.135009766, 16130279.742736816 -4558715.651489258, 16130307.683898926 -4558756.579467773, 16130312.325927734 -4558761.626708984, 16130317.825134277 -4558766.208740234, 16130325.227844238 -4558772.595397949, 16130330.760437012 -4558781.5056762695, 16130332.541564941 -4558795.124938965, 16130366.67211914 -4558774.710205078, 16130372.226928711 -4558783.296203613, 16130363.699890137 -4558789.217651367, 16130372.839172363 -4558805.38873291, 16130376.64630127 -4558810.421936035, 16130381.177001953 -4558812.508483887, 16130385.696594238 -4558815.807556152, 16130388.401672363 -4558818.204345703, 16130390.193908691 -4558822.41986084, 16130388.90258789 -4558827.608154297, 16130388.223571777 -4558832.190185547, 16130389.592773438 -4558836.476196289, 16130393.255187988 -4558844.159912109, 16130399.255310059 -4558850.081359863, 16130405.28881836 -4558863.446899414, 16130408.739746094 -4558866.633239746, 16130415.9866333 -4558872.512390137, 16130418.35772705 -4558876.417724609, 16130420.561889648 -4558880.492248535, 16130411.923461914 -4558886.596984863, 16130399.54473877 -4558895.154907227, 16130400.156982422 -4558897.142822266, 16130415.274169922 -4558892.574829102, 16130415.819641113 -4558894.29486084, 16130473.026733398 -4559107.653015137, 16130491.750671387 -4559177.485656738, 16130527.395202637 -4559307.522521973, 16130531.202331543 -4559309.327270508, 16130532.73852539 -4559311.851013184, 16130533.183776855 -4559314.868286133, 16130530.790405273 -4559322.129394531, 16130519.73638916 -4559352.865966797, 16130519.035095215 -4559358.012207031, 16130511.086853027 -4559381.487670898, 16130501.301879883 -4559397.899353027, 16130489.72467041 -4559413.352294922, 16130482.16607666 -4559430.398498535, 16130478.581604004 -4559439.972045898, 16130475.030517578 -4559457.906555176, 16130472.75958252 -4559476.362854004, 16130473.739196777 -4559495.904785156, 16130476.588989258 -4559513.261291504, 16130483.290405273 -4559543.025512695, 16130491.327636719 -4559570.815917969, 16130511.187072754 -4559572.000244141, 16130588.453918457 -4559569.067504883, 16130590.958618164 -4559574.397216797, 16130594.977233887 -4559625.170166016, 16130595.133117676 -4559629.061645508, 16130548.37890625 -4559638.7763671875, 16130549.981933594 -4559645.8826293945, 16130579.414794922 -4559639.890258789, 16130583.400024414 -4559647.828430176, 16130585.98260498 -4559655.371765137, 16130586.884338379 -4559663.3240356445, 16130585.938110352 -4559671.473693848, 16130582.876831055 -4559675.548522949, 16130587.774902344 -4559697.910827637, 16130609.671386719 -4559690.452026367, 16130610.573120117 -4559693.568054199, 16130624.599365234 -4559739.13873291, 16130625.434265137 -4559741.860046387, 16130627.972351074 -4559748.472900391, 16130628.617980957 -4559750.883972168, 16130630.209838867 -4559756.143188477, 16130631.623596191 -4559761.8536987305, 16130639.282409668 -4559792.492797852, 16130640.941040039 -4559797.596984863, 16130644.280639648 -4559812.895446777, 16130579.414794922 -4559827.488891602, 16130585.481689453 -4559844.874206543, 16130586.583740234 -4559849.555419922, 16130594.342712402 -4559871.974487305, 16130613.923828125 -4559893.928283691, 16130615.894165039 -4559897.368713379, 16130630.477050781 -4559914.387512207, 16130664.730041504 -4559891.051879883, 16130682.886230469 -4559910.848388672, 16130690.17767334 -4559917.8279418945, 16130692.793701172 -4559920.436462402, 16130702.812438965 -4559929.559204102, 16130729.417785645 -4559946.507568359, 16130758.516723633 -4559961.312744141, 16130760.35345459 -4559954.61517334, 16130780.123840332 -4559960.509033203, 16130781.504211426 -4559953.712768555, 16130789.585998535 -4559956.448181152, 16130794.66217041 -4559958.253051758, 16130826.610839844 -4559970.407348633, 16130867.24243164 -4559985.790710449, 16130875.569152832 -4559988.935058594, 16130891.610290527 -4559944.984802246, 16130897.076049805 -4559947.142089844, 16130974.632385254 -4559980.85559082, 16131035.134521484 -4559927.782592773, 16131058.166503906 -4559937.751403809, 16131064.444946289 -4559940.472717285, 16131073.317077637 -4559944.3220825195, 16131114.316040039 -4559961.961364746, 16131117.366210938 -4559963.3291015625, 16131120.561096191 -4559964.767333984, 16131105.043151855 -4560002.612243652, 16131153.211120605 -4560022.888366699, 16131160.558166504 -4560004.854187012, 16131201.891113281 -4559903.347106934, 16131202.15826416 -4559902.754943848, 16131204.919006348 -4559903.812438965, 16131211.253051758 -4559906.2517700195, 16131218.655822754 -4559909.099975586, 16131238.32598877 -4559916.305114746, 16131241.665588379 -4559917.686950684, 16131258.196533203 -4559924.031982422, 16131275.050292969 -4559884.819641113, 16131299.095275879 -4559894.534545898, 16131343.010803223 -4559912.286560059, 16131351.615844727 -4559915.825744629, 16131397.969238281 -4559934.508361816, 16131399.472045898 -4559930.941040039, 16131412.685668945 -4559936.172180176, 16131454.731079102 -4559953.515380859, 16131456.478759766 -4559951.259338379, 16131478.609069824 -4559960.142456055, 16131564.859436035 -4560048.240661621, 16131562.454956055 -4560051.286315918, 16131594.526062012 -4560084.563110352, 16131596.440795898 -4560086.551208496, 16131605.758239746 -4560097.39440918, 16131596.930603027 -4560106.658325195, 16131597.06414795 -4560120.265197754, 16131619.50616455 -4560143.09375, 16131615.376220703 -4560147.535400391, 16131635.658630371 -4560168.107910156, 16131640.578979492 -4560163.581665039, 16131657.443847656 -4560181.602050781, 16131657.967041016 -4560190.287963867, 16131654.427124023 -4560201.751647949, 16131661.473632812 -4560205.5587768555, 16131701.704467773 -4560248.551269531, 16131707.571044922 -4560270.1251831055, 16131703.708251953 -4560289.161010742, 16131671.425598145 -4560342.743469238, 16131687.344299316 -4560350.104003906, 16131691.95288086 -4560352.416564941, 16131697.14038086 -4560355.490478516, 16131722.109375 -4560366.686462402, 16131873.114257812 -4560371.269226074, 16131902.25769043 -4560416.053222656, 16131907.389526367 -4560423.935546875, 16131914.335876465 -4560419.860412598, 16131922.0836792 -4560431.817871094, 16131929.59777832 -4560442.4921875, 16131921.894470215 -4560448.499145508, 16131812.122314453 -4560530.806030273, 16131693.711730957 -4560627.637634277, 16131691.296142578 -4560630.218139648, 16131686.119750977 -4560636.577758789, 16131682.134521484 -4560641.597717285, 16131677.648376465 -4560645.870361328, 16131673.184448242 -4560652.737609863, 16131672.705749512 -4560664.328735352, 16131671.614807129 -4560673.268859863, 16131670.412597656 -4560684.97277832, 16131668.353149414 -4560698.848327637, 16131664.980163574 -4560707.661560059, 16131658.100646973 -4560717.081176758, 16131653.870483398 -4560722.862670898, 16131649.284118652 -4560726.134155273, 16131641.55859375 -4560731.563110352, 16131633.721679688 -4560736.512634277, 16131628.645507812 -4560740.00970459, 16131624.426513672 -4560743.394042969, 16131618.003356934 -4560747.864135742, 16131610.54498291 -4560752.686767578, 16131598.789611816 -4560759.920654297, 16131561.274963379 -4560786.036254883, 16131498.167907715 -4560829.327270508, 16131470.772216797 -4560843.090209961, 16131454.96484375 -4560852.256103516, 16131435.40600586 -4560868.190673828, 16131418.151489258 -4560882.066467285, 16131401.253173828 -4560899.693237305, 16131399.316223145 -4560908.02722168, 16131396.90057373 -4560909.592468262, 16131388.095214844 -4560914.894592285, 16131380.369628906 -4560923.947753906, 16131375.538391113 -4560928.54486084, 16131372.287841797 -4560932.761169434, 16131369.74975586 -4560933.607299805, 16131367.579040527 -4560934.326477051, 16131364.56225586 -4560935.53918457, 16131360.577026367 -4560936.864746094, 16131355.990661621 -4560939.882446289, 16131349.589782715 -4560945.198730469, 16131344.40234375 -4560950.02142334, 16131264.92022705 -4561019.358764648, 16131224.399902344 -4561049.536254883, 16131217.631713867 -4561052.116882324, 16131210.551757812 -4561056.149963379, 16131197.93927002 -4561067.247924805, 16131168.194702148 -4561084.212280273, 16131121.162231445 -4561115.419372559, 16131117.188110352 -4561118.549987793, 16131113.837402344 -4561122.583068848, 16131096.04852295 -4561149.869995117, 16131093.243286133 -4561155.28503418, 16131092.041015625 -4561160.657836914, 16131090.415771484 -4561165.029418945, 16131089.046569824 -4561168.681762695, 16131086.920349121 -4561171.107299805, 16131083.046447754 -4561175.66217041, 16131080.07421875 -4561177.9607543945, 16131072.504455566 -4561179.963195801, 16131065.51361084 -4561183.516906738, 16131060.437438965 -4561188.776855469, 16131054.88256836 -4561192.133117676, 16131048.748901367 -4561195.291931152, 16131040.611450195 -4561199.028930664, 16131034.288452148 -4561201.327514648, 16131027.575927734 -4561202.667175293, 16131016.366027832 -4561207.560546875, 16131009.954040527 -4561212.439758301, 16131004.48828125 -4561213.694824219, 16130998.287780762 -4561213.793579102, 16130986.766174316 -4561213.497436523, 16130980.443237305 -4561215.034545898, 16130974.988586426 -4561218.57409668, 16130969.044128418 -4561220.773986816, 16130958.791625977 -4561220.111206055, 16130951.1328125 -4561219.2509765625, 16130946.91381836 -4561218.475402832, 16130941.069519043 -4561216.472900391, 16130934.546203613 -4561214.696044922, 16130925.284423828 -4561215.147338867, 16130913.00592041 -4561215.598571777, 16130888.671447754 -4561222.508483887, 16130871.695251465 -4561227.980041504, 16130868.589416504 -4561229.235107422, 16130864.915893555 -4561233.225952148, 16130863.379638672 -4561237.738525391, 16130861.94366455 -4561241.658874512, 16130857.24597168 -4561242.815246582, 16130853.516784668 -4561242.3357543945, 16130847.961914062 -4561238.4013671875, 16130844.911743164 -4561237.343688965, 16130841.984069824 -4561237.837280273, 16130819.887145996 -4561243.365234375, 16130816.458496094 -4561245.283081055, 16130795.55267334 -4561258.736328125, 16130778.988342285 -4561270.976806641, 16130776.706298828 -4561273.0498046875, 16130774.802734375 -4561276.208618164, 16130763.45928955 -4561301.437011719, 16130761.878540039 -4561304.609985352, 16130760.32006836 -4561308.3046875, 16130747.039672852 -4561352.641479492, 16130743.444030762 -4561359.551513672, 16130735.840942383 -4561369.817810059, 16130697.825317383 -4561412.899780273, 16130691.302001953 -4561417.130432129, 16130681.884399414 -4561421.473876953, 16130622.384094238 -4561448.634643555, 16130614.6585083 -4561450.679443359, 16130605.007141113 -4561453.217834473, 16130532.649475098 -4561460.424072266, 16130525.870117188 -4561460.35357666, 16130517.721496582 -4561459.676635742, 16130444.818359375 -4561435.886291504, 16130378.739135742 -4561409.374206543, 16130369.622070312 -4561406.539672852, 16130359.325012207 -4561403.70513916, 16130336.226196289 -4561399.375793457, 16130309.008605957 -4561395.568237305, 16130261.864807129 -4561376.403503418, 16130252.280212402 -4561368.196044922, 16130245.901611328 -4561361.96295166, 16130240.569396973 -4561353.600402832, 16130232.053466797 -4561345.237915039, 16130223.693359375 -4561338.384338379, 16130215.477966309 -4561335.197265625, 16130206.661437988 -4561332.306335449, 16130195.562927246 -4561330.021789551, 16130184.152648926 -4561328.047546387, 16130178.074645996 -4561325.622009277, 16130169.258117676 -4561322.124694824, 16130154.051879883 -4561311.139221191, 16130143.598999023 -4561304.116394043, 16130135.684143066 -4561298.898681641, 16130126.277648926 -4561292.3271484375, 16130116.72644043 -4561284.712097168, 16130112.496276855 -4561279.564880371, 16130108.967468262 -4561273.670288086, 16130107.475769043 -4561266.055236816, 16130104.191894531 -4561253.518615723, 16130101.787353516 -4561246.354797363, 16130094.30670166 -4561239.741027832, 16130087.705444336 -4561235.0451049805, 16130078.88897705 -4561230.927307129, 16130069.014892578 -4561224.327636719, 16130063.938720703 -4561219.631713867, 16130059.051818848 -4561211.776977539, 16130054.4765625 -4561205.882385254, 16130051.938476562 -4561198.2673950195, 16130043.422546387 -4561188.875549316, 16130037.901123047 -4561184.236083984, 16130032.223815918 -4561178.41204834, 16130027.125366211 -4561173.716125488, 16130016.516662598 -4561168.568969727, 16130007.744689941 -4561165.579345703, 16129991.770324707 -4561159.4591674805, 16129984.49005127 -4561157.513122559, 16129974.905395508 -4561155.28503418, 16129964.753051758 -4561153.183898926, 16129951.461547852 -4561149.009765625, 16129943.045776367 -4561145.822753906, 16129932.448181152 -4561143.848510742, 16129923.30883789 -4561141.169189453, 16129913.534973145 -4561140.407653809, 16129906.521850586 -4561138.5603637695, 16129899.653442383 -4561137.319396973, 16129890.547485352 -4561136.473266602, 16129882.92211914 -4561136.995056152, 16129869.207580566 -4561128.519897461, 16129837.737548828 -4561109.86328125, 16129801.402832031 -4561088.0197143555, 16129791.718078613 -4561084.818664551, 16129782.378356934 -4561083.84564209, 16129771.36883545 -4561084.452026367, 16129760.448425293 -4561086.567260742, 16129748.281188965 -4561088.569702148, 16129735.356994629 -4561087.906921387, 16129725.104492188 -4561085.410949707, 16129714.75177002 -4561078.134460449, 16129709.965026855 -4561072.000244141, 16129700.95928955 -4561061.565002441, 16129693.200317383 -4561052.652709961, 16129688.023986816 -4561047.477416992, 16129681.222351074 -4561038.381896973, 16129678.161071777 -4561032.050231934, 16129676.43560791 -4561021.713745117, 16129672.984680176 -4561012.900268555, 16129643.618591309 -4560983.089538574, 16129573.275817871 -4560901.399536133, 16129550.822692871 -4560913.695983887, 16129545.145385742 -4560900.299621582, 16129534.525512695 -4560877.850158691, 16129526.56616211 -4560868.063781738, 16129517.872131348 -4560857.685119629, 16129507.497131348 -4560847.673156738, 16129498.803100586 -4560839.705871582, 16129484.476257324 -4560828.185058594, 16129471.195861816 -4560819.230712891, 16129454.38659668 -4560804.100036621, 16129439.291687012 -4560790.943481445, 16129416.359863281 -4560775.855102539, 16129397.780639648 -4560765.589416504, 16129385.223815918 -4560759.5681762695, 16129366.510986328 -4560750.515136719, 16129331.389709473 -4560737.471496582, 16129314.135192871 -4560733.001403809, 16129289.633789062 -4560727.69934082, 16129258.130371094 -4560720.57824707, 16129242.679199219 -4560717.56060791, 16129221.317016602 -4560713.809692383, 16129201.401916504 -4560711.045837402, 16129182.811584473 -4560709.466491699, 16129165.434631348 -4560708.380737305, 16129143.70501709 -4560708.028198242, 16129124.157348633 -4560708.380737305, 16129106.891662598 -4560708.5076293945, 16129089.993408203 -4560712.611083984, 16129070.323242188 -4560718.519470215, 16129055.840576172 -4560724.794494629, 16129042.437683105 -4560731.67590332, 16129029.279724121 -4560741.9415893555, 16129013.594848633 -4560753.278991699, 16129001.037963867 -4560768.607055664, 16128992.711303711 -4560779.71887207, 16128987.768676758 -4560788.532165527, 16128984.017211914 -4560798.783813477, 16128981.612731934 -4560807.23046875, 16128979.798217773 -4560816.043823242, 16128979.19708252 -4560825.336608887, 16128981.000488281 -4560838.140625, 16128981.690673828 -4560839.861022949, 16129032.663879395 -4560849.717834473, 16129051.85534668 -4560855.513549805, 16129055.239440918 -4560860.350280762, 16129053.78112793 -4560864.326904297, 16129048.960998535 -4560864.6935424805, 16128985.341918945 -4560852.890686035, 16128929.760131836 -4560843.908081055, 16128908.175292969 -4560840.410949707, 16128881.11352539 -4560861.154052734, 16128872.085510254 -4560863.509033203, 16128862.356140137 -4560863.241088867, 16128815.101013184 -4560858.629943848, 16128532.015563965 -4560830.878417969, 16128540.642822266 -4560766.802124023, 16128535.288391113 -4560764.997131348, 16128524.75756836 -4560761.443603516, 16128511.989196777 -4560757.1427612305, 16128499.187438965 -4560748.86529541, 16128477.35772705 -4560734.764099121, 16128476.522827148 -4560761.598754883, 16128454.303405762 -4560737.415100098, 16128438.284545898 -4560719.774475098, 16128434.143493652 -4560735.20123291, 16128425.56072998 -4560735.017883301, 16128410.476928711 -4560731.436218262, 16128400.947998047 -4560727.445556641, 16128387.634216309 -4560719.35144043, 16128380.776916504 -4560712.216247559, 16128374.498474121 -4560705.913024902, 16128370.179321289 -4560698.326599121, 16128386.532104492 -4560654.345153809, 16128392.39868164 -4560624.7892456055, 16128393.278076172 -4560600.338012695, 16128402.8515625 -4560574.349853516, 16128407.638305664 -4560551.506225586, 16128393.712219238 -4560526.096313477, 16128381.110900879 -4560515.6193237305, 16128362.98803711 -4560509.781555176, 16128283.606140137 -4560498.035522461, 16128243.108093262 -4560507.680541992, 16128204.580444336 -4560516.352600098, 16128175.670776367 -4560513.23626709, 16128150.323303223 -4560506.481933594, 16128105.227783203 -4560482.327087402, 16128077.854309082 -4560477.081604004, 16128061.96899414 -4560481.39642334, 16128051.74987793 -4560493.170715332, 16128049.85748291 -4560503.168212891, 16128053.764770508 -4560514.082336426, 16128055.712890625 -4560519.186889648, 16128052.339904785 -4560525.757873535, 16128043.122619629 -4560534.16204834, 16128029.218811035 -4560541.452270508, 16128013.845581055 -4560544.949279785, 16128002.324035645 -4560547.431091309, 16127987.106689453 -4560543.778930664, 16127968.939331055 -4560543.764831543, 16127947.844299316 -4560546.528625488, 16127925.335510254 -4560561.588439941, 16127913.902954102 -4560570.4861450195, 16127903.973266602 -4560578.1994018555, 16127899.932373047 -4560582.866821289, 16127899.921264648 -4560589.212280273, 16127900.477844238 -4560597.433227539, 16127898.329345703 -4560608.474304199, 16127891.327392578 -4560620.432006836, 16127887.78741455 -4560633.010192871, 16127896.526000977 -4560646.490844727, 16127905.353637695 -4560673.395751953, 16127931.54711914 -4560693.659118652, 16127939.061157227 -4560701.950561523, 16127937.135375977 -4560713.922485352, 16127920.081176758 -4560733.861572266, 16127908.492858887 -4560749.979309082, 16127895.880371094 -4560772.104187012, 16127887.954406738 -4560791.733154297, 16127869.987426758 -4560804.932006836, 16127860.92602539 -4560816.537353516, 16127838.127807617 -4560805.030700684, 16127815.396362305 -4560804.53717041, 16127797.852416992 -4560791.056274414, 16127792.932067871 -4560760.456542969, 16127775.521728516 -4560762.7127075195, 16127755.640075684 -4560745.622009277, 16127756.129882812 -4560731.7041015625, 16127762.942626953 -4560720.860290527, 16127777.859436035 -4560706.716796875, 16127786.809509277 -4560684.126708984, 16127782.022766113 -4560665.358093262, 16127770.078186035 -4560650.171203613, 16127761.963012695 -4560640.779846191, 16127751.688232422 -4560631.543640137, 16127736.504211426 -4560631.5859375, 16127721.086486816 -4560636.46496582, 16127712.949035645 -4560647.379211426, 16127709.798706055 -4560661.466186523, 16127708.273620605 -4560681.884643555, 16127699.256713867 -4560691.656738281, 16127692.132263184 -4560689.936401367, 16127684.985595703 -4560670.857543945, 16127683.827819824 -4560667.938598633, 16127673.664367676 -4560642.232299805, 16127620.364624023 -4560532.159729004, 16127576.838684082 -4560438.684997559, 16127527.691101074 -4560367.391479492, 16127440.672668457 -4560279.544372559, 16127341.13079834 -4560168.841125488, 16127322.217590332 -4560137.890686035, 16127323.60913086 -4560120.547180176, 16127329.664855957 -4560086.565307617, 16127343.56866455 -4560069.489807129, 16127351.650512695 -4560051.836242676, 16127359.46508789 -4560024.735534668, 16127366.778808594 -4559990.612976074, 16127374.70477295 -4559957.9287109375, 16127375.127746582 -4559929.784851074, 16127372.311401367 -4559911.271362305, 16127360.667358398 -4559897.960876465, 16127323.909667969 -4559861.173828125, 16127295.100158691 -4559836.893615723, 16127271.02178955 -4559816.589599609, 16127259.255310059 -4559809.581970215, 16127249.982421875 -4559800.205444336, 16127249.670715332 -4559794.128417969, 16127249.102966309 -4559782.820251465, 16127250.015808105 -4559768.804931641, 16127243.436828613 -4559757.144287109, 16127230.31225586 -4559743.058532715, 16127150.763305664 -4559685.770874023, 16127118.803527832 -4559665.058288574, 16127070.757995605 -4559638.635375977, 16127022.701416016 -4559614.567138672, 16127002.708435059 -4559605.994567871, 16126980.310913086 -4559596.378540039, 16126962.377380371 -4559589.1736450195, 16126947.00415039 -4559589.906799316, 16126926.910949707 -4559590.625915527, 16126909.166625977 -4559588.821166992, 16126895.663574219 -4559585.789733887, 16126885.31085205 -4559579.416687012, 16126875.459106445 -4559574.552307129, 16126867.744628906 -4559574.876586914, 16126858.63873291 -4559576.8787231445, 16126851.915039062 -4559579.13470459, 16126839.63647461 -4559574.834289551, 16126813.242614746 -4559564.485168457, 16126689.588928223 -4559543.716430664, 16126675.017211914 -4559541.263061523, 16126656.972351074 -4559537.851013184, 16126595.557373047 -4559538.090698242, 16126514.260742188 -4559544.9853515625, 16126475.2210083 -4559552.937561035, 16126452.133361816 -4559560.057861328, 16126301.718444824 -4559623.351257324, 16126207.53100586 -4559665.185241699, 16126190.543640137 -4559665.283935547, 16126062.28137207 -4559682.203674316, 16126044.303222656 -4559682.203674316, 16126030.032104492 -4559679.552856445, 16126020.169189453 -4559682.020324707, 16126000.432250977 -4559685.545288086, 16125991.448730469 -4559686.250305176, 16125979.648864746 -4559683.782836914, 16125965.555847168 -4559684.135314941, 16125950.394104004 -4559691.721008301, 16125931.291687012 -4559697.671142578, 16125920.738647461 -4559695.781738281, 16125913.7477417 -4559695.006286621, 16125909.083496094 -4559693.4552612305, 16125904.875610352 -4559688.788269043, 16125902.760498047 -4559685.009521484, 16125898.775268555 -4559677.9454956055, 16125895.235351562 -4559673.377197266, 16125892.029296875 -4559669.231872559, 16125875.09765625 -4559680.751342773, 16125861.505493164 -4559680.088684082, 16125848.38092041 -4559687.138549805, 16125840.722167969 -4559694.442260742, 16125832.618103027 -4559706.779602051, 16125828.29888916 -4559716.381591797, 16125827.063293457 -4559721.936950684, 16125828.120788574 -4559726.33605957, 16125830.14678955 -4559729.861022949, 16125828.822143555 -4559738.574768066, 16125826.18383789 -4559747.824279785, 16125824.3359375 -4559750.122558594, 16125814.907165527 -4559752.152954102, 16125806.691772461 -4559753.280944824, 16125780.420410156 -4559756.8623046875, 16125771.147460938 -4559759.061889648, 16125757.555358887 -4559769.05871582, 16125753.325256348 -4559773.288696289, 16125751.04321289 -4559781.748657227, 16125749.985656738 -4559791.872375488, 16125745.221191406 -4559801.7423706055, 16125743.81854248 -4559806.5927734375, 16125735.792419434 -4559814.347717285, 16125722.4118042 -4559827.291503906, 16125717.825439453 -4559833.805725098, 16125716.767944336 -4559840.066101074, 16125710.511779785 -4559847.384033203, 16125697.209106445 -4559850.641113281, 16125684.262634277 -4559851.430725098, 16125677.216125488 -4559855.124938965, 16125673.954467773 -4559859.270324707, 16125672.718811035 -4559864.120727539, 16125672.451599121 -4559871.960388184, 16125667.876403809 -4559875.739196777, 16125661.353088379 -4559876.274963379, 16125658.447631836 -4559876.007080078, 16125654.039367676 -4559872.580749512, 16125644.96685791 -4559870.6349487305, 16125636.161499023 -4559871.777099609, 16125629.638122559 -4559873.807495117, 16125625.062927246 -4559874.597106934, 16125616.869812012 -4559878.305419922, 16125608.409545898 -4559880.772888184, 16125602.420532227 -4559883.141723633, 16125598.10131836 -4559885.524597168, 16125594.928710938 -4559890.107177734, 16125594.227416992 -4559896.184265137, 16125594.049316406 -4559900.061828613, 16125595.808166504 -4559905.5185546875, 16125594.227416992 -4559910.538146973, 16125590.698608398 -4559912.484008789, 16125588.149353027 -4559916.262817383, 16125584.364501953 -4559915.473205566, 16125581.013793945 -4559915.571899414, 16125573.878234863 -4559918.4765625, 16125570.616577148 -4559921.550354004, 16125568.67956543 -4559925.174072266, 16125568.768676758 -4559928.600402832, 16125568.768676758 -4559933.182983398, 16125568.67956543 -4559937.328430176, 16125549.120788574 -4559947.367675781, 16125513.3649292 -4559969.603637695, 16125501.643005371 -4559971.591796875, 16125491.067626953 -4559975.652648926, 16125482.696411133 -4559982.688659668, 16125476.618347168 -4559989.654174805, 16125469.215637207 -4559996.083862305, 16125464.028137207 -4559998.819274902, 16125456.269165039 -4559999.608886719, 16125449.22265625 -4560002.428955078, 16125444.380249023 -4560005.333618164, 16125437.066589355 -4560015.288330078, 16125427.459716797 -4560022.606384277, 16125419.010559082 -4560025.158508301, 16125413.010437012 -4560029.036071777, 16125405.529724121 -4560035.4658203125, 16125400.854309082 -4560043.305603027, 16125396.891357422 -4560049.904541016, 16125390.991455078 -4560056.954650879, 16125387.90789795 -4560061.184753418, 16125365.532653809 -4560083.3927612305, 16125357.072387695 -4560090.174987793, 16125351.083374023 -4560098.014831543, 16125344.916259766 -4560109.901428223, 16125343.157409668 -4560114.836547852, 16125342.188964844 -4560120.293395996, 16125345.016479492 -4560129.966247559, 16125345.172302246 -4560135.310302734, 16125344.125915527 -4560138.9482421875, 16125343.21307373 -4560141.486328125, 16125331.235107422 -4560149.946533203, 16125318.333190918 -4560153.274230957, 16125298.818908691 -4560149.368408203, 16125270.643920898 -4560143.34753418, 16125223.633728027 -4560137.707397461, 16125158.422729492 -4560132.208251953, 16125115.653808594 -4560127.667907715, 16125058.969909668 -4560129.261230469, 16124997.499267578 -4560135.874328613, 16124997.310058594 -4560143.967956543, 16124970.760375977 -4560159.760437012, 16124915.37890625 -4560176.060546875, 16124776.852905273 -4560203.006591797, 16124712.5881958 -4560254.699157715, 16124604.563720703 -4560320.083679199, 16124542.269348145 -4560358.775939941, 16124514.695495605 -4560396.32623291, 16124481.288574219 -4560415.531494141, 16124494.123657227 -4560428.842651367, 16124485.440734863 -4560435.131591797, 16124477.75970459 -4560441.899963379, 16124462.698181152 -4560455.168884277, 16124445.532714844 -4560470.299072266, 16124437.094726562 -4560461.077087402, 16124433.343261719 -4560464.433105469, 16124432.675354004 -4560465.02532959, 16124429.858947754 -4560467.53527832, 16124425.617675781 -4560471.328430176, 16124416.92364502 -4560479.08392334, 16124416.021911621 -4560479.887634277, 16124412.904968262 -4560482.665527344, 16124412.660095215 -4560482.87701416, 16124408.340881348 -4560486.740661621, 16124405.58013916 -4560489.208312988, 16124402.296264648 -4560492.141296387, 16124380.355163574 -4560511.374938965, 16124377.93951416 -4560513.786193848, 16124374.56652832 -4560516.95892334, 16124375.946899414 -4560518.43951416, 16124367.186096191 -4560526.152709961, 16124353.059631348 -4560538.589782715, 16124339.912780762 -4560550.166625977, 16124333.233642578 -4560559.00793457, 16124328.302185059 -4560565.776428223, 16124319.274169922 -4560568.413330078, 16124314.676635742 -4560574.688293457, 16124300.01586914 -4560585.390930176, 16124291.9118042 -4560596.051330566, 16124286.368103027 -4560605.3720703125, 16124287.369995117 -4560612.3662109375, 16124286.913574219 -4560618.570678711, 16124281.23626709 -4560621.856201172, 16124272.27508545 -4560629.682312012, 16124267.41040039 -4560635.224060059, 16124228.793701172 -4560689.8377075195, 16124226.43371582 -4560718.928405762, 16124191.612976074 -4560766.56237793, 16124165.463989258 -4560801.815612793, 16124153.44152832 -4560811.6865234375, 16124138.869812012 -4560819.174316406, 16124122.20526123 -4560820.513977051, 16124119.778503418 -4560820.8947143555, 16124117.518737793 -4560821.162597656, 16124101.132507324 -4560819.625549316, 16124078.712768555 -4560830.173339844, 16124059.977661133 -4560843.583740234, 16124037.313049316 -4560861.478393555, 16124013.991577148 -4560886.0712890625, 16123985.104187012 -4560910.66418457, 16123982.844421387 -4560914.951049805, 16123973.382263184 -4560917.038024902, 16123959.623168945 -4560925.654052734, 16123889.803588867 -4560971.935180664, 16123883.024230957 -4560972.809509277, 16123873.18359375 -4560977.589904785, 16123871.613952637 -4560987.277709961, 16123867.417236328 -4560997.303894043, 16123866.014587402 -4561008.994140625, 16123856.519042969 -4561017.553771973, 16123856.06262207 -4561025.055847168, 16123800.469665527 -4561058.0818481445, 16123787.25604248 -4561058.970275879, 16123763.489318848 -4561062.749511719, 16123736.216064453 -4561063.440490723, 16123724.137878418 -4561064.455810547, 16123694.460144043 -4561100.612548828, 16123668.055114746 -4561138.179626465, 16123636.039672852 -4561183.714294434, 16123574.903015137 -4561259.6388549805, 16123561.333129883 -4561272.415161133, 16123552.34967041 -4561279.014892578, 16123537.499633789 -4561284.810791016, 16123525.889038086 -4561286.221008301, 16123468.414733887 -4561282.018615723, 16123386.327758789 -4561274.192016602, 16123302.960632324 -4561266.619262695, 16123267.037780762 -4561263.841186523, 16123217.823486328 -4561262.8681640625, 16123152.935302734 -4561275.306091309, 16123085.24194336 -4561288.872192383, 16123052.124389648 -4561296.26159668, 16123046.36920166 -4561297.544921875, 16123019.09588623 -4561303.270324707, 16123009.032592773 -4561307.331665039, 16123002.87664795 -4561308.586730957, 16122996.731811523 -4561308.727783203, 16122992.7131958 -4561308.614929199, 16122987.603637695 -4561308.699584961, 16122983.295532227 -4561311.449462891, 16122981.069152832 -4561315.496704102, 16122979.043151855 -4561320.093994141, 16122977.37335205 -4561327.8923950195, 16122976.705444336 -4561336.8048706055, 16122969.447387695 -4561350.5966796875, 16122953.094604492 -4561339.075317383, 16122920.511352539 -4561316.286437988, 16122897.713134766 -4561310.828979492, 16122871.842468262 -4561297.121826172, 16122836.331542969 -4561287.053039551, 16122791.859436035 -4561275.4470825195, 16122759.832824707 -4561267.7615356445, 16122742.2109375 -4561264.1232299805, 16122729.487121582 -4561261.4580078125, 16122712.421813965 -4561259.779846191, 16122698.017089844 -4561258.3837890625, 16122663.341064453 -4561255.450561523, 16122640.409240723 -4561254.745483398, 16122616.909729004 -4561254.1954956055, 16122598.731262207 -4561253.913452148, 16122577.335632324 -4561253.208374023, 16122553.568908691 -4561249.302124023, 16122527.408813477 -4561247.891906738, 16122503.475158691 -4561246.876586914, 16122481.077697754 -4561247.553466797, 16122468.264770508 -4561249.161071777, 16122454.550231934 -4561249.71105957, 16122434.134216309 -4561248.455993652, 16122408.831298828 -4561252.094299316, 16122391.0647583 -4561251.812255859, 16122361.565063477 -4561250.275146484, 16122322.636657715 -4561245.480529785, 16122305.059265137 -4561244.408752441, 16122289.263061523 -4561243.280578613, 16122277.69696045 -4561243.1677856445, 16122270.23852539 -4561242.349853516, 16122264.405395508 -4561242.321655273, 16122234.438232422 -4561241.602478027, 16122215.424865723 -4561240.065368652, 16122203.814208984 -4561239.501281738, 16122191.368713379 -4561239.233337402, 16122165.754089355 -4561238.514160156, 16122141.096801758 -4561239.289733887, 16122135.742370605 -4561239.614135742, 16122131.245056152 -4561240.5166015625, 16122123.40814209 -4561242.307556152, 16122115.014648438 -4561242.7306518555, 16122099.351989746 -4561242.998535156, 16122090.390808105 -4561245.099731445, 16122082.409179688 -4561244.084411621, 16122072.390441895 -4561244.958740234, 16122063.206542969 -4561246.495849609, 16122049.848205566 -4561247.200927734, 16122041.744140625 -4561247.765014648, 16122036.979675293 -4561247.623962402, 16122030.456359863 -4561247.736816406, 16122020.059143066 -4561247.454772949, 16122009.005126953 -4561247.511169434, 16121998.06237793 -4561247.891906738, 16121988.778381348 -4561248.766235352, 16121969.375366211 -4561250.54309082, 16121945.44165039 -4561254.421142578, 16121917.99029541 -4561258.778625488, 16121889.247619629 -4561262.656616211, 16121870.434631348 -4561267.0564575195, 16121855.473266602 -4561269.8486328125, 16121834.088806152 -4561274.333068848, 16121821.754577637 -4561276.955993652, 16121810.856384277 -4561281.186584473, 16121803.008361816 -4561282.103210449, 16121800.370117188 -4561274.883056641, 16121798.13256836 -4561268.0294799805, 16121793.802246094 -4561259.5119018555, 16121783.594299316 -4561257.2697143555, 16121772.540222168 -4561255.873596191, 16121759.671691895 -4561256.282592773, 16121714.921264648 -4561260.3439331055, 16121656.612121582 -4561274.883056641, 16121590.187805176 -4561291.39642334, 16121526.42401123 -4561310.264892578, 16121487.11706543 -4561324.2540893555, 16121451.606140137 -4561342.290588379, 16121332.739196777 -4561391.9298706055, 16121307.425170898 -4561405.355102539, 16121307.848144531 -4561410.121643066, 16121134.824279785 -4561494.988647461, 16121050.243713379 -4561542.259521484, 16121014.755065918 -4561560.042541504, 16120978.821166992 -4561582.225524902, 16120959.228942871 -4561595.989440918, 16120939.469726562 -4561610.966186523, 16120935.473327637 -4561616.296936035, 16120931.476989746 -4561624.730163574, 16120929.150390625 -4561629.510864258, 16120926.846069336 -4561636.872375488, 16120883.531677246 -4561660.479858398, 16120808.724975586 -4561704.5361328125, 16120764.876220703 -4561731.838623047, 16120738.794067383 -4561752.047668457, 16120726.24835205 -4561761.693847656, 16120723.476501465 -4561766.136169434, 16120712.489257812 -4561773.906738281, 16120681.965454102 -4561794.6658325195, 16120652.332214355 -4561811.306945801, 16120644.784729004 -4561813.08392334, 16120639.563842773 -4561816.637756348, 16120637.571228027 -4561819.303161621, 16120636.569396973 -4561822.518615723, 16120625.13684082 -4561831.741760254, 16120596.728149414 -4561854.8279418945, 16120557.543640137 -4561891.678466797, 16120543.227966309 -4561896.0079956055, 16120523.023498535 -4561907.882568359, 16120507.594604492 -4561918.205810547, 16120492.499694824 -4561930.404785156, 16120480.510559082 -4561943.844787598, 16120468.421264648 -4561957.383544922, 16120418.360900879 -4562007.660461426, 16120393.492126465 -4562034.413757324, 16120372.296875 -4562062.492858887, 16120364.860778809 -4562076.031738281, 16120361.643615723 -4562088.456542969, 16120364.415466309 -4562102.672424316, 16120366.630737305 -4562114.66003418, 16120366.630737305 -4562121.979553223, 16120364.526794434 -4562126.873352051, 16120359.194580078 -4562134.418518066, 16120340.771240234 -4562150.1716918945, 16120313.687194824 -4562170.155883789, 16120286.714477539 -4562192.0158081055, 16120275.460083008 -4562208.826843262, 16120265.6305542 -4562224.326293945, 16120259.752868652 -4562233.972900391, 16120244.880615234 -4562257.948486328, 16120228.394165039 -4562280.358581543, 16120198.11529541 -4562313.177062988, 16120176.396850586 -4562333.288513184, 16120104.695983887 -4562420.772155762, 16120088.821838379 -4562438.542602539, 16120062.962280273 -4562468.286987305, 16120011.343444824 -4562509.455444336, 16119999.910949707 -4562521.4435424805, 16119991.584228516 -4562534.108581543, 16119984.482055664 -4562539.214111328, 16120019.514282227 -4562587.124389648, 16120023.944824219 -4562604.443786621, 16120017.276794434 -4562642.340637207, 16119984.771484375 -4562707.923583984, 16119983.36883545 -4562710.758483887, 16119998.063049316 -4562727.344665527, 16119989.981262207 -4562766.821655273, 16119903.986938477 -4563103.150512695, 16119891.574829102 -4563537.8865356445, 16119954.626159668 -4563483.427185059, 16119977.557983398 -4563401.25201416, 16119992.853271484 -4563303.7880859375, 16119985.205627441 -4563215.887878418, 16119973.739746094 -4563149.004150391, 16119996.671569824 -4563162.389343262, 16120018.645996094 -4563275.127319336, 16120013.870422363 -4563361.123840332, 16120009.094787598 -4563423.227416992, 16119989.680664062 -4563510.847229004, 16119977.557983398 -4563540.7498168945, 16119980.430053711 -4563601.895324707, 16120004.319213867 -4563649.669433594, 16119975.053283691 -4563779.649475098, 16119717.070373535 -4564139.850036621, 16119417.053222656 -4564558.3505859375, 16119334.888305664 -4564774.279663086, 16119242.203674316 -4564871.743347168, 16119193.468017578 -4565040.871582031, 16119183.582824707 -4565075.180053711, 16119162.910827637 -4565146.914978027, 16119015.768737793 -4565649.482421875, 16118805.56414795 -4566217.989685059, 16118786.450561523 -4566302.063537598, 16118711.922180176 -4566504.625305176, 16118724.612609863 -4566525.563110352, 16118731.035705566 -4566536.159057617, 16118726.260131836 -4566605.9005737305, 16118790.279968262 -4566644.122436523, 16118702.37097168 -4567034.899475098, 16118698.363464355 -4567140.554016113, 16118696.638000488 -4567185.8751831055, 16118771.166381836 -4567450.525390625, 16118818.93359375 -4567749.598388672, 16118781.88647461 -4567868.157531738, 16118771.166381836 -4567902.461242676, 16118854.288635254 -4568223.504516602, 16118935.507385254 -4568389.752197266, 16118968.947753906 -4568569.380554199, 16118936.46472168 -4568601.866088867, 16118728.174804688 -4568623.838378906, 16118575.299743652 -4568728.944580078, 16118513.194580078 -4568881.821899414, 16118406.183166504 -4569061.444641113, 16118293.438781738 -4569163.676330566, 16118034.197998047 -4569224.459777832, 16117662.825012207 -4569243.9353637695, 16117108.665466309 -4569211.447814941, 16116866.144836426 -4569237.471740723, 16116663.409790039 -4569259.219543457, 16116200.99975586 -4569423.395080566, 16116178.991882324 -4569431.213684082, 16115634.383544922 -4569753.19342041, 16115588.998596191 -4569787.362243652, 16115282.803161621 -4570021.239990234, 16115224.482910156 -4570061.817321777, 16114729.55645752 -4570579.669006348, 16114591.019348145 -4570587.319091797, 16114361.701171875 -4570696.242126465, 16114010.098571777 -4570787.960205078, 16113984.29473877 -4570818.533081055, 16113747.340026855 -4570851.985473633, 16113409.886108398 -4570834.567626953, 16113225.463134766 -4570839.451416016, 16112959.47631836 -4570876.263244629, 16112637.095092773 -4570965.682189941, 16112524.350708008 -4571048.8064575195, 16112539.634887695 -4571128.106140137, 16112490.832397461 -4571142.207397461, 16112394.57446289 -4571170.028808594, 16112288.353393555 -4571200.729858398, 16111915.72253418 -4571378.444763184, 16111679.725219727 -4571516.03125, 16111647.231079102 -4571755.848571777, 16111503.918334961 -4571930.69329834, 16111258.358703613 -4572071.152770996, 16111153.26196289 -4572221.157104492, 16110866.625366211 -4572357.79486084, 16110608.642456055 -4572524.99810791, 16110506.406677246 -4572620.544067383, 16110496.85546875 -4572704.627502441, 16110372.645141602 -4572789.657470703, 16110281.87524414 -4572942.538146973, 16110324.877929688 -4573061.961425781, 16110133.78692627 -4573210.060058594, 16110019.127868652 -4573405.9375, 16109818.474487305 -4573577.915649414, 16109770.70727539 -4573689.707946777, 16109449.595092773 -4574048.064208984, 16109272.296508789 -4574137.213928223, 16109246.893432617 -4574154.270263672, 16109182.194519043 -4574197.701782227, 16109175.470825195 -4574203.744934082, 16109067.379577637 -4574300.831298828, 16109057.627990723 -4574309.585510254, 16109008.84777832 -4574353.398925781, 16108987.530151367 -4574369.368347168, 16108937.82598877 -4574406.602233887, 16108882.956604004 -4574447.690917969, 16108872.069519043 -4574455.852233887, 16108874.262512207 -4574476.919128418, 16108870.299560547 -4574499.96282959, 16108840.85559082 -4574525.731689453, 16108830.436035156 -4574541.277832031, 16108828.231933594 -4574556.583862305, 16108822.599182129 -4574571.720581055, 16108813.593444824 -4574586.235961914, 16108810.66571045 -4574600.906677246, 16108867.806030273 -4574670.278869629, 16108873.472167969 -4574672.2556762695, 16108881.008483887 -4574672.961669922, 16108890.125549316 -4574667.539550781, 16108931.959411621 -4574630.558898926, 16108934.163574219 -4574629.796386719, 16108937.180297852 -4574630.756591797, 16108939.1618042 -4574633.029907227, 16108972.769165039 -4574671.592041016, 16108972.813659668 -4574674.048950195, 16108971.76727295 -4574676.6611328125, 16108922.731018066 -4574720.038452148, 16108920.526916504 -4574724.076782227, 16108919.769958496 -4574728.835327148, 16108920.727294922 -4574733.269104004, 16108968.539001465 -4574792.6025390625, 16108971.02142334 -4574797.798828125, 16108972.179138184 -4574803.418762207, 16108972.123535156 -4574808.459716797, 16108971.032592773 -4574813.6701049805, 16108968.962036133 -4574818.541625977, 16108966.268127441 -4574822.45300293, 16108962.839477539 -4574825.743041992, 16108959.076843262 -4574828.199951172, 16108955.069335938 -4574830.162719727, 16108951.852233887 -4574830.7275390625, 16108948.657348633 -4574831.108764648, 16108945.440246582 -4574831.1228637695, 16108942.534790039 -4574830.868713379, 16108938.326904297 -4574829.724975586, 16108935.165405273 -4574828.552978516, 16108931.63659668 -4574826.51965332, 16108928.48626709 -4574824.66986084, 16108926.170837402 -4574822.537719727, 16108767.429199219 -4574634.343078613, 16108720.886535645 -4574672.326293945, 16108700.971496582 -4574691.049621582, 16108691.49822998 -4574703.743713379, 16108614.453979492 -4574877.084838867, 16108598.635498047 -4574956.893920898, 16108543.387634277 -4575025.957763672, 16108477.441955566 -4575078.543212891, 16108473.968811035 -4575102.703796387, 16108476.094970703 -4575113.266113281, 16108464.629089355 -4575127.457519531, 16108453.33013916 -4575139.544921875, 16108438.780700684 -4575152.041809082, 16108423.09576416 -4575170.285949707, 16108387.08392334 -4575209.2595825195, 16108369.784851074 -4575218.960632324, 16108363.773620605 -4575222.335510254, 16108346.21850586 -4575231.344665527, 16108299.542297363 -4575248.93939209, 16108283.356445312 -4575255.350280762, 16108271.58996582 -4575264.627746582, 16108251.964294434 -4575286.261108398, 16108107.171081543 -4575409.5947265625, 16108091.886901855 -4575435.394104004, 16107885.500549316 -4575540.498779297, 16107878.821411133 -4575584.444519043, 16107834.861328125 -4575638.910949707, 16107801.420959473 -4575649.417358398, 16107794.741760254 -4575656.110961914, 16107762.247619629 -4575657.057067871, 16107733.582824707 -4575677.123779297, 16107689.633911133 -4575695.284057617, 16107498.542907715 -4575773.63079834, 16107381.022888184 -4575820.444213867, 16107261.109558105 -4575862.3576049805, 16107170.128112793 -4575869.418518066, 16107115.069519043 -4575880.5888671875, 16107058.419006348 -4575890.685974121, 16107049.647033691 -4575890.445922852, 16107018.856079102 -4575893.933959961, 16106989.734863281 -4575894.032836914, 16106967.80493164 -4575893.044311523, 16106933.062133789 -4575890.219970703, 16106890.059387207 -4575884.317016602, 16106871.713928223 -4575881.916320801, 16106855.572631836 -4575876.677124023, 16106835.924743652 -4575869.9833984375, 16106810.710876465 -4575856.666564941, 16106793.823730469 -4575846.159912109, 16106774.420715332 -4575832.4900512695, 16106721.210021973 -4575801.436401367, 16106681.557983398 -4575772.148010254, 16106660.073364258 -4575756.543579102, 16106637.97644043 -4575739.357543945, 16106615.434204102 -4575715.647399902, 16106593.125793457 -4575679.863342285, 16106554.241882324 -4575724.685241699, 16106529.506713867 -4575749.878173828, 16106518.07421875 -4575760.737731934, 16106432.469482422 -4575832.885498047, 16106426.513916016 -4575846.145812988, 16106393.919555664 -4575922.799072266, 16106389.578125 -4575946.100158691, 16106353.3102417 -4576003.37878418, 16106314.315002441 -4576051.520690918, 16106272.180603027 -4576082.631652832, 16106175.065429688 -4576142.749511719, 16106015.50012207 -4576248.807556152, 16106001.173278809 -4576268.875427246, 16105931.420471191 -4576327.158508301, 16105910.403381348 -4576351.99987793, 16105898.959716797 -4576372.195007324, 16105894.718444824 -4576390.963806152, 16105893.237915039 -4576409.1818237305, 16105893.916931152 -4576423.841064453, 16105891.913208008 -4576435.478027344, 16105886.358337402 -4576444.742431641, 16105846.383544922 -4576479.074523926, 16105781.417480469 -4576539.265319824, 16105733.639160156 -4576574.614501953, 16105642.86920166 -4576620.4853515625, 16105628.520141602 -4576634.325744629, 16105622.809448242 -4576654.874511719, 16105622.096984863 -4576698.86730957, 16105618.890991211 -4576726.5341796875, 16105607.514160156 -4576756.16418457, 16104770.603088379 -4577462.875305176, 16104024.473083496 -4578157.967468262, 16102558.250671387 -4579552.557922363, 16102168.242858887 -4579630.3263549805, 16101753.666809082 -4579661.391479492, 16101467.442138672 -4579680.858459473, 16101248.543457031 -4579671.294494629, 16100913.037658691 -4579621.04498291, 16100647.340270996 -4579608.966430664, 16100548.521972656 -4579645.202026367, 16100531.645935059 -4579655.232116699, 16100513.389526367 -4579666.067504883, 16100460.657531738 -4579670.333862305, 16100296.260864258 -4579601.168395996, 16100103.822875977 -4579536.38269043, 16099846.897460938 -4579512.225952148, 16099726.126953125 -4579463.912475586, 16099368.190307617 -4579444.149291992, 16099076.143615723 -4579494.666381836, 16098922.422485352 -4579499.045654297, 16098746.760375977 -4579545.169616699, 16098553.520874023 -4579575.909545898, 16098137.041259766 -4579607.991699219, 16097458.159301758 -4579694.9572143555, 16097093.265197754 -4579769.491638184, 16096939.688781738 -4579838.502807617, 16096660.354797363 -4579946.548278809, 16096470.755432129 -4580019.897766113, 16096336.47076416 -4580055.527282715, 16096267.296813965 -4580061.559692383, 16096203.622070312 -4580059.369934082, 16096142.12915039 -4580057.717041016, 16096105.349243164 -4580061.559692383, 16096013.59967041 -4580084.093139648, 16095980.448730469 -4580095.338623047, 16095947.965698242 -4580107.332946777, 16095902.825683594 -4580120.951965332, 16095617.858886719 -4580271.849853516, 16095227.606140137 -4580513.042480469, 16094968.621398926 -4580733.584899902, 16094865.55065918 -4580983.788330078, 16094544.393920898 -4581281.245605469, 16094255.419677734 -4581641.503479004, 16094100.284790039 -4582062.095703125, 16094089.35321045 -4582181.101318359, 16094127.569213867 -4582219.917602539, 16094064.328613281 -4582307.526794434, 16094009.09185791 -4582353.32409668, 16093748.904846191 -4582388.325744629, 16093553.104980469 -4582425.644958496, 16093344.392089844 -4582532.007568359, 16093215.706726074 -4582638.470214844, 16093152.466125488 -4582756.535644531, 16093149.237854004 -4582870.532592773, 16093102.839904785 -4582955.815612793, 16093096.628295898 -4583073.870483398, 16093146.911315918 -4583103.363220215, 16093151.330688477 -4583124.631408691, 16093076.624145508 -4583151.9197387695, 16092907.262695312 -4583241.911010742, 16092796.310546875 -4583401.34777832, 16092703.09161377 -4583627.972106934, 16092719.934265137 -4583819.425231934, 16092694.442077637 -4583922.382019043, 16092591.838928223 -4584006.048461914, 16092597.694335938 -4584076.981689453, 16092543.893615723 -4584135.450073242, 16092424.403259277 -4584215.471801758, 16092250.032409668 -4584439.45690918, 16092240.291992188 -4584572.750671387, 16092247.104736328 -4584692.067810059, 16092195.151916504 -4584738.469238281, 16092104.315185547 -4584766.892578125, 16092036.29901123 -4584687.106811523, 16091925.046325684 -4584611.858581543, 16091778.327209473 -4584561.528564453, 16091645.879272461 -4584533.939697266, 16091488.807495117 -4584531.890319824, 16091364.274353027 -4584563.026733398, 16091321.817138672 -4584571.973327637, 16091250.516967773 -4584653.736877441, 16091176.790100098 -4584676.266174316, 16091087.600891113 -4584728.264587402, 16090977.483642578 -4584794.128662109, 16090822.760681152 -4584817.237731934, 16090637.102050781 -4584873.858703613, 16090608.893676758 -4584911.992553711, 16090524.24633789 -4584927.017150879, 16090453.936950684 -4584957.674194336, 16090398.49987793 -4584998.974365234, 16090377.11541748 -4585011.2146606445, 16090346.780822754 -4585049.504455566, 16090293.080322266 -4585046.041503906, 16090157.482055664 -4585083.016967773, 16089951.930603027 -4585186.820251465, 16089851.119689941 -4585265.22467041, 16089751.56665039 -4585361.510131836, 16089631.864807129 -4585629.649719238, 16089531.432373047 -4585711.252075195, 16089415.660095215 -4585786.169006348, 16089130.682189941 -4585842.710449219, 16088951.457824707 -4585920.455505371, 16088617.499328613 -4586109.873046875, 16088525.760925293 -4586127.330749512, 16088298.012390137 -4586170.657043457, 16087809.319824219 -4586364.320007324, 16087608.944763184 -4586361.492797852, 16087566.643310547 -4586295.053283691, 16087593.359985352 -4586272.435668945, 16087731.3961792 -4586245.577331543, 16087833.810119629 -4586275.262878418, 16087954.03515625 -4586210.237487793, 16087980.751831055 -4586228.614196777, 16087931.771240234 -4586234.2685546875, 16087912.846923828 -4586259.713256836, 16087950.69555664 -4586280.917236328, 16088373.709655762 -4586121.181640625, 16088549.594421387 -4586013.750244141, 16088737.724365234 -4586006.682434082, 16088915.835571289 -4585899.252258301, 16089208.605834961 -4585776.274291992, 16089338.849609375 -4585764.966064453, 16089483.564941406 -4585715.492614746, 16089589.318481445 -4585615.1329956055, 16089627.167114258 -4585538.803771973, 16089631.619873047 -4585444.0997924805, 16089754.071350098 -4585271.655944824, 16090050.181152344 -4585005.928405762, 16090024.577697754 -4584998.86126709, 16089895.44708252 -4585061.052185059, 16089597.110839844 -4585400.281799316, 16089451.282287598 -4585407.349182129, 16089392.282958984 -4585432.791870117, 16089457.961486816 -4585458.234619141, 16089565.94140625 -4585451.167175293, 16089492.47052002 -4585497.81237793, 16089342.189208984 -4585485.090942383, 16088854.609863281 -4585579.795288086, 16087891.696228027 -4585660.365356445, 16087600.03918457 -4585752.244262695, 16087413.022460938 -4585869.567749023, 16087355.136291504 -4585933.177429199, 16087241.590393066 -4586153.694030762, 16087198.220336914 -4586307.436401367, 16087120.252197266 -4586590.500305176, 16087051.234069824 -4586747.4158325195, 16087100.214660645 -4587035.807312012, 16087214.873718262 -4587180.0060424805, 16087152.534851074 -4587237.968811035, 16087057.913269043 -4587400.549194336, 16087017.838256836 -4587523.546447754, 16087000.02709961 -4587628.16595459, 16087003.366699219 -4587765.303894043, 16087036.762573242 -4587860.029174805, 16087111.346618652 -4587983.031860352, 16087160.327209473 -4588004.239379883, 16087199.289001465 -4588067.862182617, 16087277.212646484 -4588118.760681152, 16087376.286987305 -4588087.656005859, 16087406.343261719 -4588048.068359375, 16087420.814819336 -4587884.064025879, 16087510.983581543 -4587737.027893066, 16087594.473205566 -4587051.358032227, 16087616.737121582 -4586976.431945801, 16087581.114868164 -4586857.682189941, 16087575.54888916 -4586770.034484863, 16087506.530822754 -4586604.636779785, 16087516.549560547 -4586582.018493652, 16087601.152404785 -4586738.933837891, 16087657.925292969 -4586935.434814453, 16087545.615112305 -4587808.043273926, 16087487.840270996 -4588197.470092773, 16087481.172241211 -4588561.531860352, 16087352.297668457 -4588903.011352539, 16087276.756225586 -4589125.980834961, 16087281.197875977 -4589204.994873047, 16087312.178100586 -4589272.442871094, 16087197.574707031 -4589407.778564453, 16087164.256774902 -4589489.80657959, 16087108.975524902 -4589637.489868164, 16087062.956054688 -4589773.127258301, 16087015.634094238 -4590031.3693237305, 16086994.160583496 -4590212.023803711, 16086979.90057373 -4590689.504211426, 16086984.175231934 -4590886.815673828, 16087003.68951416 -4591050.811035156, 16087029.270751953 -4591178.278381348, 16087032.977722168 -4591231.879333496, 16087029.170593262 -4591295.069396973, 16086986.256896973 -4591339.944763184, 16086932.901489258 -4591466.326660156, 16086906.273864746 -4591540.549865723, 16086874.60345459 -4591685.84387207, 16086851.081665039 -4591800.462524414, 16086848.565856934 -4591836.061889648, 16086854.410095215 -4591866.895019531, 16086867.30090332 -4591872.425170898, 16086931.754882812 -4591901.688415527, 16086924.229675293 -4591924.714355469, 16086786.004272461 -4591874.476013184, 16086761.792297363 -4591877.163269043, 16086732.949401855 -4591889.850158691, 16086710.38494873 -4591916.213989258, 16086594.256469727 -4592111.045349121, 16086514.16204834 -4592330.746032715, 16086417.870727539 -4592410.364440918, 16086337.609375 -4592290.279296875, 16086256.602172852 -4592133.010986328, 16086032.716430664 -4591924.869934082, 16085813.383605957 -4591832.356323242, 16085480.282287598 -4591811.551086426, 16085245.476135254 -4591544.439208984, 16085177.226135254 -4591488.941467285, 16084989.741821289 -4591475.067077637, 16084956.067687988 -4591344.399780273, 16084860.499938965 -4591249.58605957, 16084797.704589844 -4591147.843322754, 16084659.390136719 -4591126.332336426, 16084343.03125 -4590566.228881836, 16084328.581970215 -4590540.6463012695, 16084206.141662598 -4590496.8916015625, 16084076.499023438 -4590491.036865234, 16084029.021240234 -4590502.619018555, 16084008.783325195 -4590535.59765625, 16084000.579101562 -4590593.5509643555, 16084001.948364258 -4590719.9802856445, 16083984.07043457 -4590718.877258301, 16083995.870300293 -4590676.479370117, 16083986.029663086 -4590591.995361328, 16083995.859191895 -4590515.417297363, 16083945.76538086 -4590486.667114258, 16083930.692749023 -4590486.28527832, 16083855.106811523 -4590484.404418945, 16083731.063476562 -4590523.492248535, 16083758.782043457 -4590602.219970703, 16083770.114379883 -4590650.344970703, 16083745.768798828 -4590670.483215332, 16083691.344665527 -4590549.994018555, 16083632.234069824 -4590527.437805176, 16083617.539855957 -4590614.975952148, 16083593.684082031 -4590638.734436035, 16083622.693969727 -4590516.774902344, 16083551.494018555 -4590467.448486328, 16083532.603088379 -4590460.759460449, 16083512.187133789 -4590472.398071289, 16083507.222229004 -4590465.129211426, 16083518.866271973 -4590444.510620117, 16083463.896728516 -4590391.903625488, 16083433.172546387 -4590364.737609863, 16083408.481872559 -4590347.895019531, 16083360.069030762 -4590333.216125488, 16083286.965515137 -4590333.71105957, 16083146.079589844 -4590351.7556762695, 16083080.312011719 -4590360.509277344, 16083066.374816895 -4590386.459106445, 16083035.83984375 -4590411.192871094, 16082933.537231445 -4590498.235046387, 16082898.41595459 -4590537.026000977, 16082895.165405273 -4590577.768615723, 16082853.754577637 -4590588.1912231445, 16082822.262268066 -4590604.199829102, 16082792.105834961 -4590645.409423828, 16082737.392333984 -4590727.518005371, 16082590.940368652 -4590951.177429199, 16082573.563415527 -4591055.930603027, 16082492.166625977 -4591168.463317871, 16082460.674316406 -4591212.107727051, 16082447.226928711 -4591237.692016602, 16082422.11328125 -4591237.564697266, 16082395.964294434 -4591214.045288086, 16082377.262634277 -4591146.881591797, 16082360.275268555 -4591114.876831055, 16082332.979736328 -4591077.285827637, 16082295.687683105 -4591038.733337402, 16082210.339050293 -4590966.635070801, 16082168.649902344 -4590926.951538086, 16082127.762268066 -4590913.0779418945, 16082087.286499023 -4591015.93560791, 16082048.235595703 -4591025.566650391, 16081990.516479492 -4590961.75592041, 16081961.117004395 -4590896.389953613, 16081951.643676758 -4590863.0283203125, 16081957.5993042 -4590834.998413086, 16082044.973937988 -4590752.125305176, 16082053.712524414 -4590718.085266113, 16082050.21710205 -4590657.416015625, 16082034.487670898 -4590588.601318359, 16081986.720458984 -4590453.1795043945, 16081905.601928711 -4590351.133422852, 16081791.020812988 -4590297.791625977, 16081724.618713379 -4590277.07434082, 16081664.172241211 -4590270.668273926, 16081593.551147461 -4590280.058166504, 16081552.763671875 -4590317.038208008, 16081543.69116211 -4590341.587890625, 16081531.768859863 -4590363.705322266, 16081510.918701172 -4590375.258972168, 16081454.947265625 -4590342.492980957, 16081411.04284668 -4590329.4685668945, 16081320.150512695 -4590294.906738281, 16081240.82421875 -4590290.522888184, 16081168.878417969 -4590305.201721191, 16081075.303283691 -4590343.553588867, 16081023.15008545 -4590376.248901367, 16080978.38848877 -4590416.962646484, 16080920.602539062 -4590482.792236328, 16080891.525939941 -4590543.403930664, 16080873.258361816 -4590648.534851074, 16080851.851623535 -4590776.718444824, 16080846.942443848 -4590843.186767578, 16080878.256652832 -4590891.680603027, 16080914.936401367 -4590926.909118652, 16080962.647949219 -4590977.680297852, 16080984.600158691 -4591042.396240234, 16080960.56628418 -4591062.549316406, 16080919.411437988 -4591052.324279785, 16080886.305053711 -4591055.7326049805, 16080857.384216309 -4591042.085083008, 16080871.688781738 -4591083.381286621, 16080868.82788086 -4591121.50970459, 16080829.064575195 -4591147.305908203, 16080751.908996582 -4591064.543395996, 16080677.614379883 -4590998.38482666, 16080599.390197754 -4590993.010681152, 16080537.28503418 -4591005.540893555, 16080492.178405762 -4591034.06628418, 16080355.043884277 -4591232.430908203, 16080183.600769043 -4591323.128845215, 16080139.518249512 -4591491.147827148, 16079943.985534668 -4591619.554016113, 16079861.16381836 -4591702.815979004, 16079838.41015625 -4591758.314880371, 16079862.076660156 -4591806.883728027, 16079856.622009277 -4591949.1123046875, 16079793.815551758 -4591989.577636719, 16079745.580810547 -4591973.397155762, 16079725.565551758 -4591982.647155762, 16079714.645141602 -4591996.522277832, 16079697.346069336 -4592058.95324707, 16079680.035888672 -4592075.841064453, 16079668.403015137 -4592083.040344238, 16079629.641540527 -4592098.952270508, 16079621.58203125 -4592110.3240356445, 16079617.407531738 -4592131.921875, 16079611.440856934 -4592204.664733887, 16079625.422546387 -4592252.288024902, 16079624.95501709 -4592291.509887695, 16079612.24230957 -4592351.325927734, 16079570.041137695 -4592396.559509277, 16079533.494934082 -4592435.102966309, 16079499.130615234 -4592390.845214844, 16079455.949768066 -4592348.412231445, 16079399.410583496 -4592320.5197143555, 16079339.053161621 -4592270.802734375, 16079293.712768555 -4592255.65435791, 16079253.715637207 -4592259.317687988, 16079169.057189941 -4592272.669799805, 16079157.346374512 -4592214.565612793, 16079152.125488281 -4592132.6290893555, 16079146.537231445 -4592063.479309082, 16079173.376342773 -4592027.483093262, 16079241.214477539 -4592003.806335449, 16079287.44543457 -4591976.324890137, 16079351.565490723 -4591967.8103637695, 16079372.67163086 -4591898.590942383, 16079426.861999512 -4591746.618225098, 16079427.240478516 -4591703.523132324, 16079408.004455566 -4591649.566162109, 16079373.606750488 -4591607.461486816, 16079293.545776367 -4591592.186767578, 16079224.672363281 -4591578.128356934, 16079164.481933594 -4591566.0076293945, 16079120.276977539 -4591569.868713379, 16079049.845153809 -4591595.538757324, 16078958.273742676 -4591612.4541015625, 16078844.293701172 -4591609.752746582, 16078287.618286133 -4591495.602905273, 16078224.310913086 -4591477.344116211, 16078194.098815918 -4591484.89654541, 16078181.875915527 -4591492.98638916, 16078076.8460083 -4591494.429016113, 16077668.926818848 -4591447.813354492, 16077601.611938477 -4591442.382446289, 16077553.3883667 -4591436.71105957, 16077509.795654297 -4591433.274291992, 16077479.038085938 -4591433.726867676, 16077441.868469238 -4591440.741821289, 16077414.851257324 -4591447.756774902, 16077394.991821289 -4591454.474731445, 16077379.7633667 -4591462.394897461, 16077373.340209961 -4591471.503051758, 16077367.67401123 -4591488.517211914, 16077359.614501953 -4591496.578735352, 16077348.25994873 -4591500.015563965, 16077319.750976562 -4591487.626159668, 16077277.34942627 -4591475.533813477, 16077139.558166504 -4591438.35168457, 16077101.932128906 -4591430.742675781, 16077067.890625 -4591431.039733887, 16077027.737731934 -4591442.382446289, 16076845.95300293 -4591514.059692383, 16076367.512939453 -4591361.7107543945, 16076038.830993652 -4591368.357910156, 16075946.51373291 -4591330.58215332, 16075817.783874512 -4591259.542602539, 16075760.699279785 -4591207.002197266, 16075650.61541748 -4591147.065490723, 16075609.260253906 -4591127.08190918, 16075562.083007812 -4591094.525634766, 16075526.549865723 -4591050.867614746, 16075492.764404297 -4591027.928466797, 16075402.48425293 -4590977.609619141, 16075298.511901855 -4590938.378601074, 16075195.240783691 -4590780.607543945, 16075125.933288574 -4590628.919921875, 16074969.551635742 -4590405.918029785, 16074805.05480957 -4590231.397521973, 16074710.366455078 -4590048.508361816, 16074785.028442383 -4589976.261474609, 16074814.606018066 -4589868.2951049805, 16074853.078063965 -4589910.237121582, 16074855.25994873 -4589974.041381836, 16074808.661560059 -4590007.3435668945, 16074805.02142334 -4590079.491699219, 16074862.540222168 -4590172.908813477, 16074965.199035645 -4590242.272277832, 16075016.895812988 -4590315.341186523, 16075056.937438965 -4590351.416259766, 16075292.834594727 -4590197.868286133, 16075657.606262207 -4590176.599731445, 16075807.486877441 -4590119.072998047, 16075894.23815918 -4589963.873962402, 16075903.700256348 -4589785.83984375, 16075907.763427734 -4589417.280822754, 16076052.0446167 -4589133.814331055, 16076200.38897705 -4588637.897521973, 16076286.850830078 -4588528.078735352, 16076314.157531738 -4588315.3869018555, 16076393.428100586 -4588135.345214844, 16076485.689697266 -4588035.329650879, 16076531.686950684 -4587979.115539551, 16076541.761352539 -4587909.258361816, 16076507.252319336 -4587899.729187012, 16076472.387023926 -4587890.101074219, 16076386.604248047 -4587979.780029297, 16076306.8883667 -4588014.942138672, 16076264.019226074 -4588021.572998047, 16076199.220153809 -4587988.361999512, 16076171.93572998 -4587964.284484863, 16076127.207580566 -4587953.864562988, 16076055.573486328 -4588000.88861084, 16075984.807678223 -4588052.818908691, 16075946.035095215 -4588074.592102051, 16075898.100891113 -4588146.189453125, 16075868.066894531 -4588170.196716309, 16075845.535827637 -4588198.643615723, 16075821.156860352 -4588250.730407715, 16075788.351013184 -4588320.3212890625, 16075773.434204102 -4588392.768737793, 16075745.103393555 -4588434.902709961, 16075708.93572998 -4588506.757080078, 16075577.946044922 -4588562.196411133, 16075549.960327148 -4588602.719177246, 16075531.347717285 -4588687.964599609, 16075315.10961914 -4588838.704711914, 16075254.240112305 -4588895.602172852, 16075209.934936523 -4588960.941345215, 16075196.576599121 -4588991.299255371, 16075166.809814453 -4589007.248901367, 16075147.106262207 -4589044.3798828125, 16075157.514587402 -4589102.353149414, 16075144.479064941 -4589150.04699707, 16075151.024658203 -4589262.771057129, 16075179.633789062 -4589349.902587891, 16075243.186096191 -4589412.968017578, 16075291.342895508 -4589436.4267578125, 16075341.71496582 -4589446.720947266, 16075368.409362793 -4589444.260498047, 16075350.999023438 -4589459.730041504, 16075327.07647705 -4589475.199584961))"

The coastline above is actually copied from OSM with a rather high granularity. We use ST_Simplify to reduce the number of points.

In [ ]:
%sql CREATE TABLE MELB_AREA_OF_INTEREST (NAME VARCHAR(100), SHAPE ST_GEOMETRY($srid))
%sql INSERT INTO MELB_AREA_OF_INTEREST VALUES ('CBD', ST_GeomFromText(:cbd_poly, 4326).ST_Transform(:srid))
%sql INSERT INTO MELB_AREA_OF_INTEREST VALUES ('BAY', ST_GeomFromText(:bay_poly, 3857).ST_Simplify(100).ST_Transform(:srid))

In [ ]:
df_si_areas = %sql SELECT NAME, SHAPE.ST_TRANSFORM(4326).ST_ASWKT() AS SHAPE FROM MELB_AREA_OF_INTEREST
df2map(df_si_areas, 'shape', location=[-37.817301, 144.9665769], zoom_start=11)

## Train a Regression Model for Housing Prices (including geo-spatial features)

Retrieve dataset containing the following types of features:
* Features describing the house (H.*). Those features have also been used by the other model
* Distance to areas of interest (CBD and Bay). Note that we compute the distance to the polygon rather than to a point
* Distance to the closest POI of different types
* Number of POIs of different types within a given range

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_HOUSING_PREDICT_GEO_VIEW AS 
(
    SELECT 
        H.*, 
        
        FLOOR(H.SHAPE.ST_DISTANCE(CBD.SHAPE)/250)*250 AS DIST_CBD,
        FLOOR(H.SHAPE.ST_DISTANCE(BAY.SHAPE)/250)*250 AS DIST_BAY,

        R.CLUSTER_NAME AS REGION,

        poi1.DIST_POI AS DIST_ENTERTAINMENT, 
        poi2.DIST_POI AS DIST_HEALTH,
        poi3.DIST_POI AS DIST_EDUCATION,
        poi4.DIST_POI AS DIST_CHILDCARE,
        poi6.DIST_POI AS DIST_COMMUNITY,
        poi7.DIST_POI AS DIST_EMERGENCY,
        poi8.DIST_POI AS DIST_GROCERY,
        poi9.DIST_POI AS DIST_SHOPPING,
        poi10.DIST_POI AS DIST_AIRPORT,

        IFNULL(cnt10.CNT_POI,0) AS CNT_AIRPORT,
        IFNULL(cnt0.CNT_POI,0) AS CNT_POI,
        IFNULL(cnt1.CNT_POI,0) AS CNT_ENTERTAINMENT,
        IFNULL(cnt2.CNT_POI,0) AS CNT_HEALTH,
        IFNULL(cnt3.CNT_POI,0) AS CNT_EDUCATION,
        IFNULL(cnt4.CNT_POI,0) AS CNT_CHILDCARE,
        IFNULL(cnt6.CNT_POI,0) AS CNT_COMMUNITY,
        IFNULL(cnt7.CNT_POI,0) AS CNT_EMERGENCY,
        IFNULL(cnt8.CNT_POI,0) AS CNT_GROCERY,
        IFNULL(cnt9.CNT_POI,0) AS CNT_SHOPPING

    FROM MELB_HOUSING_PREDICT_VIEW H

    LEFT JOIN MELB_AREA_OF_INTEREST CBD ON CBD.NAME = 'CBD'
    LEFT JOIN MELB_AREA_OF_INTEREST BAY ON BAY.NAME = 'BAY'

    LEFT JOIN MELB_REGION_PROXY R ON H.SHAPE.ST_INTERSECTS(R.CLUSTER_CELL) = 1

    LEFT JOIN MELB_HOUSING_CLOSEST_POI('entertainment') poi1 ON H.ID = poi1.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('health') poi2 ON H.ID = poi2.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('education') poi3 ON H.ID = poi3.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('childcare') poi4 ON H.ID = poi4.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('community') poi6 ON H.ID = poi6.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('emergency') poi7 ON H.ID = poi7.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('grocery') poi8 ON H.ID = poi8.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('shopping') poi9 ON H.ID = poi9.ID
    LEFT JOIN MELB_HOUSING_CLOSEST_POI('airport') poi10 ON H.ID = poi10.ID

    LEFT JOIN MELB_HOUSING_COUNT_POI(null, 1000) cnt0 ON H.ID = cnt0.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('entertainment', 1000) cnt1 ON H.ID = cnt1.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('health', 1000) cnt2 ON H.ID = cnt2.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('education', 1000) cnt3 ON H.ID = cnt3.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('childcare', 1000) cnt4 ON H.ID = cnt4.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('community', 1000) cnt6 ON H.ID = cnt6.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('emergency', 1000) cnt7 ON H.ID = cnt7.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('grocery', 1000) cnt8 ON H.ID = cnt8.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('shopping', 1000) cnt9 ON H.ID = cnt9.ID
    LEFT JOIN MELB_HOUSING_COUNT_POI('airport', 1000) cnt10 ON H.ID = cnt10.ID
)

In [ ]:
%%time
df_housing_geo = %sql SELECT * FROM MELB_HOUSING_PREDICT_GEO_VIEW

Generate numerical features for xgboost. Note that this time we include **geo-spatial features** describing the location of the house.

In [ ]:
x_housing_geo = pd.get_dummies(
    df_housing_geo[[
        'rooms',
        'bedroom2',
        'bathroom',
        'car',
        'landsize',
        'buildingarea',
        'yearbuilt',
        'dist_cbd',
        'dist_bay',
        'region',
        'cnt_poi',
        'dist_entertainment',
        'cnt_entertainment',
        'dist_health',
        'cnt_health',
        'dist_education',
        'cnt_education',
        'dist_childcare',
        'cnt_childcare',
        'dist_community',
        'cnt_community',
        'dist_emergency',
        'cnt_emergency',
        'dist_grocery',
        'cnt_grocery',
        'dist_shopping',
        'cnt_shopping',
        'dist_airport',
        'cnt_airport',
        'type'
]])
y_housing_geo = df_housing_geo[['price']]

Again, use the same training and test split as before. We want to ensure maximum comparability between both models.

In [ ]:
x_train_geo = x_housing_geo[df_housing_geo["year"] <= 2017]
y_train_geo = y_housing_geo[df_housing_geo["year"] <= 2017]
x_test_geo = x_housing_geo[df_housing_geo["year"] == 2018]
y_test_geo = y_housing_geo[df_housing_geo["year"] == 2018]

Train the regression model

In [ ]:
model_geo = train_model(x_train_geo, y_train_geo)

Calculate the mean absolute error on training and test dataset

In [ ]:
mae_train_geo = mae_predict(model_geo,x_train_geo,y_train_geo)
print('MAE on train: ' + str(mae_train_geo))
mae_test_geo = mae_predict(model_geo,x_test_geo,y_test_geo)
print('MAE on test: ' + str(mae_test_geo))

Visually explain the model to understand the impact of geo-spatial features

In [ ]:
explain_model(model_geo, x_train_geo)

**Comparison of models with and without geo dimension**

Determine the median housing price to evaluate the quality of prediction using mae

In [ ]:
df_med_price = %sql SELECT MEDIAN(PRICE) AS p FROM MELB_HOUSING_PREDICT_VIEW
median_price = df_med_price['p'][0]

Compare quality of prediction between non-spatial and spatial model

In [ ]:
print(str(round(mae_test_nogeo,2)) + ' AUD - mae on test data without spatial features')
print(str(round(mae_test_geo,2)) + ' AUD - mae on test data with spatial features')

print('===')
err_ratio_nogeo = 100 * mae_test_nogeo / median_price
err_ratio_geo = 100 * mae_test_geo / median_price
print(str(round(err_ratio_nogeo,2)) + '% - error ratio without spatial features')
print(str(round(err_ratio_geo,2)) + '% - error ratio with spatial features')

print('===')
print(str(round(err_ratio_nogeo - err_ratio_geo,2)) + '% - decrease in error ratio')

Compare robustness of prediction between non-spatial and spatial model

In [ ]:
mae_dev_nogeo = abs(100*(1-mae_train_nogeo/mae_test_nogeo))
print(str(round(mae_dev_nogeo,2)) + '% - MAE Deviation Train/Test without spatial features')

mae_dev_geo = abs(100*(1-mae_train_geo/mae_test_geo))
print(str(round(mae_dev_geo,2)) + '% - MAE Deviation Train/Test with spatial features')

print('===')
print(str(round(mae_dev_nogeo - mae_dev_geo,2)) + '% - increase in prediction robustness')

**Prediction with reference house**

So far, we predicted the price of a house based on the attributes of the house as well as the geo location. To isolate the impact of the geo dimension, we will now predict a "standard" reference house for different locations in the city. We are spanning an equidistant (hexagonal) grid and predict for each grid cell's centroid. The below feature values are more or less the average values from the dataset.

In [ ]:
%%sql
CREATE OR REPLACE VIEW MELB_HOUSING_PREDICT_VIEW AS
(
    SELECT
        ST_ClusterId() AS ID,
        3 AS ROOMS,
        'h' as TYPE,
        3 as BEDROOM2,
        2 as BATHROOM,
        2 as CAR,
        600 as LANDSIZE,
        160 as BUILDINGAREA,
        1965 as YEARBUILT,
        ST_ClusterCell().ST_Centroid() AS SHAPE,
        ST_ClusterCell().ST_Transform(4326).ST_ASWKT() AS SHAPE_CELL_WKT,
        2020 as YEAR,
        CAST(0 as DOUBLE) as PRICE    
    FROM MELB_HOUSING
    GROUP CLUSTER BY SHAPE USING HEXAGON X CELLS 75
)

Prepare the geo-spatial features similar to above.

In [ ]:
%%time
df_housing_ref_geo = %sql SELECT * FROM MELB_HOUSING_PREDICT_GEO_VIEW

In [ ]:
x_housing_ref_geo = pd.get_dummies(
    df_housing_ref_geo[[
        'rooms',
        'bedroom2',
        'bathroom',
        'car',
        'landsize',
        'buildingarea',
        'yearbuilt',
        'dist_cbd',
        'dist_bay',
        'region',
        'cnt_poi',
        'dist_entertainment',
        'cnt_entertainment',
        'dist_health',
        'cnt_health',
        'dist_education',
        'cnt_education',
        'dist_childcare',
        'cnt_childcare',
        'dist_community',
        'cnt_community',
        'dist_emergency',
        'cnt_emergency',
        'dist_grocery',
        'cnt_grocery',
        'dist_shopping',
        'cnt_shopping',
        'dist_airport',
        'cnt_airport',
        'type'
]])
x_housing_ref_geo['type_t'] = 0
x_housing_ref_geo['type_u'] = 0

Predict prices for our standard house and different locations

In [ ]:
df_housing_ref_geo['price'] = model_geo.predict(x_housing_ref_geo)

Determine cell color based on predicted price

In [ ]:
df_housing_ref_vis = df_housing_ref_geo.copy()[[
    'id','price','shape_cell_wkt',
    'dist_cbd','dist_bay','region',
    'cnt_poi','dist_health','dist_community'
]]


min_price = df_housing_ref_vis['price'].min()
print('minimum predicted price: ' + str(min_price) + ' AUD')
max_price = df_housing_ref_vis['price'].max()
print('maximum predicted price: ' + str(max_price) + ' AUD')

def price_to_rgb(price):
    ratio = (price-min_price)/(max_price-min_price)
    margin = 0.1
    ratio = min(1,max(0,ratio-margin)/(1-2*margin))
    return '#%02x%02x%02x' % (int((ratio**(1/float(5)))*255), int((1-ratio)*255), int((1-ratio)*255))

df_housing_ref_vis['color'] = df_housing_ref_vis['price'].apply(price_to_rgb)

In [ ]:
df2map(
    df_housing_ref_vis, 
    'shape_cell_wkt', 
    location=[-37.817301, 144.9665769], 
    zoom_start=10,
    style_function = lambda feature: {
        'fillColor': feature['properties']['color'],
        'color' : feature['properties']['color'],
        'weight' : 0.5,
        'fillOpacity' : 0.7,
    },
    tiles='Stamen Toner'
)

Example: Cell ID 272 - Frankston East. Far away from CBD, but still 1.9m AUD predicted reference price  
https://www.google.com/maps/@-38.1591711,145.1008108,876m/data=!3m1!1e3